In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW23OT-MakeConcordance-Isaiah.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

Isaiah01 = [
['Isaiah','Isa','01','001','The',"vision of Isaiah the son of Amoz, which he saw concerning Judah and Jerusalem in the days of Uzziah, Jotham, Ahaz, \emph{and} Hezekiah, kings of Judah."],
['Isaiah','Isa','01','002','2',"Hear, O heavens, and give ear, O earth: for the LORD hath spoken, I have nourished and brought up children, and they have rebelled against me."],
['Isaiah','Isa','01','003','3',"The ox knoweth his owner, and the ass his master's crib: \emph{but} Israel doth not know, my people doth not consider."],
['Isaiah','Isa','01','004','4',"Ah sinful nation, a people laden with iniquity, a seed of evildoers, children that are corrupters: they have forsaken the LORD, they have provoked the Holy One of Israel unto anger, they are gone away backward."],
['Isaiah','Isa','01','005','5',"Why should ye be stricken any more? ye will revolt more and more: the whole head is sick, and the whole heart faint."],
['Isaiah','Isa','01','006','6',"From the sole of the foot even unto the head \emph{there} \emph{is} no soundness in it; \emph{but} wounds, and bruises, and putrifying sores: they have not been closed, neither bound up, neither mollified with ointment."],
['Isaiah','Isa','01','007','7',"Your country \emph{is} desolate, your cities \emph{are} burned with fire: your land, strangers devour it in your presence, and \emph{it} \emph{is} desolate, as overthrown by strangers."],
['Isaiah','Isa','01','008','8',"And the daughter of Zion is left as a cottage in a vineyard, as a lodge in a garden of cucumbers, as a besieged city."],
['Isaiah','Isa','01','009','9',"Except the LORD of hosts had left unto us a very small remnant, we should have been as Sodom, \emph{and} we should have been like unto Gomorrah."],
['Isaiah','Isa','01','010','10',"Hear the word of the LORD, ye rulers of Sodom; give ear unto the law of our God, ye people of Gomorrah."],
['Isaiah','Isa','01','011','11',"To what purpose \emph{is} the multitude of your sacrifices unto me? saith the LORD: I am full of the burnt offerings of rams, and the fat of fed beasts; and I delight not in the blood of bullocks, or of lambs, or of he goats."],
['Isaiah','Isa','01','012','12',"When ye come to appear before me, who hath required this at your hand, to tread my courts?"],
['Isaiah','Isa','01','013','13',"Bring no more vain oblations; incense is an abomination unto me; the new moons and sabbaths, the calling of assemblies, I cannot away with; \emph{it} \emph{is} iniquity, even the solemn meeting."],
['Isaiah','Isa','01','014','14',"Your new moons and your appointed feasts my soul hateth: they are a trouble unto me; I am weary to bear \emph{them}."],
['Isaiah','Isa','01','015','15',"And when ye spread forth your hands, I will hide mine eyes from you: yea, when ye make many prayers, I will not hear: your hands are full of blood."],
['Isaiah','Isa','01','016','16',"Wash you, make you clean; put away the evil of your doings from before mine eyes; cease to do evil;"],
['Isaiah','Isa','01','017','17',"Learn to do well; seek judgment, relieve the oppressed, judge the fatherless, plead for the widow."],
['Isaiah','Isa','01','018','18',"Come now, and let us reason together, saith the LORD: though your sins be as scarlet, they shall be as white as snow; though they be red like crimson, they shall be as wool."],
['Isaiah','Isa','01','019','19',"If ye be willing and obedient, ye shall eat the good of the land:"],
['Isaiah','Isa','01','020','20',"But if ye refuse and rebel, ye shall be devoured with the sword: for the mouth of the LORD hath spoken \emph{it}."],
['Isaiah','Isa','01','021','21',"How is the faithful city become an harlot! it was full of judgment; righteousness lodged in it; but now murderers."],
['Isaiah','Isa','01','022','22',"Thy silver is become dross, thy wine mixed with water:"],
['Isaiah','Isa','01','023','23',"Thy princes \emph{are} rebellious, and companions of thieves: every one loveth gifts, and followeth after rewards: they judge not the fatherless, neither doth the cause of the widow come unto them."],
['Isaiah','Isa','01','024','24',"Therefore saith the Lord, the LORD of hosts, the mighty One of Israel, Ah, I will ease me of mine adversaries, and avenge me of mine enemies:"],
['Isaiah','Isa','01','025','25',"And I will turn my hand upon thee, and purely purge away thy dross, and take away all thy tin:"],
['Isaiah','Isa','01','026','26',"And I will restore thy judges as at the first, and thy counsellors as at the beginning: afterward thou shalt be called, The city of righteousness, the faithful city."],
['Isaiah','Isa','01','027','27',"Zion shall be redeemed with judgment, and her converts with righteousness."],
['Isaiah','Isa','01','028','28',"And the destruction of the transgressors and of the sinners \emph{shall} \emph{be} together, and they that forsake the LORD shall be consumed."],
['Isaiah','Isa','01','029','29',"For they shall be ashamed of the oaks which ye have desired, and ye shall be confounded for the gardens that ye have chosen."],
['Isaiah','Isa','01','030','30',"For ye shall be as an oak whose leaf fadeth, and as a garden that hath no water."],
['Isaiah','Isa','01','031','31',"And the strong shall be as tow, and the maker of it as a spark, and they shall both burn together, and none shall quench \emph{them}."]
]

Isaiah02 = [
['Isaiah','Isa','02','001','1',"The word that Isaiah the son of Amoz saw concerning Judah and Jerusalem."],
['Isaiah','Isa','02','002','2',"And it shall come to pass in the last days, \emph{that} the mountain of the LORD'S house shall be established in the top of the mountains, and shall be exalted above the hills; and all nations shall flow unto it."],
['Isaiah','Isa','02','003','3',"And many people shall go and say, Come ye, and let us go up to the mountain of the LORD, to the house of the God of Jacob; and he will teach us of his ways, and we will walk in his paths: for out of Zion shall go forth the law, and the word of the LORD from Jerusalem."],
['Isaiah','Isa','02','004','4',"And he shall judge among the nations, and shall rebuke many people: and they shall beat their swords into plowshares, and their spears into pruninghooks: nation shall not lift up sword against nation, neither shall they learn war any more."],
['Isaiah','Isa','02','005','5',"O house of Jacob, come ye, and let us walk in the light of the LORD."],
['Isaiah','Isa','02','006','6',"Therefore thou hast forsaken thy people the house of Jacob, because they be replenished from the east, and \emph{are} soothsayers like the Philistines, and they please themselves in the children of strangers."],
['Isaiah','Isa','02','007','7',"Their land also is full of silver and gold, neither \emph{is} \emph{there} \emph{any} end of their treasures; their land is also full of horses, neither \emph{is} \emph{there} \emph{any} end of their chariots:"],
['Isaiah','Isa','02','008','8',"Their land also is full of idols; they worship the work of their own hands, that which their own fingers have made:"],
['Isaiah','Isa','02','009','9',"And the mean man boweth down, and the great man humbleth himself: therefore forgive them not."],
['Isaiah','Isa','02','010','10',"Enter into the rock, and hide thee in the dust, for fear of the LORD, and for the glory of his majesty."],
['Isaiah','Isa','02','011','11',"The lofty looks of man shall be humbled, and the haughtiness of men shall be bowed down, and the LORD alone shall be exalted in that day."],
['Isaiah','Isa','02','012','12',"For the day of the LORD of hosts \emph{shall} \emph{be} upon every \emph{one} \emph{that} \emph{is} proud and lofty, and upon every \emph{one} \emph{that} \emph{is} lifted up; and he shall be brought low:"],
['Isaiah','Isa','02','013','13',"And upon all the cedars of Lebanon, \emph{that} \emph{are} high and lifted up, and upon all the oaks of Bashan,"],
['Isaiah','Isa','02','014','14',"And upon all the high mountains, and upon all the hills \emph{that} \emph{are} lifted up,"],
['Isaiah','Isa','02','015','15',"And upon every high tower, and upon every fenced wall,"],
['Isaiah','Isa','02','016','16',"And upon all the ships of Tarshish, and upon all pleasant pictures."],
['Isaiah','Isa','02','017','17',"And the loftiness of man shall be bowed down, and the haughtiness of men shall be made low: and the LORD alone shall be exalted in that day."],
['Isaiah','Isa','02','018','18',"And the idols he shall utterly abolish."],
['Isaiah','Isa','02','019','19',"And they shall go into the holes of the rocks, and into the caves of the earth, for fear of the LORD, and for the glory of his majesty, when he ariseth to shake terribly the earth."],
['Isaiah','Isa','02','020','20',"In that day a man shall cast his idols of silver, and his idols of gold, which they made \emph{each} \emph{one} for himself to worship, to the moles and to the bats;"],
['Isaiah','Isa','02','021','21',"To go into the clefts of the rocks, and into the tops of the ragged rocks, for fear of the LORD, and for the glory of his majesty, when he ariseth to shake terribly the earth."],
['Isaiah','Isa','02','022','22',"Cease ye from man, whose breath \emph{is} in his nostrils: for wherein is he to be accounted of?"]
]

Isaiah03 = [
['Isaiah','Isa','03','001','1',"For, behold, the Lord, the LORD of hosts, doth take away from Jerusalem and from Judah the stay and the staff, the whole stay of bread, and the whole stay of water,"],
['Isaiah','Isa','03','002','2',"The mighty man, and the man of war, the judge, and the prophet, and the prudent, and the ancient,"],
['Isaiah','Isa','03','003','3',"The captain of fifty, and the honourable man, and the counsellor, and the cunning artificer, and the eloquent orator."],
['Isaiah','Isa','03','004','4',"And I will give children \emph{to} \emph{be} their princes, and babes shall rule over them."],
['Isaiah','Isa','03','005','5',"And the people shall be oppressed, every one by another, and every one by his neighbour: the child shall behave himself proudly against the ancient, and the base against the honourable."],
['Isaiah','Isa','03','006','6',"When a man shall take hold of his brother of the house of his father, \emph{saying}, Thou hast clothing, be thou our ruler, and \emph{let} this ruin \emph{be} under thy hand:"],
['Isaiah','Isa','03','007','7',"In that day shall he swear, saying, I will not be an healer; for in my house \emph{is} neither bread nor clothing: make me not a ruler of the people."],
['Isaiah','Isa','03','008','8',"For Jerusalem is ruined, and Judah is fallen: because their tongue and their doings \emph{are} against the LORD, to provoke the eyes of his glory."],
['Isaiah','Isa','03','009','9',"The shew of their countenance doth witness against them; and they declare their sin as Sodom, they hide \emph{it} not. Woe unto their soul! for they have rewarded evil unto themselves."],
['Isaiah','Isa','03','010','10',"Say ye to the righteous, that \emph{it} \emph{shall} \emph{be} well \emph{with} \emph{him}: for they shall eat the fruit of their doings."],
['Isaiah','Isa','03','011','11',"Woe unto the wicked! \emph{it} \emph{shall} \emph{be} ill \emph{with} \emph{him}: for the reward of his hands shall be given him."],
['Isaiah','Isa','03','012','12',"\emph{As} \emph{for} my people, children \emph{are} their oppressors, and women rule over them. O my people, they which lead thee cause \emph{thee} to err, and destroy the way of thy paths."],
['Isaiah','Isa','03','013','13',"The LORD standeth up to plead, and standeth to judge the people."],
['Isaiah','Isa','03','014','14',"The LORD will enter into judgment with the ancients of his people, and the princes thereof: for ye have eaten up the vineyard; the spoil of the poor \emph{is} in your houses."],
['Isaiah','Isa','03','015','15',"What mean ye \emph{that} ye beat my people to pieces, and grind the faces of the poor? saith the Lord GOD of hosts."],
['Isaiah','Isa','03','016','16',"Moreover the LORD saith, Because the daughters of Zion are haughty, and walk with stretched forth necks and wanton eyes, walking and mincing \emph{as} they go, and making a tinkling with their feet:"],
['Isaiah','Isa','03','017','17',"Therefore the Lord will smite with a scab the crown of the head of the daughters of Zion, and the LORD will discover their secret parts."],
['Isaiah','Isa','03','018','18',"In that day the Lord will take away the bravery of \emph{their} tinkling ornaments \emph{about} \emph{their} \emph{feet}, and \emph{their} cauls, and \emph{their} round tires like the moon,"],
['Isaiah','Isa','03','019','19',"The chains, and the bracelets, and the mufflers,"],
['Isaiah','Isa','03','020','20',"The bonnets, and the ornaments of the legs, and the headbands, and the tablets, and the earrings,"],
['Isaiah','Isa','03','021','21',"The rings, and nose jewels,"],
['Isaiah','Isa','03','022','22',"The changeable suits of apparel, and the mantles, and the wimples, and the crisping pins,"],
['Isaiah','Isa','03','023','23',"The glasses, and the fine linen, and the hoods, and the vails."],
['Isaiah','Isa','03','024','24',"And it shall come to pass, \emph{that} instead of sweet smell there shall be stink; and instead of a girdle a rent; and instead of well set hair baldness; and instead of a stomacher a girding of sackcloth; \emph{and} burning instead of beauty."],
['Isaiah','Isa','03','025','25',"Thy men shall fall by the sword, and thy mighty in the war."],
['Isaiah','Isa','03','026','26',"And her gates shall lament and mourn; and she \emph{being} desolate shall sit upon the ground."]
]

Isaiah04 = [
['Isaiah','Isa','04','001','1',"And in that day seven women shall take hold of one man, saying, We will eat our own bread, and wear our own apparel: only let us be called by thy name, to take away our reproach."],
['Isaiah','Isa','04','002','2',"In that day shall the branch of the LORD be beautiful and glorious, and the fruit of the earth \emph{shall} \emph{be} excellent and comely for them that are escaped of Israel."],
['Isaiah','Isa','04','003','3',"And it shall come to pass, \emph{that} \emph{he} \emph{that} \emph{is} left in Zion, and \emph{he} \emph{that} remaineth in Jerusalem, shall be called holy, \emph{even} every one that is written among the living in Jerusalem:"],
['Isaiah','Isa','04','004','4',"When the Lord shall have washed away the filth of the daughters of Zion, and shall have purged the blood of Jerusalem from the midst thereof by the spirit of judgment, and by the spirit of burning."],
['Isaiah','Isa','04','005','5',"And the LORD will create upon every dwelling place of mount Zion, and upon her assemblies, a cloud and smoke by day, and the shining of a flaming fire by night: for upon all the glory \emph{shall} \emph{be} a defence."],
['Isaiah','Isa','04','006','6',"And there shall be a tabernacle for a shadow in the daytime from the heat, and for a place of refuge, and for a covert from storm and from rain."],
]

Isaiah05 = [
['Isaiah','Isa','05','001','1',"Now will I sing to my wellbeloved a song of my beloved touching his vineyard. My wellbeloved hath a vineyard in a very fruitful hill:"],
['Isaiah','Isa','05','002','2',"And he fenced it, and gathered out the stones thereof, and planted it with the choicest vine, and built a tower in the midst of it, and also made a winepress therein: and he looked that it should bring forth grapes, and it brought forth wild grapes."],
['Isaiah','Isa','05','003','3',"And now, O inhabitants of Jerusalem, and men of Judah, judge, I pray you, betwixt me and my vineyard."],
['Isaiah','Isa','05','004','4',"What could have been done more to my vineyard, that I have not done in it? wherefore, when I looked that it should bring forth grapes, brought it forth wild grapes?"],
['Isaiah','Isa','05','005','5',"And now go to; I will tell you what I will do to my vineyard: I will take away the hedge thereof, and it shall be eaten up; \emph{and} break down the wall thereof, and it shall be trodden down:"],
['Isaiah','Isa','05','006','6',"And I will lay it waste: it shall not be pruned, nor digged; but there shall come up briers and thorns: I will also command the clouds that they rain no rain upon it."],
['Isaiah','Isa','05','007','7',"For the vineyard of the LORD of hosts \emph{is} the house of Israel, and the men of Judah his pleasant plant: and he looked for judgment, but behold oppression; for righteousness, but behold a cry."],
['Isaiah','Isa','05','008','8',"Woe unto them that join house to house, \emph{that} lay field to field, till \emph{there} \emph{be} no place, that they may be placed alone in the midst of the earth!"],
['Isaiah','Isa','05','009','9',"In mine ears \emph{said} the LORD of hosts, Of a truth many houses shall be desolate, \emph{even} great and fair, without inhabitant."],
['Isaiah','Isa','05','010','10',"Yea, ten acres of vineyard shall yield one bath, and the seed of an homer shall yield an ephah."],
['Isaiah','Isa','05','011','11',"Woe unto them that rise up early in the morning, \emph{that} they may follow strong drink; that continue until night, \emph{till} wine inflame them!"],
['Isaiah','Isa','05','012','12',"And the harp, and the viol, the tabret, and pipe, and wine, are in their feasts: but they regard not the work of the LORD, neither consider the operation of his hands."],
['Isaiah','Isa','05','013','13',"Therefore my people are gone into captivity, because \emph{they} \emph{have} no knowledge: and their honourable men \emph{are} famished, and their multitude dried up with thirst."],
['Isaiah','Isa','05','014','14',"Therefore hell hath enlarged herself, and opened her mouth without measure: and their glory, and their multitude, and their pomp, and he that rejoiceth, shall descend into it."],
['Isaiah','Isa','05','015','15',"And the mean man shall be brought down, and the mighty man shall be humbled, and the eyes of the lofty shall be humbled:"],
['Isaiah','Isa','05','016','16',"But the LORD of hosts shall be exalted in judgment, and God that is holy shall be sanctified in righteousness."],
['Isaiah','Isa','05','017','17',"Then shall the lambs feed after their manner, and the waste places of the fat ones shall strangers eat."],
['Isaiah','Isa','05','018','18',"Woe unto them that draw iniquity with cords of vanity, and sin as it were with a cart rope:"],
['Isaiah','Isa','05','019','19',"That say, Let him make speed, \emph{and} hasten his work, that we may see \emph{it}: and let the counsel of the Holy One of Israel draw nigh and come, that we may know \emph{it}!"],
['Isaiah','Isa','05','020','20',"Woe unto them that call evil good, and good evil; that put darkness for light, and light for darkness; that put bitter for sweet, and sweet for bitter!"],
['Isaiah','Isa','05','021','21',"Woe unto \emph{them} \emph{that} \emph{are} wise in their own eyes, and prudent in their own sight!"],
['Isaiah','Isa','05','022','22',"Woe unto \emph{them} \emph{that} \emph{are} mighty to drink wine, and men of strength to mingle strong drink:"],
['Isaiah','Isa','05','023','23',"Which justify the wicked for reward, and take away the righteousness of the righteous from him!"],
['Isaiah','Isa','05','024','24',"Therefore as the fire devoureth the stubble, and the flame consumeth the chaff, \emph{so} their root shall be as rottenness, and their blossom shall go up as dust: because they have cast away the law of the LORD of hosts, and despised the word of the Holy One of Israel."],
['Isaiah','Isa','05','025','25',"Therefore is the anger of the LORD kindled against his people, and he hath stretched forth his hand against them, and hath smitten them: and the hills did tremble, and their carcases \emph{were} torn in the midst of the streets. For all this his anger is not turned away, but his hand \emph{is} stretched out still."],
['Isaiah','Isa','05','026','26',"And he will lift up an ensign to the nations from far, and will hiss unto them from the end of the earth: and, behold, they shall come with speed swiftly:"],
['Isaiah','Isa','05','027','27',"None shall be weary nor stumble among them; none shall slumber nor sleep; neither shall the girdle of their loins be loosed, nor the latchet of their shoes be broken:"],
['Isaiah','Isa','05','028','28',"Whose arrows \emph{are} sharp, and all their bows bent, their horses' hoofs shall be counted like flint, and their wheels like a whirlwind:"],
['Isaiah','Isa','05','029','29',"Their roaring \emph{shall} \emph{be} like a lion, they shall roar like young lions: yea, they shall roar, and lay hold of the prey, and shall carry \emph{it} away safe, and none shall deliver \emph{it}."],
['Isaiah','Isa','05','030','30',"And in that day they shall roar against them like the roaring of the sea: and if \emph{one} look unto the land, behold darkness \emph{and} sorrow, and the light is darkened in the heavens thereof."]
]

Isaiah06 = [
['Isaiah','Isa','06','001','1',"In the year that king Uzziah died I saw also the Lord sitting upon a throne, high and lifted up, and his train filled the temple."],
['Isaiah','Isa','06','002','2',"Above it stood the seraphims: each one had six wings; with twain he covered his face, and with twain he covered his feet, and with twain he did fly."],
['Isaiah','Isa','06','003','3',"And one cried unto another, and said, Holy, holy, holy, \emph{is} the LORD of hosts: the whole earth \emph{is} full of his glory."],
['Isaiah','Isa','06','004','4',"And the posts of the door moved at the voice of him that cried, and the house was filled with smoke."],
['Isaiah','Isa','06','005','5',"Then said I, Woe \emph{is} me! for I am undone; because I \emph{am} a man of unclean lips, and I dwell in the midst of a people of unclean lips: for mine eyes have seen the King, the LORD of hosts."],
['Isaiah','Isa','06','006','6',"Then flew one of the seraphims unto me, having a live coal in his hand, \emph{which} he had taken with the tongs from off the altar:"],
['Isaiah','Isa','06','007','7',"And he laid \emph{it} upon my mouth, and said, Lo, this hath touched thy lips; and thine iniquity is taken away, and thy sin purged."],
['Isaiah','Isa','06','008','8',"Also I heard the voice of the Lord, saying, Whom shall I send, and who will go for us? Then said I, Here \emph{am} I; send me."],
['Isaiah','Isa','06','009','9',"And he said, Go, and tell this people, Hear ye indeed, but understand not; and see ye indeed, but perceive not."],
['Isaiah','Isa','06','010','10',"Make the heart of this people fat, and make their ears heavy, and shut their eyes; lest they see with their eyes, and hear with their ears, and understand with their heart, and convert, and be healed."],
['Isaiah','Isa','06','011','11',"Then said I, Lord, how long? And he answered, Until the cities be wasted without inhabitant, and the houses without man, and the land be utterly desolate,"],
['Isaiah','Isa','06','012','12',"And the LORD have removed men far away, and \emph{there} \emph{be} a great forsaking in the midst of the land."],
['Isaiah','Isa','06','013','13',"But yet in it \emph{shall} \emph{be} a tenth, and \emph{it} shall return, and shall be eaten: as a teil tree, and as an oak, whose substance \emph{is} in them, when they cast \emph{their} \emph{leaves:} \emph{so} the holy seed \emph{shall} \emph{be} the substance thereof."]
]

Isaiah07 = [
['Isaiah','Isa','07','001','1',"And it came to pass in the days of Ahaz the son of Jotham, the son of Uzziah, king of Judah, \emph{that} Rezin the king of Syria, and Pekah the son of Remaliah, king of Israel, went up toward Jerusalem to war against it, but could not prevail against it."],
['Isaiah','Isa','07','002','2',"And it was told the house of David, saying, Syria is confederate with Ephraim. And his heart was moved, and the heart of his people, as the trees of the wood are moved with the wind."],
['Isaiah','Isa','07','003','3',"Then said the LORD unto Isaiah, Go forth now to meet Ahaz, thou, and Shear-jashub thy son, at the end of the conduit of the upper pool in the highway of the fuller's field;"],
['Isaiah','Isa','07','004','4',"And say unto him, Take heed, and be quiet; fear not, neither be fainthearted for the two tails of these smoking firebrands, for the fierce anger of Rezin with Syria, and of the son of Remaliah."],
['Isaiah','Isa','07','005','5',"Because Syria, Ephraim, and the son of Remaliah, have taken evil counsel against thee, saying,"],
['Isaiah','Isa','07','006','6',"Let us go up against Judah, and vex it, and let us make a breach therein for us, and set a king in the midst of it, \emph{even} the son of Tabeal:"],
['Isaiah','Isa','07','007','7',"Thus saith the Lord GOD, It shall not stand, neither shall it come to pass."],
['Isaiah','Isa','07','008','8',"For the head of Syria \emph{is} Damascus, and the head of Damascus \emph{is} Rezin; and within threescore and five years shall Ephraim be broken, that it be not a people."],
['Isaiah','Isa','07','009','9',"And the head of Ephraim \emph{is} Samaria, and the head of Samaria \emph{is} Remaliah's son. If ye will not believe, surely ye shall not be established."],
['Isaiah','Isa','07','010','10',"Moreover the LORD spake again unto Ahaz, saying,"],
['Isaiah','Isa','07','011','11',"Ask thee a sign of the LORD thy God; ask it either in the depth, or in the height above."],
['Isaiah','Isa','07','012','12',"But Ahaz said, I will not ask, neither will I tempt the LORD."],
['Isaiah','Isa','07','013','13',"And he said, Hear ye now, O house of David; \emph{Is} \emph{it} a small thing for you to weary men, but will ye weary my God also?"],
['Isaiah','Isa','07','014','14',"Therefore the Lord himself shall give you a sign; Behold, a virgin shall conceive, and bear a son, and shall call his name Immanuel."],
['Isaiah','Isa','07','015','15',"Butter and honey shall he eat, that he may know to refuse the evil, and choose the good."],
['Isaiah','Isa','07','016','16',"For before the child shall know to refuse the evil, and choose the good, the land that thou abhorrest shall be forsaken of both her kings."],
['Isaiah','Isa','07','017','17',"The LORD shall bring upon thee, and upon thy people, and upon thy father's house, days that have not come, from the day that Ephraim departed from Judah; \emph{even} the king of Assyria."],
['Isaiah','Isa','07','018','18',"And it shall come to pass in that day, \emph{that} the LORD shall hiss for the fly that \emph{is} in the uttermost part of the rivers of Egypt, and for the bee that \emph{is} in the land of Assyria."],
['Isaiah','Isa','07','019','19',"And they shall come, and shall rest all of them in the desolate valleys, and in the holes of the rocks, and upon all thorns, and upon all bushes."],
['Isaiah','Isa','07','020','20',"In the same day shall the Lord shave with a razor that is hired, \emph{namely}, by them beyond the river, by the king of Assyria, the head, and the hair of the feet: and it shall also consume the beard."],
['Isaiah','Isa','07','021','21',"And it shall come to pass in that day, \emph{that} a man shall nourish a young cow, and two sheep;"],
['Isaiah','Isa','07','022','22',"And it shall come to pass, for the abundance of milk \emph{that} they shall give he shall eat butter: for butter and honey shall every one eat that is left in the land."],
['Isaiah','Isa','07','023','23',"And it shall come to pass in that day, \emph{that} every place shall be, where there were a thousand vines at a thousand silverlings, it shall \emph{even} be for briers and thorns."],
['Isaiah','Isa','07','024','24',"With arrows and with bows shall \emph{men} come thither; because all the land shall become briers and thorns."],
['Isaiah','Isa','07','025','25',"And \emph{on} all hills that shall be digged with the mattock, there shall not come thither the fear of briers and thorns: but it shall be for the sending forth of oxen, and for the treading of lesser cattle."]
]

Isaiah08 = [
['Isaiah','Isa','08','001','1',"Moreover the LORD said unto me, Take thee a great roll, and write in it with a man's pen concerning Maher-shalal-hash-baz."],
['Isaiah','Isa','08','002','2',"And I took unto me faithful witnesses to record, Uriah the priest, and Zechariah the son of Jeberechiah."],
['Isaiah','Isa','08','003','3',"And I went unto the prophetess; and she conceived, and bare a son. Then said the LORD to me, Call his name Maher-shalal-hash-baz."],
['Isaiah','Isa','08','004','4',"For before the child shall have knowledge to cry, My father, and my mother, the riches of Damascus and the spoil of Samaria shall be taken away before the king of Assyria."],
['Isaiah','Isa','08','005','5',"The LORD spake also unto me again, saying,"],
['Isaiah','Isa','08','006','6',"Forasmuch as this people refuseth the waters of Shiloah that go softly, and rejoice in Rezin and Remaliah's son;"],
['Isaiah','Isa','08','007','7',"Now therefore, behold, the Lord bringeth up upon them the waters of the river, strong and many, \emph{even} the king of Assyria, and all his glory: and he shall come up over all his channels, and go over all his banks:"],
['Isaiah','Isa','08','008','8',"And he shall pass through Judah; he shall overflow and go over, he shall reach \emph{even} to the neck; and the stretching out of his wings shall fill the breadth of thy land, O Immanuel."],
['Isaiah','Isa','08','009','9',"Associate yourselves, O ye people, and ye shall be broken in pieces; and give ear, all ye of far countries: gird yourselves, and ye shall be broken in pieces; gird yourselves, and ye shall be broken in pieces."],
['Isaiah','Isa','08','010','10',"Take counsel together, and it shall come to nought; speak the word, and it shall not stand: for God \emph{is} with us."],
['Isaiah','Isa','08','011','11',"For the LORD spake thus to me with a strong hand, and instructed me that I should not walk in the way of this people, saying,"],
['Isaiah','Isa','08','012','12',"Say ye not, A confederacy, to all \emph{them} \emph{to} whom this people shall say, A confederacy; neither fear ye their fear, nor be afraid."],
['Isaiah','Isa','08','013','13',"Sanctify the LORD of hosts himself; and \emph{let} him \emph{be} your fear, and \emph{let} him \emph{be} your dread."],
['Isaiah','Isa','08','014','14',"And he shall be for a sanctuary; but for a stone of stumbling and for a rock of offence to both the houses of Israel, for a gin and for a snare to the inhabitants of Jerusalem."],
['Isaiah','Isa','08','015','15',"And many among them shall stumble, and fall, and be broken, and be snared, and be taken."],
['Isaiah','Isa','08','016','16',"Bind up the testimony, seal the law among my disciples."],
['Isaiah','Isa','08','017','17',"And I will wait upon the LORD, that hideth his face from the house of Jacob, and I will look for him."],
['Isaiah','Isa','08','018','18',"Behold, I and the children whom the LORD hath given me \emph{are} for signs and for wonders in Israel from the LORD of hosts, which dwelleth in mount Zion."],
['Isaiah','Isa','08','019','19',"And when they shall say unto you, Seek unto them that have familiar spirits, and unto wizards that peep, and that mutter: should not a people seek unto their God? for the living to the dead?"],
['Isaiah','Isa','08','020','20',"To the law and to the testimony: if they speak not according to this word, \emph{it} \emph{is} because \emph{there} \emph{is} no light in them."],
['Isaiah','Isa','08','021','21',"And they shall pass through it, hardly bestead and hungry: and it shall come to pass, that when they shall be hungry, they shall fret themselves, and curse their king and their God, and look upward."],
['Isaiah','Isa','08','022','22',"And they shall look unto the earth; and behold trouble and darkness, dimness of anguish; and \emph{they} \emph{shall} \emph{be} driven to darkness."]
]

Isaiah09 = [
['Isaiah','Isa','09','001','1',"Nevertheless the dimness \emph{shall} not \emph{be} such as \emph{was} in her vexation, when at the first he lightly afflicted the land of Zebulun and the land of Naphtali, and afterward did more grievously afflict \emph{her} \emph{by} the way of the sea, beyond Jordan, in Galilee of the nations."],
['Isaiah','Isa','09','002','2',"The people that walked in darkness have seen a great light: they that dwell in the land of the shadow of death, upon them hath the light shined."],
['Isaiah','Isa','09','003','3',"Thou hast multiplied the nation, \emph{and} not increased the joy: they joy before thee according to the joy in harvest, \emph{and} as \emph{men} rejoice when they divide the spoil."],
['Isaiah','Isa','09','004','4',"For thou hast broken the yoke of his burden, and the staff of his shoulder, the rod of his oppressor, as in the day of Midian."],
['Isaiah','Isa','09','005','5',"For every battle of the warrior \emph{is} with confused noise, and garments rolled in blood; but \emph{this} shall be with burning \emph{and} fuel of fire."],
['Isaiah','Isa','09','006','6',"For unto us a child is born, unto us a son is given: and the government shall be upon his shoulder: and his name shall be called Wonderful, Counsellor, The mighty God, The everlasting Father, The Prince of Peace."],
['Isaiah','Isa','09','007','7',"Of the increase of \emph{his} government and peace \emph{there} \emph{shall} \emph{be} no end, upon the throne of David, and upon his kingdom, to order it, and to establish it with judgment and with justice from henceforth even for ever. The zeal of the LORD of hosts will perform this."],
['Isaiah','Isa','09','008','8',"The Lord sent a word into Jacob, and it hath lighted upon Israel."],
['Isaiah','Isa','09','009','9',"And all the people shall know, \emph{even} Ephraim and the inhabitant of Samaria, that say in the pride and stoutness of heart,"],
['Isaiah','Isa','09','010','10',"The bricks are fallen down, but we will build with hewn stones: the sycomores are cut down, but we will change \emph{them} \emph{into} cedars."],
['Isaiah','Isa','09','011','11',"Therefore the LORD shall set up the adversaries of Rezin against him, and join his enemies together;"],
['Isaiah','Isa','09','012','12',"The Syrians before, and the Philistines behind; and they shall devour Israel with open mouth. For all this his anger is not turned away, but his hand \emph{is} stretched out still."],
['Isaiah','Isa','09','013','13',"For the people turneth not unto him that smiteth them, neither do they seek the LORD of hosts."],
['Isaiah','Isa','09','014','14',"Therefore the LORD will cut off from Israel head and tail, branch and rush, in one day."],
['Isaiah','Isa','09','015','15',"The ancient and honourable, he \emph{is} the head; and the prophet that teacheth lies, he \emph{is} the tail."],
['Isaiah','Isa','09','016','16',"For the leaders of this people cause \emph{them} to err; and \emph{they} \emph{that} \emph{are} led of them \emph{are} destroyed."],
['Isaiah','Isa','09','017','17',"Therefore the Lord shall have no joy in their young men, neither shall have mercy on their fatherless and widows: for every one \emph{is} an hypocrite and an evildoer, and every mouth speaketh folly. For all this his anger is not turned away, but his hand \emph{is} stretched out still."],
['Isaiah','Isa','09','018','18',"For wickedness burneth as the fire: it shall devour the briers and thorns, and shall kindle in the thickets of the forest, and they shall mount up \emph{like} the lifting up of smoke."],
['Isaiah','Isa','09','019','19',"Through the wrath of the LORD of hosts is the land darkened, and the people shall be as the fuel of the fire: no man shall spare his brother."],
['Isaiah','Isa','09','020','20',"And he shall snatch on the right hand, and be hungry; and he shall eat on the left hand, and they shall not be satisfied: they shall eat every man the flesh of his own arm:"],
['Isaiah','Isa','09','021','21',"Manasseh, Ephraim; and Ephraim, Manasseh: \emph{and} they together \emph{shall} \emph{be} against Judah. For all this his anger is not turned away, but his hand \emph{is} stretched out still."]
]

Isaiah10 = [
['Isaiah','Isa','10','001','1',"Woe unto them that decree unrighteous decrees, and that write grievousness \emph{which} they have prescribed;"],
['Isaiah','Isa','10','002','2',"To turn aside the needy from judgment, and to take away the right from the poor of my people, that widows may be their prey, and \emph{that} they may rob the fatherless!"],
['Isaiah','Isa','10','003','3',"And what will ye do in the day of visitation, and in the desolation \emph{which} shall come from far? to whom will ye flee for help? and where will ye leave your glory?"],
['Isaiah','Isa','10','004','4',"Without me they shall bow down under the prisoners, and they shall fall under the slain. For all this his anger is not turned away, but his hand \emph{is} stretched out still."],
['Isaiah','Isa','10','005','5',"O Assyrian, the rod of mine anger, and the staff in their hand is mine indignation."],
['Isaiah','Isa','10','006','6',"I will send him against an hypocritical nation, and against the people of my wrath will I give him a charge, to take the spoil, and to take the prey, and to tread them down like the mire of the streets."],
['Isaiah','Isa','10','007','7',"Howbeit he meaneth not so, neither doth his heart think so; but \emph{it} \emph{is} in his heart to destroy and cut off nations not a few."],
['Isaiah','Isa','10','008','8',"For he saith, \emph{Are} not my princes altogether kings?"],
['Isaiah','Isa','10','009','9',"\emph{Is} not Calno as Carchemish? \emph{is} not Hamath as Arpad? \emph{is} not Samaria as Damascus?"],
['Isaiah','Isa','10','010','10',"As my hand hath found the kingdoms of the idols, and whose graven images did excel them of Jerusalem and of Samaria;"],
['Isaiah','Isa','10','011','11',"Shall I not, as I have done unto Samaria and her idols, so do to Jerusalem and her idols?"],
['Isaiah','Isa','10','012','12',"Wherefore it shall come to pass, \emph{that} when the Lord hath performed his whole work upon mount Zion and on Jerusalem, I will punish the fruit of the stout heart of the king of Assyria, and the glory of his high looks."],
['Isaiah','Isa','10','013','13',"For he saith, By the strength of my hand I have done \emph{it}, and by my wisdom; for I am prudent: and I have removed the bounds of the people, and have robbed their treasures, and I have put down the inhabitants like a valiant \emph{man}:"],
['Isaiah','Isa','10','014','14',"And my hand hath found as a nest the riches of the people: and as one gathereth eggs \emph{that} \emph{are} left, have I gathered all the earth; and there was none that moved the wing, or opened the mouth, or peeped."],
['Isaiah','Isa','10','015','15',"Shall the axe boast itself against him that heweth therewith? \emph{or} shall the saw magnify itself against him that shaketh it? as if the rod should shake \emph{itself} against them that lift it up, \emph{or} as if the staff should lift up \emph{itself,} \emph{as} \emph{if} \emph{it} \emph{were} no wood."],
['Isaiah','Isa','10','016','16',"Therefore shall the Lord, the Lord of hosts, send among his fat ones leanness; and under his glory he shall kindle a burning like the burning of a fire."],
['Isaiah','Isa','10','017','17',"And the light of Israel shall be for a fire, and his Holy One for a flame: and it shall burn and devour his thorns and his briers in one day;"],
['Isaiah','Isa','10','018','18',"And shall consume the glory of his forest, and of his fruitful field, both soul and body: and they shall be as when a standardbearer fainteth."],
['Isaiah','Isa','10','019','19',"And the rest of the trees of his forest shall be few, that a child may write them."],
['Isaiah','Isa','10','020','20',"And it shall come to pass in that day, \emph{that} the remnant of Israel, and such as are escaped of the house of Jacob, shall no more again stay upon him that smote them; but shall stay upon the LORD, the Holy One of Israel, in truth."],
['Isaiah','Isa','10','021','21',"The remnant shall return, \emph{even} the remnant of Jacob, unto the mighty God."],
['Isaiah','Isa','10','022','22',"For though thy people Israel be as the sand of the sea, \emph{yet} a remnant of them shall return: the consumption decreed shall overflow with righteousness."],
['Isaiah','Isa','10','023','23',"For the Lord GOD of hosts shall make a consumption, even determined, in the midst of all the land."],
['Isaiah','Isa','10','024','24',"Therefore thus saith the Lord GOD of hosts, O my people that dwellest in Zion, be not afraid of the Assyrian: he shall smite thee with a rod, and shall lift up his staff against thee, after the manner of Egypt."],
['Isaiah','Isa','10','025','25',"For yet a very little while, and the indignation shall cease, and mine anger in their destruction."],
['Isaiah','Isa','10','026','26',"And the LORD of hosts shall stir up a scourge for him according to the slaughter of Midian at the rock of Oreb: and \emph{as} his rod \emph{was} upon the sea, so shall he lift it up after the manner of Egypt."],
['Isaiah','Isa','10','027','27',"And it shall come to pass in that day, \emph{that} his burden shall be taken away from off thy shoulder, and his yoke from off thy neck, and the yoke shall be destroyed because of the anointing."],
['Isaiah','Isa','10','028','28',"He is come to Aiath, he is passed to Migron; at Michmash he hath laid up his carriages:"],
['Isaiah','Isa','10','029','29',"They are gone over the passage: they have taken up their lodging at Geba; Ramah is afraid; Gibeah of Saul is fled."],
['Isaiah','Isa','10','030','30',"Lift up thy voice, O daughter of Gallim: cause it to be heard unto Laish, O poor Anathoth."],
['Isaiah','Isa','10','031','31',"Madmenah is removed; the inhabitants of Gebim gather themselves to flee."],
['Isaiah','Isa','10','032','32',"As yet shall he remain at Nob that day: he shall shake his hand \emph{against} the mount of the daughter of Zion, the hill of Jerusalem."],
['Isaiah','Isa','10','033','33',"Behold, the Lord, the LORD of hosts, shall lop the bough with terror: and the high ones of stature \emph{shall} \emph{be} hewn down, and the haughty shall be humbled."],
['Isaiah','Isa','10','034','34',"And he shall cut down the thickets of the forest with iron, and Lebanon shall fall by a mighty one."]
]

Isaiah11 = [
['Isaiah','Isa','11','001','1',"And there shall come forth a rod out of the stem of Jesse, and a Branch shall grow out of his roots:"],
['Isaiah','Isa','11','002','2',"And the spirit of the LORD shall rest upon him, the spirit of wisdom and understanding, the spirit of counsel and might, the spirit of knowledge and of the fear of the LORD;"],
['Isaiah','Isa','11','003','3',"And shall make him of quick understanding in the fear of the LORD: and he shall not judge after the sight of his eyes, neither reprove after the hearing of his ears:"],
['Isaiah','Isa','11','004','4',"But with righteousness shall he judge the poor, and reprove with equity for the meek of the earth: and he shall smite the earth with the rod of his mouth, and with the breath of his lips shall he slay the wicked."],
['Isaiah','Isa','11','005','5',"And righteousness shall be the girdle of his loins, and faithfulness the girdle of his reins."],
['Isaiah','Isa','11','006','6',"The wolf also shall dwell with the lamb, and the leopard shall lie down with the kid; and the calf and the young lion and the fatling together; and a little child shall lead them."],
['Isaiah','Isa','11','007','7',"And the cow and the bear shall feed; their young ones shall lie down together: and the lion shall eat straw like the ox."],
['Isaiah','Isa','11','008','8',"And the sucking child shall play on the hole of the asp, and the weaned child shall put his hand on the cockatrice' den."],
['Isaiah','Isa','11','009','9',"They shall not hurt nor destroy in all my holy mountain: for the earth shall be full of the knowledge of the LORD, as the waters cover the sea."],
['Isaiah','Isa','11','010','10',"And in that day there shall be a root of Jesse, which shall stand for an ensign of the people; to it shall the Gentiles seek: and his rest shall be glorious."],
['Isaiah','Isa','11','011','11',"And it shall come to pass in that day, \emph{that} the Lord shall set his hand again the second time to recover the remnant of his people, which shall be left, from Assyria, and from Egypt, and from Pathros, and from Cush, and from Elam, and from Shinar, and from Hamath, and from the islands of the sea."],
['Isaiah','Isa','11','012','12',"And he shall set up an ensign for the nations, and shall assemble the outcasts of Israel, and gather together the dispersed of Judah from the four corners of the earth."],
['Isaiah','Isa','11','013','13',"The envy also of Ephraim shall depart, and the adversaries of Judah shall be cut off: Ephraim shall not envy Judah, and Judah shall not vex Ephraim."],
['Isaiah','Isa','11','014','14',"But they shall fly upon the shoulders of the Philistines toward the west; they shall spoil them of the east together: they shall lay their hand upon Edom and Moab; and the children of Ammon shall obey them."],
['Isaiah','Isa','11','015','15',"And the LORD shall utterly destroy the tongue of the Egyptian sea; and with his mighty wind shall he shake his hand over the river, and shall smite it in the seven streams, and make \emph{men} go over dryshod."],
['Isaiah','Isa','11','016','16',"And there shall be an highway for the remnant of his people, which shall be left, from Assyria; like as it was to Israel in the day that he came up out of the land of Egypt."]
]

Isaiah12 = [
['Isaiah','Isa','12','001','1',"And in that day thou shalt say, O LORD, I will praise thee: though thou wast angry with me, thine anger is turned away, and thou comfortedst me."],
['Isaiah','Isa','12','002','2',"Behold, God \emph{is} my salvation; I will trust, and not be afraid: for the LORD JEHOVAH \emph{is} my strength and \emph{my} song; he also is become my salvation."],
['Isaiah','Isa','12','003','3',"Therefore with joy shall ye draw water out of the wells of salvation."],
['Isaiah','Isa','12','004','4',"And in that day shall ye say, Praise the LORD, call upon his name, declare his doings among the people, make mention that his name is exalted."],
['Isaiah','Isa','12','005','5',"Sing unto the LORD; for he hath done excellent things: this \emph{is} known in all the earth."],
['Isaiah','Isa','12','006','6',"Cry out and shout, thou inhabitant of Zion: for great \emph{is} the Holy One of Israel in the midst of thee."]
]

Isaiah13 = [
['Isaiah','Isa','13','001','1',"The burden of Babylon, which Isaiah the son of Amoz did see."],
['Isaiah','Isa','13','002','2',"Lift ye up a banner upon the high mountain, exalt the voice unto them, shake the hand, that they may go into the gates of the nobles."],
['Isaiah','Isa','13','003','3',"I have commanded my sanctified ones, I have also called my mighty ones for mine anger, \emph{even} them that rejoice in my highness."],
['Isaiah','Isa','13','004','4',"The noise of a multitude in the mountains, like as of a great people; a tumultuous noise of the kingdoms of nations gathered together: the LORD of hosts mustereth the host of the battle."],
['Isaiah','Isa','13','005','5',"They come from a far country, from the end of heaven, \emph{even} the LORD, and the weapons of his indignation, to destroy the whole land."],
['Isaiah','Isa','13','006','6',"Howl ye; for the day of the LORD \emph{is} at hand; it shall come as a destruction from the Almighty."],
['Isaiah','Isa','13','007','7',"Therefore shall all hands be faint, and every man's heart shall melt:"],
['Isaiah','Isa','13','008','8',"And they shall be afraid: pangs and sorrows shall take hold of them; they shall be in pain as a woman that travaileth: they shall be amazed one at another; their faces \emph{shall} \emph{be} \emph{as} flames."],
['Isaiah','Isa','13','009','9',"Behold, the day of the LORD cometh, cruel both with wrath and fierce anger, to lay the land desolate: and he shall destroy the sinners thereof out of it."],
['Isaiah','Isa','13','010','10',"For the stars of heaven and the constellations thereof shall not give their light: the sun shall be darkened in his going forth, and the moon shall not cause her light to shine."],
['Isaiah','Isa','13','011','11',"And I will punish the world for \emph{their} evil, and the wicked for their iniquity; and I will cause the arrogancy of the proud to cease, and will lay low the haughtiness of the terrible."],
['Isaiah','Isa','13','012','12',"I will make a man more precious than fine gold; even a man than the golden wedge of Ophir."],
['Isaiah','Isa','13','013','13',"Therefore I will shake the heavens, and the earth shall remove out of her place, in the wrath of the LORD of hosts, and in the day of his fierce anger."],
['Isaiah','Isa','13','014','14',"And it shall be as the chased roe, and as a sheep that no man taketh up: they shall every man turn to his own people, and flee every one into his own land."],
['Isaiah','Isa','13','015','15',"Every one that is found shall be thrust through; and every one that is joined \emph{unto} \emph{them} shall fall by the sword."],
['Isaiah','Isa','13','016','16',"Their children also shall be dashed to pieces before their eyes; their houses shall be spoiled, and their wives ravished."],
['Isaiah','Isa','13','017','17',"Behold, I will stir up the Medes against them, which shall not regard silver; and \emph{as} \emph{for} gold, they shall not delight in it."],
['Isaiah','Isa','13','018','18',"\emph{Their} bows also shall dash the young men to pieces; and they shall have no pity on the fruit of the womb; their eye shall not spare children."],
['Isaiah','Isa','13','019','19',"And Babylon, the glory of kingdoms, the beauty of the Chaldees' excellency, shall be as when God overthrew Sodom and Gomorrah."],
['Isaiah','Isa','13','020','20',"It shall never be inhabited, neither shall it be dwelt in from generation to generation: neither shall the Arabian pitch tent there; neither shall the shepherds make their fold there."],
['Isaiah','Isa','13','021','21',"But wild beasts of the desert shall lie there; and their houses shall be full of doleful creatures; and owls shall dwell there, and satyrs shall dance there."],
['Isaiah','Isa','13','022','22',"And the wild beasts of the islands shall cry in their desolate houses, and dragons in \emph{their} pleasant palaces: and her time \emph{is} near to come, and her days shall not be prolonged."]
]

Isaiah14 = [
['Isaiah','Isa','14','001','1',"For the LORD will have mercy on Jacob, and will yet choose Israel, and set them in their own land: and the strangers shall be joined with them, and they shall cleave to the house of Jacob."],
['Isaiah','Isa','14','002','2',"And the people shall take them, and bring them to their place: and the house of Israel shall possess them in the land of the LORD for servants and handmaids: and they shall take them captives, whose captives they were; and they shall rule over their oppressors."],
['Isaiah','Isa','14','003','3',"And it shall come to pass in the day that the LORD shall give thee rest from thy sorrow, and from thy fear, and from the hard bondage wherein thou wast made to serve,"],
['Isaiah','Isa','14','004','4',"That thou shalt take up this proverb against the king of Babylon, and say, How hath the oppressor ceased! the golden city ceased!"],
['Isaiah','Isa','14','005','5',"The LORD hath broken the staff of the wicked, \emph{and} the sceptre of the rulers."],
['Isaiah','Isa','14','006','6',"He who smote the people in wrath with a continual stroke, he that ruled the nations in anger, is persecuted, \emph{and} none hindereth."],
['Isaiah','Isa','14','007','7',"The whole earth is at rest, \emph{and} is quiet: they break forth into singing."],
['Isaiah','Isa','14','008','8',"Yea, the fir trees rejoice at thee, \emph{and} the cedars of Lebanon, \emph{saying}, Since thou art laid down, no feller is come up against us."],
['Isaiah','Isa','14','009','9',"Hell from beneath is moved for thee to meet \emph{thee} at thy coming: it stirreth up the dead for thee, \emph{even} all the chief ones of the earth; it hath raised up from their thrones all the kings of the nations."],
['Isaiah','Isa','14','010','10',"All they shall speak and say unto thee, Art thou also become weak as we? art thou become like unto us?"],
['Isaiah','Isa','14','011','11',"Thy pomp is brought down to the grave, \emph{and} the noise of thy viols: the worm is spread under thee, and the worms cover thee."],
['Isaiah','Isa','14','012','12',"How art thou fallen from heaven, O Lucifer, son of the morning! \emph{how} art thou cut down to the ground, which didst weaken the nations!"],
['Isaiah','Isa','14','013','13',"For thou hast said in thine heart, I will ascend into heaven, I will exalt my throne above the stars of God: I will sit also upon the mount of the congregation, in the sides of the north:"],
['Isaiah','Isa','14','014','14',"I will ascend above the heights of the clouds; I will be like the most High."],
['Isaiah','Isa','14','015','15',"Yet thou shalt be brought down to hell, to the sides of the pit."],
['Isaiah','Isa','14','016','16',"They that see thee shall narrowly look upon thee, \emph{and} consider thee, \emph{saying,} \emph{Is} this the man that made the earth to tremble, that did shake kingdoms;"],
['Isaiah','Isa','14','017','17',"\emph{That} made the world as a wilderness, and destroyed the cities thereof; \emph{that} opened not the house of his prisoners?"],
['Isaiah','Isa','14','018','18',"All the kings of the nations, \emph{even} all of them, lie in glory, every one in his own house."],
['Isaiah','Isa','14','019','19',"But thou art cast out of thy grave like an abominable branch, \emph{and} \emph{as} the raiment of those that are slain, thrust through with a sword, that go down to the stones of the pit; as a carcase trodden under feet."],
['Isaiah','Isa','14','020','20',"Thou shalt not be joined with them in burial, because thou hast destroyed thy land, \emph{and} slain thy people: the seed of evildoers shall never be renowned."],
['Isaiah','Isa','14','021','21',"Prepare slaughter for his children for the iniquity of their fathers; that they do not rise, nor possess the land, nor fill the face of the world with cities."],
['Isaiah','Isa','14','022','22',"For I will rise up against them, saith the LORD of hosts, and cut off from Babylon the name, and remnant, and son, and nephew, saith the LORD."],
['Isaiah','Isa','14','023','23',"I will also make it a possession for the bittern, and pools of water: and I will sweep it with the besom of destruction, saith the LORD of hosts."],
['Isaiah','Isa','14','024','24',"The LORD of hosts hath sworn, saying, Surely as I have thought, so shall it come to pass; and as I have purposed, \emph{so} shall it stand:"],
['Isaiah','Isa','14','025','25',"That I will break the Assyrian in my land, and upon my mountains tread him under foot: then shall his yoke depart from off them, and his burden depart from off their shoulders."],
['Isaiah','Isa','14','026','26',"This \emph{is} the purpose that is purposed upon the whole earth: and this \emph{is} the hand that is stretched out upon all the nations."],
['Isaiah','Isa','14','027','27',"For the LORD of hosts hath purposed, and who shall disannul \emph{it}? and his hand \emph{is} stretched out, and who shall turn it back?"],
['Isaiah','Isa','14','028','28',"In the year that king Ahaz died was this burden."],
['Isaiah','Isa','14','029','29',"Rejoice not thou, whole Palestina, because the rod of him that smote thee is broken: for out of the serpent's root shall come forth a cockatrice, and his fruit \emph{shall} \emph{be} a fiery flying serpent."],
['Isaiah','Isa','14','030','30',"And the firstborn of the poor shall feed, and the needy shall lie down in safety: and I will kill thy root with famine, and he shall slay thy remnant."],
['Isaiah','Isa','14','031','31',"Howl, O gate; cry, O city; thou, whole Palestina, \emph{art} dissolved: for there shall come from the north a smoke, and none \emph{shall} \emph{be} alone in his appointed times."],
['Isaiah','Isa','14','032','32',"What shall \emph{one} then answer the messengers of the nation? That the LORD hath founded Zion, and the poor of his people shall trust in it."]
]

Isaiah15 = [
['Isaiah','Isa','15','001','1',"The burden of Moab. Because in the night Ar of Moab is laid waste, \emph{and} brought to silence; because in the night Kir of Moab is laid waste, \emph{and} brought to silence;"],
['Isaiah','Isa','15','002','2',"He is gone up to Bajith, and to Dibon, the high places, to weep: Moab shall howl over Nebo, and over Medeba: on all their heads \emph{shall} \emph{be} baldness, \emph{and} every beard cut off."],
['Isaiah','Isa','15','003','3',"In their streets they shall gird themselves with sackcloth: on the tops of their houses, and in their streets, every one shall howl, weeping abundantly."],
['Isaiah','Isa','15','004','4',"And Heshbon shall cry, and Elealeh: their voice shall be heard \emph{even} unto Jahaz: therefore the armed soldiers of Moab shall cry out; his life shall be grievous unto him."],
['Isaiah','Isa','15','005','5',"My heart shall cry out for Moab; his fugitives \emph{shall} \emph{flee} unto Zoar, an heifer of three years old: for by the mounting up of Luhith with weeping shall they go it up; for in the way of Horonaim they shall raise up a cry of destruction."],
['Isaiah','Isa','15','006','6',"For the waters of Nimrim shall be desolate: for the hay is withered away, the grass faileth, there is no green thing."],
['Isaiah','Isa','15','007','7',"Therefore the abundance they have gotten, and that which they have laid up, shall they carry away to the brook of the willows."],
['Isaiah','Isa','15','008','8',"For the cry is gone round about the borders of Moab; the howling thereof unto Eglaim, and the howling thereof unto Beer-elim."],
['Isaiah','Isa','15','009','9',"For the waters of Dimon shall be full of blood: for I will bring more upon Dimon, lions upon him that escapeth of Moab, and upon the remnant of the land."]
]

Isaiah16 = [
['Isaiah','Isa','16','001','1',"Send ye the lamb to the ruler of the land from Sela to the wilderness, unto the mount of the daughter of Zion."],
['Isaiah','Isa','16','002','2',"For it shall be, \emph{that}, as a wandering bird cast out of the nest, \emph{so} the daughters of Moab shall be at the fords of Arnon."],
['Isaiah','Isa','16','003','3',"Take counsel, execute judgment; make thy shadow as the night in the midst of the noonday; hide the outcasts; bewray not him that wandereth."],
['Isaiah','Isa','16','004','4',"Let mine outcasts dwell with thee, Moab; be thou a covert to them from the face of the spoiler: for the extortioner is at an end, the spoiler ceaseth, the oppressors are consumed out of the land."],
['Isaiah','Isa','16','005','5',"And in mercy shall the throne be established: and he shall sit upon it in truth in the tabernacle of David, judging, and seeking judgment, and hasting righteousness."],
['Isaiah','Isa','16','006','6',"We have heard of the pride of Moab; \emph{he} \emph{is} very proud: \emph{even} of his haughtiness, and his pride, and his wrath: \emph{but} his lies \emph{shall} not \emph{be} so."],
['Isaiah','Isa','16','007','7',"Therefore shall Moab howl for Moab, every one shall howl: for the foundations of Kir-hareseth shall ye mourn; surely \emph{they} \emph{are} stricken."],
['Isaiah','Isa','16','008','8',"For the fields of Heshbon languish, \emph{and} the vine of Sibmah: the lords of the heathen have broken down the principal plants thereof, they are come \emph{even} unto Jazer, they wandered \emph{through} the wilderness: her branches are stretched out, they are gone over the sea."],
['Isaiah','Isa','16','009','9',"Therefore I will bewail with the weeping of Jazer the vine of Sibmah: I will water thee with my tears, O Heshbon, and Elealeh: for the shouting for thy summer fruits and for thy harvest is fallen."],
['Isaiah','Isa','16','010','10',"And gladness is taken away, and joy out of the plentiful field; and in the vineyards there shall be no singing, neither shall there be shouting: the treaders shall tread out no wine in \emph{their} presses; I have made \emph{their} \emph{vintage} shouting to cease."],
['Isaiah','Isa','16','011','11',"Wherefore my bowels shall sound like an harp for Moab, and mine inward parts for Kir-haresh."],
['Isaiah','Isa','16','012','12',"And it shall come to pass, when it is seen that Moab is weary on the high place, that he shall come to his sanctuary to pray; but he shall not prevail."],
['Isaiah','Isa','16','013','13',"This \emph{is} the word that the LORD hath spoken concerning Moab since that time."],
['Isaiah','Isa','16','014','14',"But now the LORD hath spoken, saying, Within three years, as the years of an hireling, and the glory of Moab shall be contemned, with all that great multitude; and the remnant \emph{shall} \emph{be} very small \emph{and} feeble."]
]

Isaiah17 = [
['Isaiah','Isa','17','001','1',"The burden of Damascus. Behold, Damascus is taken away from \emph{being} a city, and it shall be a ruinous heap."],
['Isaiah','Isa','17','002','2',"The cities of Aroer \emph{are} forsaken: they shall be for flocks, which shall lie down, and none shall make \emph{them} afraid."],
['Isaiah','Isa','17','003','3',"The fortress also shall cease from Ephraim, and the kingdom from Damascus, and the remnant of Syria: they shall be as the glory of the children of Israel, saith the LORD of hosts."],
['Isaiah','Isa','17','004','4',"And in that day it shall come to pass, \emph{that} the glory of Jacob shall be made thin, and the fatness of his flesh shall wax lean."],
['Isaiah','Isa','17','005','5',"And it shall be as when the harvestman gathereth the corn, and reapeth the ears with his arm; and it shall be as he that gathereth ears in the valley of Rephaim."],
['Isaiah','Isa','17','006','6',"Yet gleaning grapes shall be left in it, as the shaking of an olive tree, two \emph{or} three berries in the top of the uppermost bough, four \emph{or} five in the outmost fruitful branches thereof, saith the LORD God of Israel."],
['Isaiah','Isa','17','007','7',"At that day shall a man look to his Maker, and his eyes shall have respect to the Holy One of Israel."],
['Isaiah','Isa','17','008','8',"And he shall not look to the altars, the work of his hands, neither shall respect \emph{that} which his fingers have made, either the groves, or the images."],
['Isaiah','Isa','17','009','9',"In that day shall his strong cities be as a forsaken bough, and an uppermost branch, which they left because of the children of Israel: and there shall be desolation."],
['Isaiah','Isa','17','010','10',"Because thou hast forgotten the God of thy salvation, and hast not been mindful of the rock of thy strength, therefore shalt thou plant pleasant plants, and shalt set it with strange slips:"],
['Isaiah','Isa','17','011','11',"In the day shalt thou make thy plant to grow, and in the morning shalt thou make thy seed to flourish: \emph{but} the harvest \emph{shall} \emph{be} a heap in the day of grief and of desperate sorrow."],
['Isaiah','Isa','17','012','12',"Woe to the multitude of many people, \emph{which} make a noise like the noise of the seas; and to the rushing of nations, \emph{that} make a rushing like the rushing of mighty waters!"],
['Isaiah','Isa','17','013','13',"The nations shall rush like the rushing of many waters: but \emph{God} shall rebuke them, and they shall flee far off, and shall be chased as the chaff of the mountains before the wind, and like a rolling thing before the whirlwind."],
['Isaiah','Isa','17','014','14',"And behold at eveningtide trouble; \emph{and} before the morning he \emph{is} not. This \emph{is} the portion of them that spoil us, and the lot of them that rob us."]
]

Isaiah18 = [
['Isaiah','Isa','18','001','1',"Woe to the land shadowing with wings, which \emph{is} beyond the rivers of Ethiopia:"],
['Isaiah','Isa','18','002','2',"That sendeth ambassadors by the sea, even in vessels of bulrushes upon the waters, \emph{saying}, Go, ye swift messengers, to a nation scattered and peeled, to a people terrible from their beginning hitherto; a nation meted out and trodden down, whose land the rivers have spoiled!"],
['Isaiah','Isa','18','003','3',"All ye inhabitants of the world, and dwellers on the earth, see ye, when he lifteth up an ensign on the mountains; and when he bloweth a trumpet, hear ye."],
['Isaiah','Isa','18','004','4',"For so the LORD said unto me, I will take my rest, and I will consider in my dwelling place like a clear heat upon herbs, \emph{and} like a cloud of dew in the heat of harvest."],
['Isaiah','Isa','18','005','5',"For afore the harvest, when the bud is perfect, and the sour grape is ripening in the flower, he shall both cut off the sprigs with pruning hooks, and take away \emph{and} cut down the branches."],
['Isaiah','Isa','18','006','6',"They shall be left together unto the fowls of the mountains, and to the beasts of the earth: and the fowls shall summer upon them, and all the beasts of the earth shall winter upon them."],
['Isaiah','Isa','18','007','7',"In that time shall the present be brought unto the LORD of hosts of a people scattered and peeled, and from a people terrible from their beginning hitherto; a nation meted out and trodden under foot, whose land the rivers have spoiled, to the place of the name of the LORD of hosts, the mount Zion."]
]

Isaiah19 = [
['Isaiah','Isa','19','001','1',"The burden of Egypt. Behold, the LORD rideth upon a swift cloud, and shall come into Egypt: and the idols of Egypt shall be moved at his presence, and the heart of Egypt shall melt in the midst of it."],
['Isaiah','Isa','19','002','2',"And I will set the Egyptians against the Egyptians: and they shall fight every one against his brother, and every one against his neighbour; city against city, \emph{and} kingdom against kingdom."],
['Isaiah','Isa','19','003','3',"And the spirit of Egypt shall fail in the midst thereof; and I will destroy the counsel thereof: and they shall seek to the idols, and to the charmers, and to them that have familiar spirits, and to the wizards."],
['Isaiah','Isa','19','004','4',"And the Egyptians will I give over into the hand of a cruel lord; and a fierce king shall rule over them, saith the Lord, the LORD of hosts."],
['Isaiah','Isa','19','005','5',"And the waters shall fail from the sea, and the river shall be wasted and dried up."],
['Isaiah','Isa','19','006','6',"And they shall turn the rivers far away; \emph{and} the brooks of defence shall be emptied and dried up: the reeds and flags shall wither."],
['Isaiah','Isa','19','007','7',"The paper reeds by the brooks, by the mouth of the brooks, and every thing sown by the brooks, shall wither, be driven away, and be no \emph{more}."],
['Isaiah','Isa','19','008','8',"The fishers also shall mourn, and all they that cast angle into the brooks shall lament, and they that spread nets upon the waters shall languish."],
['Isaiah','Isa','19','009','9',"Moreover they that work in fine flax, and they that weave networks, shall be confounded."],
['Isaiah','Isa','19','010','10',"And they shall be broken in the purposes thereof, all that make sluices \emph{and} ponds for fish."],
['Isaiah','Isa','19','011','11',"Surely the princes of Zoan \emph{are} fools, the counsel of the wise counsellors of Pharaoh is become brutish: how say ye unto Pharaoh, I \emph{am} the son of the wise, the son of ancient kings?"],
['Isaiah','Isa','19','012','12',"Where \emph{are} they? where \emph{are} thy wise \emph{men}? and let them tell thee now, and let them know what the LORD of hosts hath purposed upon Egypt."],
['Isaiah','Isa','19','013','13',"The princes of Zoan are become fools, the princes of Noph are deceived; they have also seduced Egypt, \emph{even} \emph{they} \emph{that} \emph{are} the stay of the tribes thereof."],
['Isaiah','Isa','19','014','14',"The LORD hath mingled a perverse spirit in the midst thereof: and they have caused Egypt to err in every work thereof, as a drunken \emph{man} staggereth in his vomit."],
['Isaiah','Isa','19','015','15',"Neither shall there be \emph{any} work for Egypt, which the head or tail, branch or rush, may do."],
['Isaiah','Isa','19','016','16',"In that day shall Egypt be like unto women: and it shall be afraid and fear because of the shaking of the hand of the LORD of hosts, which he shaketh over it."],
['Isaiah','Isa','19','017','17',"And the land of Judah shall be a terror unto Egypt, every one that maketh mention thereof shall be afraid in himself, because of the counsel of the LORD of hosts, which he hath determined against it."],
['Isaiah','Isa','19','018','18',"In that day shall five cities in the land of Egypt speak the language of Canaan, and swear to the LORD of hosts; one shall be called, The city of destruction."],
['Isaiah','Isa','19','019','19',"In that day shall there be an altar to the LORD in the midst of the land of Egypt, and a pillar at the border thereof to the LORD."],
['Isaiah','Isa','19','020','20',"And it shall be for a sign and for a witness unto the LORD of hosts in the land of Egypt: for they shall cry unto the LORD because of the oppressors, and he shall send them a saviour, and a great one, and he shall deliver them."],
['Isaiah','Isa','19','021','21',"And the LORD shall be known to Egypt, and the Egyptians shall know the LORD in that day, and shall do sacrifice and oblation; yea, they shall vow a vow unto the LORD, and perform \emph{it}."],
['Isaiah','Isa','19','022','22',"And the LORD shall smite Egypt: he shall smite and heal \emph{it}: and they shall return \emph{even} to the LORD, and he shall be intreated of them, and shall heal them."],
['Isaiah','Isa','19','023','23',"In that day shall there be a highway out of Egypt to Assyria, and the Assyrian shall come into Egypt, and the Egyptian into Assyria, and the Egyptians shall serve with the Assyrians."],
['Isaiah','Isa','19','024','24',"In that day shall Israel be the third with Egypt and with Assyria, \emph{even} a blessing in the midst of the land:"],
['Isaiah','Isa','19','025','25',"Whom the LORD of hosts shall bless, saying, Blessed \emph{be} Egypt my people, and Assyria the work of my hands, and Israel mine inheritance."]
]

Isaiah20 = [
['Isaiah','Isa','20','001','1',"In the year that Tartan came unto Ashdod, (when Sargon the king of Assyria sent him,) and fought against Ashdod, and took it;"],
['Isaiah','Isa','20','002','2',"At the same time spake the LORD by Isaiah the son of Amoz, saying, Go and loose the sackcloth from off thy loins, and put off thy shoe from thy foot. And he did so, walking naked and barefoot."],
['Isaiah','Isa','20','003','3',"And the LORD said, Like as my servant Isaiah hath walked naked and barefoot three years \emph{for} a sign and wonder upon Egypt and upon Ethiopia;"],
['Isaiah','Isa','20','004','4',"So shall the king of Assyria lead away the Egyptians prisoners, and the Ethiopians captives, young and old, naked and barefoot, even with \emph{their} buttocks uncovered, to the shame of Egypt."],
['Isaiah','Isa','20','005','5',"And they shall be afraid and ashamed of Ethiopia their expectation, and of Egypt their glory."],
['Isaiah','Isa','20','006','6',"And the inhabitant of this isle shall say in that day, Behold, such \emph{is} our expectation, whither we flee for help to be delivered from the king of Assyria: and how shall we escape?"]
]

Isaiah21 = [
['Isaiah','Isa','21','001','1',"The burden of the desert of the sea. As whirlwinds in the south pass through; \emph{so} it cometh from the desert, from a terrible land."],
['Isaiah','Isa','21','002','2',"A grievous vision is declared unto me; the treacherous dealer dealeth treacherously, and the spoiler spoileth. Go up, O Elam: besiege, O Media; all the sighing thereof have I made to cease."],
['Isaiah','Isa','21','003','3',"Therefore are my loins filled with pain: pangs have taken hold upon me, as the pangs of a woman that travaileth: I was bowed down at the hearing \emph{of} \emph{it}; I was dismayed at the seeing \emph{of} \emph{it}."],
['Isaiah','Isa','21','004','4',"My heart panted, fearfulness affrighted me: the night of my pleasure hath he turned into fear unto me."],
['Isaiah','Isa','21','005','5',"Prepare the table, watch in the watchtower, eat, drink: arise, ye princes, \emph{and} anoint the shield."],
['Isaiah','Isa','21','006','6',"For thus hath the Lord said unto me, Go, set a watchman, let him declare what he seeth."],
['Isaiah','Isa','21','007','7',"And he saw a chariot \emph{with} a couple of horsemen, a chariot of asses, \emph{and} a chariot of camels; and he hearkened diligently with much heed:"],
['Isaiah','Isa','21','008','8',"And he cried, A lion: My lord, I stand continually upon the watchtower in the daytime, and I am set in my ward whole nights:"],
['Isaiah','Isa','21','009','9',"And, behold, here cometh a chariot of men, \emph{with} a couple of horsemen. And he answered and said, Babylon is fallen, is fallen; and all the graven images of her gods he hath broken unto the ground."],
['Isaiah','Isa','21','010','10',"O my threshing, and the corn of my floor: that which I have heard of the LORD of hosts, the God of Israel, have I declared unto you."],
['Isaiah','Isa','21','011','11',"The burden of Dumah. He calleth to me out of Seir, Watchman, what of the night? Watchman, what of the night?"],
['Isaiah','Isa','21','012','12',"The watchman said, The morning cometh, and also the night: if ye will enquire, enquire ye: return, come."],
['Isaiah','Isa','21','013','13',"The burden upon Arabia. In the forest in Arabia shall ye lodge, O ye travelling companies of Dedanim."],
['Isaiah','Isa','21','014','14',"The inhabitants of the land of Tema brought water to him that was thirsty, they prevented with their bread him that fled."],
['Isaiah','Isa','21','015','15',"For they fled from the swords, from the drawn sword, and from the bent bow, and from the grievousness of war."],
['Isaiah','Isa','21','016','16',"For thus hath the Lord said unto me, Within a year, according to the years of an hireling, and all the glory of Kedar shall fail:"],
['Isaiah','Isa','21','017','17',"And the residue of the number of archers, the mighty men of the children of Kedar, shall be diminished: for the LORD God of Israel hath spoken \emph{it}."]
]

Isaiah22 = [
['Isaiah','Isa','22','001','1',"The burden of the valley of vision. What aileth thee now, that thou art wholly gone up to the housetops?"],
['Isaiah','Isa','22','002','2',"Thou that art full of stirs, a tumultuous city, a joyous city: thy slain \emph{men} \emph{are} not slain with the sword, nor dead in battle."],
['Isaiah','Isa','22','003','3',"All thy rulers are fled together, they are bound by the archers: all that are found in thee are bound together, \emph{which} have fled from far."],
['Isaiah','Isa','22','004','4',"Therefore said I, Look away from me; I will weep bitterly, labour not to comfort me, because of the spoiling of the daughter of my people."],
['Isaiah','Isa','22','005','5',"For \emph{it} \emph{is} a day of trouble, and of treading down, and of perplexity by the Lord GOD of hosts in the valley of vision, breaking down the walls, and of crying to the mountains."],
['Isaiah','Isa','22','006','6',"And Elam bare the quiver with chariots of men \emph{and} horsemen, and Kir uncovered the shield."],
['Isaiah','Isa','22','007','7',"And it shall come to pass, \emph{that} thy choicest valleys shall be full of chariots, and the horsemen shall set themselves in array at the gate."],
['Isaiah','Isa','22','008','8',"And he discovered the covering of Judah, and thou didst look in that day to the armour of the house of the forest."],
['Isaiah','Isa','22','009','9',"Ye have seen also the breaches of the city of David, that they are many: and ye gathered together the waters of the lower pool."],
['Isaiah','Isa','22','010','10',"And ye have numbered the houses of Jerusalem, and the houses have ye broken down to fortify the wall."],
['Isaiah','Isa','22','011','11',"Ye made also a ditch between the two walls for the water of the old pool: but ye have not looked unto the maker thereof, neither had respect unto him that fashioned it long ago."],
['Isaiah','Isa','22','012','12',"And in that day did the Lord GOD of hosts call to weeping, and to mourning, and to baldness, and to girding with sackcloth:"],
['Isaiah','Isa','22','013','13',"And behold joy and gladness, slaying oxen, and killing sheep, eating flesh, and drinking wine: let us eat and drink; for to morrow we shall die."],
['Isaiah','Isa','22','014','14',"And it was revealed in mine ears by the LORD of hosts, Surely this iniquity shall not be purged from you till ye die, saith the Lord GOD of hosts."],
['Isaiah','Isa','22','015','15',"Thus saith the Lord GOD of hosts, Go, get thee unto this treasurer, \emph{even} unto Shebna, which \emph{is} over the house, \emph{and} \emph{say},"],
['Isaiah','Isa','22','016','16',"What hast thou here? and whom hast thou here, that thou hast hewed thee out a sepulchre here, \emph{as} he that heweth him out a sepulchre on high, \emph{and} that graveth an habitation for himself in a rock?"],
['Isaiah','Isa','22','017','17',"Behold, the LORD will carry thee away with a mighty captivity, and will surely cover thee."],
['Isaiah','Isa','22','018','18',"He will surely violently turn and toss thee \emph{like} a ball into a large country: there shalt thou die, and there the chariots of thy glory \emph{shall} \emph{be} the shame of thy lord's house."],
['Isaiah','Isa','22','019','19',"And I will drive thee from thy station, and from thy state shall he pull thee down."],
['Isaiah','Isa','22','020','20',"And it shall come to pass in that day, that I will call my servant Eliakim the son of Hilkiah:"],
['Isaiah','Isa','22','021','21',"And I will clothe him with thy robe, and strengthen him with thy girdle, and I will commit thy government into his hand: and he shall be a father to the inhabitants of Jerusalem, and to the house of Judah."],
['Isaiah','Isa','22','022','22',"And the key of the house of David will I lay upon his shoulder; so he shall open, and none shall shut; and he shall shut, and none shall open."],
['Isaiah','Isa','22','023','23',"And I will fasten him \emph{as} a nail in a sure place; and he shall be for a glorious throne to his father's house."],
['Isaiah','Isa','22','024','24',"And they shall hang upon him all the glory of his father's house, the offspring and the issue, all vessels of small quantity, from the vessels of cups, even to all the vessels of flagons."],
['Isaiah','Isa','22','025','25',"In that day, saith the LORD of hosts, shall the nail that is fastened in the sure place be removed, and be cut down, and fall; and the burden that \emph{was} upon it shall be cut off: for the LORD hath spoken \emph{it}."]
]

Isaiah23 = [
['Isaiah','Isa','23','001','1',"The burden of Tyre. Howl, ye ships of Tarshish; for it is laid waste, so that there is no house, no entering in: from the land of Chittim it is revealed to them."],
['Isaiah','Isa','23','002','2',"Be still, ye inhabitants of the isle; thou whom the merchants of Zidon, that pass over the sea, have replenished."],
['Isaiah','Isa','23','003','3',"And by great waters the seed of Sihor, the harvest of the river, \emph{is} her revenue; and she is a mart of nations."],
['Isaiah','Isa','23','004','4',"Be thou ashamed, O Zidon: for the sea hath spoken, \emph{even} the strength of the sea, saying, I travail not, nor bring forth children, neither do I nourish up young men, \emph{nor} bring up virgins."],
['Isaiah','Isa','23','005','5',"As at the report concerning Egypt, \emph{so} shall they be sorely pained at the report of Tyre."],
['Isaiah','Isa','23','006','6',"Pass ye over to Tarshish; howl, ye inhabitants of the isle."],
['Isaiah','Isa','23','007','7',"\emph{Is} this your joyous \emph{city}, whose antiquity \emph{is} of ancient days? her own feet shall carry her afar off to sojourn."],
['Isaiah','Isa','23','008','8',"Who hath taken this counsel against Tyre, the crowning \emph{city}, whose merchants \emph{are} princes, whose traffickers \emph{are} the honourable of the earth?"],
['Isaiah','Isa','23','009','9',"The LORD of hosts hath purposed it, to stain the pride of all glory, \emph{and} to bring into contempt all the honourable of the earth."],
['Isaiah','Isa','23','010','10',"Pass through thy land as a river, O daughter of Tarshish: \emph{there} \emph{is} no more strength."],
['Isaiah','Isa','23','011','11',"He stretched out his hand over the sea, he shook the kingdoms: the LORD hath given a commandment against the merchant \emph{city}, to destroy the strong holds thereof."],
['Isaiah','Isa','23','012','12',"And he said, Thou shalt no more rejoice, O thou oppressed virgin, daughter of Zidon: arise, pass over to Chittim; there also shalt thou have no rest."],
['Isaiah','Isa','23','013','13',"Behold the land of the Chaldeans; this people was not, \emph{till} the Assyrian founded it for them that dwell in the wilderness: they set up the towers thereof, they raised up the palaces thereof; \emph{and} he brought it to ruin."],
['Isaiah','Isa','23','014','14',"Howl, ye ships of Tarshish: for your strength is laid waste."],
['Isaiah','Isa','23','015','15',"And it shall come to pass in that day, that Tyre shall be forgotten seventy years, according to the days of one king: after the end of seventy years shall Tyre sing as an harlot."],
['Isaiah','Isa','23','016','16',"Take an harp, go about the city, thou harlot that hast been forgotten; make sweet melody, sing many songs, that thou mayest be remembered."],
['Isaiah','Isa','23','017','17',"And it shall come to pass after the end of seventy years, that the LORD will visit Tyre, and she shall turn to her hire, and shall commit fornication with all the kingdoms of the world upon the face of the earth."],
['Isaiah','Isa','23','018','18',"And her merchandise and her hire shall be holiness to the LORD: it shall not be treasured nor laid up; for her merchandise shall be for them that dwell before the LORD, to eat sufficiently, and for durable clothing."]
]

Isaiah24 = [
['Isaiah','Isa','24','001','1',"Behold, the LORD maketh the earth empty, and maketh it waste, and turneth it upside down, and scattereth abroad the inhabitants thereof."],
['Isaiah','Isa','24','002','2',"And it shall be, as with the people, so with the priest; as with the servant, so with his master; as with the maid, so with her mistress; as with the buyer, so with the seller; as with the lender, so with the borrower; as with the taker of usury, so with the giver of usury to him."],
['Isaiah','Isa','24','003','3',"The land shall be utterly emptied, and utterly spoiled: for the LORD hath spoken this word."],
['Isaiah','Isa','24','004','4',"The earth mourneth \emph{and} fadeth away, the world languisheth \emph{and} fadeth away, the haughty people of the earth do languish."],
['Isaiah','Isa','24','005','5',"The earth also is defiled under the inhabitants thereof; because they have transgressed the laws, changed the ordinance, broken the everlasting covenant."],
['Isaiah','Isa','24','006','6',"Therefore hath the curse devoured the earth, and they that dwell therein are desolate: therefore the inhabitants of the earth are burned, and few men left."],
['Isaiah','Isa','24','007','7',"The new wine mourneth, the vine languisheth, all the merryhearted do sigh."],
['Isaiah','Isa','24','008','8',"The mirth of tabrets ceaseth, the noise of them that rejoice endeth, the joy of the harp ceaseth."],
['Isaiah','Isa','24','009','9',"They shall not drink wine with a song; strong drink shall be bitter to them that drink it."],
['Isaiah','Isa','24','010','10',"The city of confusion is broken down: every house is shut up, that no man may come in."],
['Isaiah','Isa','24','011','11',"\emph{There} \emph{is} a crying for wine in the streets; all joy is darkened, the mirth of the land is gone."],
['Isaiah','Isa','24','012','12',"In the city is left desolation, and the gate is smitten with destruction."],
['Isaiah','Isa','24','013','13',"When thus it shall be in the midst of the land among the people, \emph{there} \emph{shall} \emph{be} as the shaking of an olive tree, \emph{and} as the gleaning grapes when the vintage is done."],
['Isaiah','Isa','24','014','14',"They shall lift up their voice, they shall sing for the majesty of the LORD, they shall cry aloud from the sea."],
['Isaiah','Isa','24','015','15',"Wherefore glorify ye the LORD in the fires, \emph{even} the name of the LORD God of Israel in the isles of the sea."],
['Isaiah','Isa','24','016','16',"From the uttermost part of the earth have we heard songs, \emph{even} glory to the righteous. But I said, My leanness, my leanness, woe unto me! the treacherous dealers have dealt treacherously; yea, the treacherous dealers have dealt very treacherously."],
['Isaiah','Isa','24','017','17',"Fear, and the pit, and the snare, \emph{are} upon thee, O inhabitant of the earth."],
['Isaiah','Isa','24','018','18',"And it shall come to pass, \emph{that} he who fleeth from the noise of the fear shall fall into the pit; and he that cometh up out of the midst of the pit shall be taken in the snare: for the windows from on high are open, and the foundations of the earth do shake."],
['Isaiah','Isa','24','019','19',"The earth is utterly broken down, the earth is clean dissolved, the earth is moved exceedingly."],
['Isaiah','Isa','24','020','20',"The earth shall reel to and fro like a drunkard, and shall be removed like a cottage; and the transgression thereof shall be heavy upon it; and it shall fall, and not rise again."],
['Isaiah','Isa','24','021','21',"And it shall come to pass in that day, \emph{that} the LORD shall punish the host of the high ones \emph{that} \emph{are} on high, and the kings of the earth upon the earth."],
['Isaiah','Isa','24','022','22',"And they shall be gathered together, \emph{as} prisoners are gathered in the pit, and shall be shut up in the prison, and after many days shall they be visited."],
['Isaiah','Isa','24','023','23',"Then the moon shall be confounded, and the sun ashamed, when the LORD of hosts shall reign in mount Zion, and in Jerusalem, and before his ancients gloriously."]
]

Isaiah25 = [
['Isaiah','Isa','25','001','1',"O LORD, thou \emph{art} my God; I will exalt thee, I will praise thy name; for thou hast done wonderful \emph{things;} \emph{thy} counsels of old \emph{are} faithfulness \emph{and} truth."],
['Isaiah','Isa','25','002','2',"For thou hast made of a city an heap; \emph{of} a defenced city a ruin: a palace of strangers to be no city; it shall never be built."],
['Isaiah','Isa','25','003','3',"Therefore shall the strong people glorify thee, the city of the terrible nations shall fear thee."],
['Isaiah','Isa','25','004','4',"For thou hast been a strength to the poor, a strength to the needy in his distress, a refuge from the storm, a shadow from the heat, when the blast of the terrible ones \emph{is} as a storm \emph{against} the wall."],
['Isaiah','Isa','25','005','5',"Thou shalt bring down the noise of strangers, as the heat in a dry place; \emph{even} the heat with the shadow of a cloud: the branch of the terrible ones shall be brought low."],
['Isaiah','Isa','25','006','6',"And in this mountain shall the LORD of hosts make unto all people a feast of fat things, a feast of wines on the lees, of fat things full of marrow, of wines on the lees well refined."],
['Isaiah','Isa','25','007','7',"And he will destroy in this mountain the face of the covering cast over all people, and the vail that is spread over all nations."],
['Isaiah','Isa','25','008','8',"He will swallow up death in victory; and the Lord GOD will wipe away tears from off all faces; and the rebuke of his people shall he take away from off all the earth: for the LORD hath spoken \emph{it}."],
['Isaiah','Isa','25','009','9',"And it shall be said in that day, Lo, this \emph{is} our God; we have waited for him, and he will save us: this \emph{is} the LORD; we have waited for him, we will be glad and rejoice in his salvation."],
['Isaiah','Isa','25','010','10',"For in this mountain shall the hand of the LORD rest, and Moab shall be trodden down under him, even as straw is trodden down for the dunghill."],
['Isaiah','Isa','25','011','11',"And he shall spread forth his hands in the midst of them, as he that swimmeth spreadeth forth \emph{his} \emph{hands} to swim: and he shall bring down their pride together with the spoils of their hands."],
['Isaiah','Isa','25','012','12',"And the fortress of the high fort of thy walls shall he bring down, lay low, \emph{and} bring to the ground, \emph{even} to the dust."]
]

Isaiah26 = [
['Isaiah','Isa','26','001','1',"In that day shall this song be sung in the land of Judah; We have a strong city; salvation will \emph{God} appoint \emph{for} walls and bulwarks."],
['Isaiah','Isa','26','002','2',"Open ye the gates, that the righteous nation which keepeth the truth may enter in."],
['Isaiah','Isa','26','003','3',"Thou wilt keep \emph{him} in perfect peace, \emph{whose} mind \emph{is} stayed \emph{on} \emph{thee}: because he trusteth in thee."],
['Isaiah','Isa','26','004','4',"Trust ye in the LORD for ever: for in the LORD JEHOVAH \emph{is} everlasting strength:"],
['Isaiah','Isa','26','005','5',"For he bringeth down them that dwell on high; the lofty city, he layeth it low; he layeth it low, \emph{even} to the ground; he bringeth it \emph{even} to the dust."],
['Isaiah','Isa','26','006','6',"The foot shall tread it down, \emph{even} the feet of the poor, \emph{and} the steps of the needy."],
['Isaiah','Isa','26','007','7',"The way of the just \emph{is} uprightness: thou, most upright, dost weigh the path of the just."],
['Isaiah','Isa','26','008','8',"Yea, in the way of thy judgments, O LORD, have we waited for thee; the desire of \emph{our} soul \emph{is} to thy name, and to the remembrance of thee."],
['Isaiah','Isa','26','009','9',"With my soul have I desired thee in the night; yea, with my spirit within me will I seek thee early: for when thy judgments \emph{are} in the earth, the inhabitants of the world will learn righteousness."],
['Isaiah','Isa','26','010','10',"Let favour be shewed to the wicked, \emph{yet} will he not learn righteousness: in the land of uprightness will he deal unjustly, and will not behold the majesty of the LORD."],
['Isaiah','Isa','26','011','11',"LORD, \emph{when} thy hand is lifted up, they will not see: \emph{but} they shall see, and be ashamed for \emph{their} envy at the people; yea, the fire of thine enemies shall devour them."],
['Isaiah','Isa','26','012','12',"LORD, thou wilt ordain peace for us: for thou also hast wrought all our works in us."],
['Isaiah','Isa','26','013','13',"O LORD our God, \emph{other} lords beside thee have had dominion over us: \emph{but} by thee only will we make mention of thy name."],
['Isaiah','Isa','26','014','14',"\emph{They} \emph{are} dead, they shall not live; \emph{they} \emph{are} deceased, they shall not rise: therefore hast thou visited and destroyed them, and made all their memory to perish."],
['Isaiah','Isa','26','015','15',"Thou hast increased the nation, O LORD, thou hast increased the nation: thou art glorified: thou hadst removed \emph{it} far \emph{unto} all the ends of the earth."],
['Isaiah','Isa','26','016','16',"LORD, in trouble have they visited thee, they poured out a prayer \emph{when} thy chastening \emph{was} upon them."],
['Isaiah','Isa','26','017','17',"Like as a woman with child, \emph{that} draweth near the time of her delivery, is in pain, \emph{and} crieth out in her pangs; so have we been in thy sight, O LORD."],
['Isaiah','Isa','26','018','18',"We have been with child, we have been in pain, we have as it were brought forth wind; we have not wrought any deliverance in the earth; neither have the inhabitants of the world fallen."],
['Isaiah','Isa','26','019','19',"Thy dead \emph{men} shall live, \emph{together} \emph{with} my dead body shall they arise. Awake and sing, ye that dwell in dust: for thy dew \emph{is} \emph{as} the dew of herbs, and the earth shall cast out the dead."],
['Isaiah','Isa','26','020','20',"Come, my people, enter thou into thy chambers, and shut thy doors about thee: hide thyself as it were for a little moment, until the indignation be overpast."],
['Isaiah','Isa','26','021','21',"For, behold, the LORD cometh out of his place to punish the inhabitants of the earth for their iniquity: the earth also shall disclose her blood, and shall no more cover her slain."]
]

Isaiah27 = [
['Isaiah','Isa','27','001','1',"In that day the LORD with his sore and great and strong sword shall punish leviathan the piercing serpent, even leviathan that crooked serpent; and he shall slay the dragon that \emph{is} in the sea."],
['Isaiah','Isa','27','002','2',"In that day sing ye unto her, A vineyard of red wine."],
['Isaiah','Isa','27','003','3',"I the LORD do keep it; I will water it every moment: lest \emph{any} hurt it, I will keep it night and day."],
['Isaiah','Isa','27','004','4',"Fury \emph{is} not in me: who would set the briers \emph{and} thorns against me in battle? I would go through them, I would burn them together."],
['Isaiah','Isa','27','005','5',"Or let him take hold of my strength, \emph{that} he may make peace with me; \emph{and} he shall make peace with me."],
['Isaiah','Isa','27','006','6',"He shall cause them that come of Jacob to take root: Israel shall blossom and bud, and fill the face of the world with fruit."],
['Isaiah','Isa','27','007','7',"Hath he smitten him, as he smote those that smote him? \emph{or} is he slain according to the slaughter of them that are slain by him?"],
['Isaiah','Isa','27','008','8',"In measure, when it shooteth forth, thou wilt debate with it: he stayeth his rough wind in the day of the east wind."],
['Isaiah','Isa','27','009','9',"By this therefore shall the iniquity of Jacob be purged; and this \emph{is} all the fruit to take away his sin; when he maketh all the stones of the altar as chalkstones that are beaten in sunder, the groves and images shall not stand up."],
['Isaiah','Isa','27','010','10',"Yet the defenced city \emph{shall} \emph{be} desolate, \emph{and} the habitation forsaken, and left like a wilderness: there shall the calf feed, and there shall he lie down, and consume the branches thereof."],
['Isaiah','Isa','27','011','11',"When the boughs thereof are withered, they shall be broken off: the women come, \emph{and} set them on fire: for it \emph{is} a people of no understanding: therefore he that made them will not have mercy on them, and he that formed them will shew them no favour."],
['Isaiah','Isa','27','012','12',"And it shall come to pass in that day, \emph{that} the LORD shall beat off from the channel of the river unto the stream of Egypt, and ye shall be gathered one by one, O ye children of Israel."],
['Isaiah','Isa','27','013','13',"And it shall come to pass in that day, \emph{that} the great trumpet shall be blown, and they shall come which were ready to perish in the land of Assyria, and the outcasts in the land of Egypt, and shall worship the LORD in the holy mount at Jerusalem."]
]

Isaiah28 = [
['Isaiah','Isa','28','001','1',"Woe to the crown of pride, to the drunkards of Ephraim, whose glorious beauty \emph{is} a fading flower, which \emph{are} on the head of the fat valleys of them that are overcome with wine!"],
['Isaiah','Isa','28','002','2',"Behold, the Lord hath a mighty and strong one, \emph{which} as a tempest of hail \emph{and} a destroying storm, as a flood of mighty waters overflowing, shall cast down to the earth with the hand."],
['Isaiah','Isa','28','003','3',"The crown of pride, the drunkards of Ephraim, shall be trodden under feet:"],
['Isaiah','Isa','28','004','4',"And the glorious beauty, which \emph{is} on the head of the fat valley, shall be a fading flower, \emph{and} as the hasty fruit before the summer; which \emph{when} he that looketh upon it seeth, while it is yet in his hand he eateth it up."],
['Isaiah','Isa','28','005','5',"In that day shall the LORD of hosts be for a crown of glory, and for a diadem of beauty, unto the residue of his people,"],
['Isaiah','Isa','28','006','6',"And for a spirit of judgment to him that sitteth in judgment, and for strength to them that turn the battle to the gate."],
['Isaiah','Isa','28','007','7',"But they also have erred through wine, and through strong drink are out of the way; the priest and the prophet have erred through strong drink, they are swallowed up of wine, they are out of the way through strong drink; they err in vision, they stumble \emph{in} judgment."],
['Isaiah','Isa','28','008','8',"For all tables are full of vomit \emph{and} filthiness, \emph{so} \emph{that} \emph{there} \emph{is} no place \emph{clean}."],
['Isaiah','Isa','28','009','9',"Whom shall he teach knowledge? and whom shall he make to understand doctrine? \emph{them} \emph{that} \emph{are} weaned from the milk, \emph{and} drawn from the breasts."],
['Isaiah','Isa','28','010','10',"For precept \emph{must} \emph{be} upon precept, precept upon precept; line upon line, line upon line; here a little, \emph{and} there a little:"],
['Isaiah','Isa','28','011','11',"For with stammering lips and another tongue will he speak to this people."],
['Isaiah','Isa','28','012','12',"To whom he said, This \emph{is} the rest \emph{wherewith} ye may cause the weary to rest; and this \emph{is} the refreshing: yet they would not hear."],
['Isaiah','Isa','28','013','13',"But the word of the LORD was unto them precept upon precept, precept upon precept; line upon line, line upon line; here a little, \emph{and} there a little; that they might go, and fall backward, and be broken, and snared, and taken."],
['Isaiah','Isa','28','014','14',"Wherefore hear the word of the LORD, ye scornful men, that rule this people which \emph{is} in Jerusalem."],
['Isaiah','Isa','28','015','15',"Because ye have said, We have made a covenant with death, and with hell are we at agreement; when the overflowing scourge shall pass through, it shall not come unto us: for we have made lies our refuge, and under falsehood have we hid ourselves:"],
['Isaiah','Isa','28','016','16',"Therefore thus saith the Lord GOD, Behold, I lay in Zion for a foundation a stone, a tried stone, a precious corner \emph{stone}, a sure foundation: he that believeth shall not make haste."],
['Isaiah','Isa','28','017','17',"Judgment also will I lay to the line, and righteousness to the plummet: and the hail shall sweep away the refuge of lies, and the waters shall overflow the hiding place."],
['Isaiah','Isa','28','018','18',"And your covenant with death shall be disannulled, and your agreement with hell shall not stand; when the overflowing scourge shall pass through, then ye shall be trodden down by it."],
['Isaiah','Isa','28','019','19',"From the time that it goeth forth it shall take you: for morning by morning shall it pass over, by day and by night: and it shall be a vexation only \emph{to} understand the report."],
['Isaiah','Isa','28','020','20',"For the bed is shorter than that \emph{a} \emph{man} can stretch himself \emph{on} \emph{it}: and the covering narrower than that he can wrap himself \emph{in} \emph{it}."],
['Isaiah','Isa','28','021','21',"For the LORD shall rise up as \emph{in} mount Perazim, he shall be wroth as \emph{in} the valley of Gibeon, that he may do his work, his strange work; and bring to pass his act, his strange act."],
['Isaiah','Isa','28','022','22',"Now therefore be ye not mockers, lest your bands be made strong: for I have heard from the Lord GOD of hosts a consumption, even determined upon the whole earth."],
['Isaiah','Isa','28','023','23',"Give ye ear, and hear my voice; hearken, and hear my speech."],
['Isaiah','Isa','28','024','24',"Doth the plowman plow all day to sow? doth he open and break the clods of his ground?"],
['Isaiah','Isa','28','025','25',"When he hath made plain the face thereof, doth he not cast abroad the fitches, and scatter the cummin, and cast in the principal wheat and the appointed barley and the rie in their place?"],
['Isaiah','Isa','28','026','26',"For his God doth instruct him to discretion, \emph{and} doth teach him."],
['Isaiah','Isa','28','027','27',"For the fitches are not threshed with a threshing instrument, neither is a cart wheel turned about upon the cummin; but the fitches are beaten out with a staff, and the cummin with a rod."],
['Isaiah','Isa','28','028','28',"Bread \emph{corn} is bruised; because he will not ever be threshing it, nor break \emph{it} \emph{with} the wheel of his cart, nor bruise it \emph{with} his horsemen."],
['Isaiah','Isa','28','029','29',"This also cometh forth from the LORD of hosts, \emph{which} is wonderful in counsel, \emph{and} excellent in working."]
]

Isaiah29 = [
['Isaiah','Isa','29','001','1',"Woe to Ariel, to Ariel, the city \emph{where} David dwelt! add ye year to year; let them kill sacrifices."],
['Isaiah','Isa','29','002','2',"Yet I will distress Ariel, and there shall be heaviness and sorrow: and it shall be unto me as Ariel."],
['Isaiah','Isa','29','003','3',"And I will camp against thee round about, and will lay siege against thee with a mount, and I will raise forts against thee."],
['Isaiah','Isa','29','004','4',"And thou shalt be brought down, \emph{and} shalt speak out of the ground, and thy speech shall be low out of the dust, and thy voice shall be, as of one that hath a familiar spirit, out of the ground, and thy speech shall whisper out of the dust."],
['Isaiah','Isa','29','005','5',"Moreover the multitude of thy strangers shall be like small dust, and the multitude of the terrible ones \emph{shall} \emph{be} as chaff that passeth away: yea, it shall be at an instant suddenly."],
['Isaiah','Isa','29','006','6',"Thou shalt be visited of the LORD of hosts with thunder, and with earthquake, and great noise, with storm and tempest, and the flame of devouring fire."],
['Isaiah','Isa','29','007','7',"And the multitude of all the nations that fight against Ariel, even all that fight against her and her munition, and that distress her, shall be as a dream of a night vision."],
['Isaiah','Isa','29','008','8',"It shall even be as when an hungry \emph{man} dreameth, and, behold, he eateth; but he awaketh, and his soul is empty: or as when a thirsty man dreameth, and, behold, he drinketh; but he awaketh, and, behold, \emph{he} \emph{is} faint, and his soul hath appetite: so shall the multitude of all the nations be, that fight against mount Zion."],
['Isaiah','Isa','29','009','9',"Stay yourselves, and wonder; cry ye out, and cry: they are drunken, but not with wine; they stagger, but not with strong drink."],
['Isaiah','Isa','29','010','10',"For the LORD hath poured out upon you the spirit of deep sleep, and hath closed your eyes: the prophets and your rulers, the seers hath he covered."],
['Isaiah','Isa','29','011','11',"And the vision of all is become unto you as the words of a book that is sealed, which \emph{men} deliver to one that is learned, saying, Read this, I pray thee: and he saith, I cannot; for it \emph{is} sealed:"],
['Isaiah','Isa','29','012','12',"And the book is delivered to him that is not learned, saying, Read this, I pray thee: and he saith, I am not learned."],
['Isaiah','Isa','29','013','13',"Wherefore the Lord said, Forasmuch as this people draw near \emph{me} with their mouth, and with their lips do honour me, but have removed their heart far from me, and their fear toward me is taught by the precept of men:"],
['Isaiah','Isa','29','014','14',"Therefore, behold, I will proceed to do a marvellous work among this people, \emph{even} a marvellous work and a wonder: for the wisdom of their wise \emph{men} shall perish, and the understanding of their prudent \emph{men} shall be hid."],
['Isaiah','Isa','29','015','15',"Woe unto them that seek deep to hide their counsel from the LORD, and their works are in the dark, and they say, Who seeth us? and who knoweth us?"],
['Isaiah','Isa','29','016','16',"Surely your turning of things upside down shall be esteemed as the potter's clay: for shall the work say of him that made it, He made me not? or shall the thing framed say of him that framed it, He had no understanding?"],
['Isaiah','Isa','29','017','17',"\emph{Is} it not yet a very little while, and Lebanon shall be turned into a fruitful field, and the fruitful field shall be esteemed as a forest?"],
['Isaiah','Isa','29','018','18',"And in that day shall the deaf hear the words of the book, and the eyes of the blind shall see out of obscurity, and out of darkness."],
['Isaiah','Isa','29','019','19',"The meek also shall increase \emph{their} joy in the LORD, and the poor among men shall rejoice in the Holy One of Israel."],
['Isaiah','Isa','29','020','20',"For the terrible one is brought to nought, and the scorner is consumed, and all that watch for iniquity are cut off:"],
['Isaiah','Isa','29','021','21',"That make a man an offender for a word, and lay a snare for him that reproveth in the gate, and turn aside the just for a thing of nought."],
['Isaiah','Isa','29','022','22',"Therefore thus saith the LORD, who redeemed Abraham, concerning the house of Jacob, Jacob shall not now be ashamed, neither shall his face now wax pale."],
['Isaiah','Isa','29','023','23',"But when he seeth his children, the work of mine hands, in the midst of him, they shall sanctify my name, and sanctify the Holy One of Jacob, and shall fear the God of Israel."],
['Isaiah','Isa','29','024','24',"They also that erred in spirit shall come to understanding, and they that murmured shall learn doctrine."]
]

Isaiah30 = [
['Isaiah','Isa','30','001','1',"Woe to the rebellious children, saith the LORD, that take counsel, but not of me; and that cover with a covering, but not of my spirit, that they may add sin to sin:"],
['Isaiah','Isa','30','002','2',"That walk to go down into Egypt, and have not asked at my mouth; to strengthen themselves in the strength of Pharaoh, and to trust in the shadow of Egypt!"],
['Isaiah','Isa','30','003','3',"Therefore shall the strength of Pharaoh be your shame, and the trust in the shadow of Egypt \emph{your} confusion."],
['Isaiah','Isa','30','004','4',"For his princes were at Zoan, and his ambassadors came to Hanes."],
['Isaiah','Isa','30','005','5',"They were all ashamed of a people \emph{that} could not profit them, nor be an help nor profit, but a shame, and also a reproach."],
['Isaiah','Isa','30','006','6',"The burden of the beasts of the south: into the land of trouble and anguish, from whence \emph{come} the young and old lion, the viper and fiery flying serpent, they will carry their riches upon the shoulders of young asses, and their treasures upon the bunches of camels, to a people \emph{that} shall not profit \emph{them}."],
['Isaiah','Isa','30','007','7',"For the Egyptians shall help in vain, and to no purpose: therefore have I cried concerning this, Their strength \emph{is} to sit still."],
['Isaiah','Isa','30','008','8',"Now go, write it before them in a table, and note it in a book, that it may be for the time to come for ever and ever:"],
['Isaiah','Isa','30','009','9',"That this \emph{is} a rebellious people, lying children, children \emph{that} will not hear the law of the LORD:"],
['Isaiah','Isa','30','010','10',"Which say to the seers, See not; and to the prophets, Prophesy not unto us right things, speak unto us smooth things, prophesy deceits:"],
['Isaiah','Isa','30','011','11',"Get you out of the way, turn aside out of the path, cause the Holy One of Israel to cease from before us."],
['Isaiah','Isa','30','012','12',"Wherefore thus saith the Holy One of Israel, Because ye despise this word, and trust in oppression and perverseness, and stay thereon:"],
['Isaiah','Isa','30','013','13',"Therefore this iniquity shall be to you as a breach ready to fall, swelling out in a high wall, whose breaking cometh suddenly at an instant."],
['Isaiah','Isa','30','014','14',"And he shall break it as the breaking of the potters' vessel that is broken in pieces; he shall not spare: so that there shall not be found in the bursting of it a sherd to take fire from the hearth, or to take water \emph{withal} out of the pit."],
['Isaiah','Isa','30','015','15',"For thus saith the Lord GOD, the Holy One of Israel; In returning and rest shall ye be saved; in quietness and in confidence shall be your strength: and ye would not."],
['Isaiah','Isa','30','016','16',"But ye said, No; for we will flee upon horses; therefore shall ye flee: and, We will ride upon the swift; therefore shall they that pursue you be swift."],
['Isaiah','Isa','30','017','17',"One thousand \emph{shall} \emph{flee} at the rebuke of one; at the rebuke of five shall ye flee: till ye be left as a beacon upon the top of a mountain, and as an ensign on an hill."],
['Isaiah','Isa','30','018','18',"And therefore will the LORD wait, that he may be gracious unto you, and therefore will he be exalted, that he may have mercy upon you: for the LORD \emph{is} a God of judgment: blessed \emph{are} all they that wait for him."],
['Isaiah','Isa','30','019','19',"For the people shall dwell in Zion at Jerusalem: thou shalt weep no more: he will be very gracious unto thee at the voice of thy cry; when he shall hear it, he will answer thee."],
['Isaiah','Isa','30','020','20',"And \emph{though} the Lord give you the bread of adversity, and the water of affliction, yet shall not thy teachers be removed into a corner any more, but thine eyes shall see thy teachers:"],
['Isaiah','Isa','30','021','21',"And thine ears shall hear a word behind thee, saying, This \emph{is} the way, walk ye in it, when ye turn to the right hand, and when ye turn to the left."],
['Isaiah','Isa','30','022','22',"Ye shall defile also the covering of thy graven images of silver, and the ornament of thy molten images of gold: thou shalt cast them away as a menstruous cloth; thou shalt say unto it, Get thee hence."],
['Isaiah','Isa','30','023','23',"Then shall he give the rain of thy seed, that thou shalt sow the ground withal; and bread of the increase of the earth, and it shall be fat and plenteous: in that day shall thy cattle feed in large pastures."],
['Isaiah','Isa','30','024','24',"The oxen likewise and the young asses that ear the ground shall eat clean provender, which hath been winnowed with the shovel and with the fan."],
['Isaiah','Isa','30','025','25',"And there shall be upon every high mountain, and upon every high hill, rivers \emph{and} streams of waters in the day of the great slaughter, when the towers fall."],
['Isaiah','Isa','30','026','26',"Moreover the light of the moon shall be as the light of the sun, and the light of the sun shall be sevenfold, as the light of seven days, in the day that the LORD bindeth up the breach of his people, and healeth the stroke of their wound."],
['Isaiah','Isa','30','027','27',"Behold, the name of the LORD cometh from far, burning \emph{with} his anger, and the burden \emph{thereof} \emph{is} heavy: his lips are full of indignation, and his tongue as a devouring fire:"],
['Isaiah','Isa','30','028','28',"And his breath, as an overflowing stream, shall reach to the midst of the neck, to sift the nations with the sieve of vanity: and \emph{there} \emph{shall} \emph{be} a bridle in the jaws of the people, causing \emph{them} to err."],
['Isaiah','Isa','30','029','29',"Ye shall have a song, as in the night \emph{when} a holy solemnity is kept; and gladness of heart, as when one goeth with a pipe to come into the mountain of the LORD, to the mighty One of Israel."],
['Isaiah','Isa','30','030','30',"And the LORD shall cause his glorious voice to be heard, and shall shew the lighting down of his arm, with the indignation of \emph{his} anger, and \emph{with} the flame of a devouring fire, \emph{with} scattering, and tempest, and hailstones."],
['Isaiah','Isa','30','031','31',"For through the voice of the LORD shall the Assyrian be beaten down, \emph{which} smote with a rod."],
['Isaiah','Isa','30','032','32',"And \emph{in} every place where the grounded staff shall pass, which the LORD shall lay upon him, \emph{it} shall be with tabrets and harps: and in battles of shaking will he fight with it."],
['Isaiah','Isa','30','033','33',"For Tophet \emph{is} ordained of old; yea, for the king it is prepared; he hath made \emph{it} deep \emph{and} large: the pile thereof \emph{is} fire and much wood; the breath of the LORD, like a stream of brimstone, doth kindle it."]
]

Isaiah31 = [
['Isaiah','Isa','31','001','1',"Woe to them that go down to Egypt for help; and stay on horses, and trust in chariots, because \emph{they} \emph{are} many; and in horsemen, because they are very strong; but they look not unto the Holy One of Israel, neither seek the LORD!"],
['Isaiah','Isa','31','002','2',"Yet he also \emph{is} wise, and will bring evil, and will not call back his words: but will arise against the house of the evildoers, and against the help of them that work iniquity."],
['Isaiah','Isa','31','003','3',"Now the Egyptians \emph{are} men, and not God; and their horses flesh, and not spirit. When the LORD shall stretch out his hand, both he that helpeth shall fall, and he that is holpen shall fall down, and they all shall fail together."],
['Isaiah','Isa','31','004','4',"For thus hath the LORD spoken unto me, Like as the lion and the young lion roaring on his prey, when a multitude of shepherds is called forth against him, \emph{he} will not be afraid of their voice, nor abase himself for the noise of them: so shall the LORD of hosts come down to fight for mount Zion, and for the hill thereof."],
['Isaiah','Isa','31','005','5',"As birds flying, so will the LORD of hosts defend Jerusalem; defending also he will deliver \emph{it;} \emph{and} passing over he will preserve it."],
['Isaiah','Isa','31','006','6',"Turn ye unto \emph{him} \emph{from} whom the children of Israel have deeply revolted."],
['Isaiah','Isa','31','007','7',"For in that day every man shall cast away his idols of silver, and his idols of gold, which your own hands have made unto you \emph{for} a sin."],
['Isaiah','Isa','31','008','8',"Then shall the Assyrian fall with the sword, not of a mighty man; and the sword, not of a mean man, shall devour him: but he shall flee from the sword, and his young men shall be discomfited."],
['Isaiah','Isa','31','009','9',"And he shall pass over to his strong hold for fear, and his princes shall be afraid of the ensign, saith the LORD, whose fire \emph{is} in Zion, and his furnace in Jerusalem."]
]

Isaiah32 = [
['Isaiah','Isa','32','001','1',"Behold, a king shall reign in righteousness, and princes shall rule in judgment."],
['Isaiah','Isa','32','002','2',"And a man shall be as an hiding place from the wind, and a covert from the tempest; as rivers of water in a dry place, as the shadow of a great rock in a weary land."],
['Isaiah','Isa','32','003','3',"And the eyes of them that see shall not be dim, and the ears of them that hear shall hearken."],
['Isaiah','Isa','32','004','4',"The heart also of the rash shall understand knowledge, and the tongue of the stammerers shall be ready to speak plainly."],
['Isaiah','Isa','32','005','5',"The vile person shall be no more called liberal, nor the churl said \emph{to} \emph{be} bountiful."],
['Isaiah','Isa','32','006','6',"For the vile person will speak villany, and his heart will work iniquity, to practise hypocrisy, and to utter error against the LORD, to make empty the soul of the hungry, and he will cause the drink of the thirsty to fail."],
['Isaiah','Isa','32','007','7',"The instruments also of the churl \emph{are} evil: he deviseth wicked devices to destroy the poor with lying words, even when the needy speaketh right."],
['Isaiah','Isa','32','008','8',"But the liberal deviseth liberal things; and by liberal things shall he stand."],
['Isaiah','Isa','32','009','9',"Rise up, ye women that are at ease; hear my voice, ye careless daughters; give ear unto my speech."],
['Isaiah','Isa','32','010','10',"Many days and years shall ye be troubled, ye careless women: for the vintage shall fail, the gathering shall not come."],
['Isaiah','Isa','32','011','11',"Tremble, ye women that are at ease; be troubled, ye careless ones: strip you, and make you bare, and gird \emph{sackcloth} upon \emph{your} loins."],
['Isaiah','Isa','32','012','12',"They shall lament for the teats, for the pleasant fields, for the fruitful vine."],
['Isaiah','Isa','32','013','13',"Upon the land of my people shall come up thorns \emph{and} briers; yea, upon all the houses of joy \emph{in} the joyous city:"],
['Isaiah','Isa','32','014','14',"Because the palaces shall be forsaken; the multitude of the city shall be left; the forts and towers shall be for dens for ever, a joy of wild asses, a pasture of flocks;"],
['Isaiah','Isa','32','015','15',"Until the spirit be poured upon us from on high, and the wilderness be a fruitful field, and the fruitful field be counted for a forest."],
['Isaiah','Isa','32','016','16',"Then judgment shall dwell in the wilderness, and righteousness remain in the fruitful field."],
['Isaiah','Isa','32','017','17',"And the work of righteousness shall be peace; and the effect of righteousness quietness and assurance for ever."],
['Isaiah','Isa','32','018','18',"And my people shall dwell in a peaceable habitation, and in sure dwellings, and in quiet resting places;"],
['Isaiah','Isa','32','019','19',"When it shall hail, coming down on the forest; and the city shall be low in a low place."],
['Isaiah','Isa','32','020','20',"Blessed \emph{are} ye that sow beside all waters, that send forth \emph{thither} the feet of the ox and the ass."]
]

Isaiah33 = [
['Isaiah','Isa','33','001','1',"Woe to thee that spoilest, and thou \emph{wast} not spoiled; and dealest treacherously, and they dealt not treacherously with thee! when thou shalt cease to spoil, thou shalt be spoiled; \emph{and} when thou shalt make an end to deal treacherously, they shall deal treacherously with thee."],
['Isaiah','Isa','33','002','2',"O LORD, be gracious unto us; we have waited for thee: be thou their arm every morning, our salvation also in the time of trouble."],
['Isaiah','Isa','33','003','3',"At the noise of the tumult the people fled; at the lifting up of thyself the nations were scattered."],
['Isaiah','Isa','33','004','4',"And your spoil shall be gathered \emph{like} the gathering of the caterpiller: as the running to and fro of locusts shall he run upon them."],
['Isaiah','Isa','33','005','5',"The LORD is exalted; for he dwelleth on high: he hath filled Zion with judgment and righteousness."],
['Isaiah','Isa','33','006','6',"And wisdom and knowledge shall be the stability of thy times, \emph{and} strength of salvation: the fear of the LORD \emph{is} his treasure."],
['Isaiah','Isa','33','007','7',"Behold, their valiant ones shall cry without: the ambassadors of peace shall weep bitterly."],
['Isaiah','Isa','33','008','8',"The highways lie waste, the wayfaring man ceaseth: he hath broken the covenant, he hath despised the cities, he regardeth no man."],
['Isaiah','Isa','33','009','9',"The earth mourneth \emph{and} languisheth: Lebanon is ashamed \emph{and} hewn down: Sharon is like a wilderness; and Bashan and Carmel shake off \emph{their} \emph{fruits}."],
['Isaiah','Isa','33','010','10',"Now will I rise, saith the LORD; now will I be exalted; now will I lift up myself."],
['Isaiah','Isa','33','011','11',"Ye shall conceive chaff, ye shall bring forth stubble: your breath, \emph{as} fire, shall devour you."],
['Isaiah','Isa','33','012','12',"And the people shall be \emph{as} the burnings of lime: \emph{as} thorns cut up shall they be burned in the fire."],
['Isaiah','Isa','33','013','13',"Hear, ye \emph{that} \emph{are} far off, what I have done; and, ye \emph{that} \emph{are} near, acknowledge my might."],
['Isaiah','Isa','33','014','14',"The sinners in Zion are afraid; fearfulness hath surprised the hypocrites. Who among us shall dwell with the devouring fire? who among us shall dwell with everlasting burnings?"],
['Isaiah','Isa','33','015','15',"He that walketh righteously, and speaketh uprightly; he that despiseth the gain of oppressions, that shaketh his hands from holding of bribes, that stoppeth his ears from hearing of blood, and shutteth his eyes from seeing evil;"],
['Isaiah','Isa','33','016','16',"He shall dwell on high: his place of defence \emph{shall} \emph{be} the munitions of rocks: bread shall be given him; his waters \emph{shall} \emph{be} sure."],
['Isaiah','Isa','33','017','17',"Thine eyes shall see the king in his beauty: they shall behold the land that is very far off."],
['Isaiah','Isa','33','018','18',"Thine heart shall meditate terror. Where \emph{is} the scribe? where \emph{is} the receiver? where \emph{is} he that counted the towers?"],
['Isaiah','Isa','33','019','19',"Thou shalt not see a fierce people, a people of a deeper speech than thou canst perceive; of a stammering tongue, \emph{that} \emph{thou} \emph{canst} not understand."],
['Isaiah','Isa','33','020','20',"Look upon Zion, the city of our solemnities: thine eyes shall see Jerusalem a quiet habitation, a tabernacle \emph{that} shall not be taken down; not one of the stakes thereof shall ever be removed, neither shall any of the cords thereof be broken."],
['Isaiah','Isa','33','021','21',"But there the glorious LORD \emph{will} \emph{be} unto us a place of broad rivers \emph{and} streams; wherein shall go no galley with oars, neither shall gallant ship pass thereby."],
['Isaiah','Isa','33','022','22',"For the LORD \emph{is} our judge, the LORD \emph{is} our lawgiver, the LORD \emph{is} our king; he will save us."],
['Isaiah','Isa','33','023','23',"Thy tacklings are loosed; they could not well strengthen their mast, they could not spread the sail: then is the prey of a great spoil divided; the lame take the prey."],
['Isaiah','Isa','33','024','24',"And the inhabitant shall not say, I am sick: the people that dwell therein \emph{shall} \emph{be} forgiven \emph{their} iniquity."]
]

Isaiah34 = [
['Isaiah','Isa','34','001','1',"Come near, ye nations, to hear; and hearken, ye people: let the earth hear, and all that is therein; the world, and all things that come forth of it."],
['Isaiah','Isa','34','002','2',"For the indignation of the LORD \emph{is} upon all nations, and \emph{his} fury upon all their armies: he hath utterly destroyed them, he hath delivered them to the slaughter."],
['Isaiah','Isa','34','003','3',"Their slain also shall be cast out, and their stink shall come up out of their carcases, and the mountains shall be melted with their blood."],
['Isaiah','Isa','34','004','4',"And all the host of heaven shall be dissolved, and the heavens shall be rolled together as a scroll: and all their host shall fall down, as the leaf falleth off from the vine, and as a falling \emph{fig} from the fig tree."],
['Isaiah','Isa','34','005','5',"For my sword shall be bathed in heaven: behold, it shall come down upon Idumea, and upon the people of my curse, to judgment."],
['Isaiah','Isa','34','006','6',"The sword of the LORD is filled with blood, it is made fat with fatness, \emph{and} with the blood of lambs and goats, with the fat of the kidneys of rams: for the LORD hath a sacrifice in Bozrah, and a great slaughter in the land of Idumea."],
['Isaiah','Isa','34','007','7',"And the unicorns shall come down with them, and the bullocks with the bulls; and their land shall be soaked with blood, and their dust made fat with fatness."],
['Isaiah','Isa','34','008','8',"For \emph{it} \emph{is} the day of the LORD'S vengeance, \emph{and} the year of recompences for the controversy of Zion."],
['Isaiah','Isa','34','009','9',"And the streams thereof shall be turned into pitch, and the dust thereof into brimstone, and the land thereof shall become burning pitch."],
['Isaiah','Isa','34','010','10',"It shall not be quenched night nor day; the smoke thereof shall go up for ever: from generation to generation it shall lie waste; none shall pass through it for ever and ever."],
['Isaiah','Isa','34','011','11',"But the cormorant and the bittern shall possess it; the owl also and the raven shall dwell in it: and he shall stretch out upon it the line of confusion, and the stones of emptiness."],
['Isaiah','Isa','34','012','12',"They shall call the nobles thereof to the kingdom, but none \emph{shall} \emph{be} there, and all her princes shall be nothing."],
['Isaiah','Isa','34','013','13',"And thorns shall come up in her palaces, nettles and brambles in the fortresses thereof: and it shall be an habitation of dragons, \emph{and} a court for owls."],
['Isaiah','Isa','34','014','14',"The wild beasts of the desert shall also meet with the wild beasts of the island, and the satyr shall cry to his fellow; the screech owl also shall rest there, and find for herself a place of rest."],
['Isaiah','Isa','34','015','15',"There shall the great owl make her nest, and lay, and hatch, and gather under her shadow: there shall the vultures also be gathered, every one with her mate."],
['Isaiah','Isa','34','016','16',"Seek ye out of the book of the LORD, and read: no one of these shall fail, none shall want her mate: for my mouth it hath commanded, and his spirit it hath gathered them."],
['Isaiah','Isa','34','017','17',"And he hath cast the lot for them, and his hand hath divided it unto them by line: they shall possess it for ever, from generation to generation shall they dwell therein."]
]

Isaiah35 = [
['Isaiah','Isa','35','001','1',"The wilderness and the solitary place shall be glad for them; and the desert shall rejoice, and blossom as the rose."],
['Isaiah','Isa','35','002','2',"It shall blossom abundantly, and rejoice even with joy and singing: the glory of Lebanon shall be given unto it, the excellency of Carmel and Sharon, they shall see the glory of the LORD, \emph{and} the excellency of our God."],
['Isaiah','Isa','35','003','3',"Strengthen ye the weak hands, and confirm the feeble knees."],
['Isaiah','Isa','35','004','4',"Say to them \emph{that} \emph{are} of a fearful heart, Be strong, fear not: behold, your God will come \emph{with} vengeance, \emph{even} God \emph{with} a recompence; he will come and save you."],
['Isaiah','Isa','35','005','5',"Then the eyes of the blind shall be opened, and the ears of the deaf shall be unstopped."],
['Isaiah','Isa','35','006','6',"Then shall the lame \emph{man} leap as an hart, and the tongue of the dumb sing: for in the wilderness shall waters break out, and streams in the desert."],
['Isaiah','Isa','35','007','7',"And the parched ground shall become a pool, and the thirsty land springs of water: in the habitation of dragons, where each lay, \emph{shall} \emph{be} grass with reeds and rushes."],
['Isaiah','Isa','35','008','8',"And an highway shall be there, and a way, and it shall be called The way of holiness; the unclean shall not pass over it; but it \emph{shall} \emph{be} for those: the wayfaring men, though fools, shall not err \emph{therein}."],
['Isaiah','Isa','35','009','9',"No lion shall be there, nor \emph{any} ravenous beast shall go up thereon, it shall not be found there; but the redeemed shall walk \emph{there}:"],
['Isaiah','Isa','35','010','10',"And the ransomed of the LORD shall return, and come to Zion with songs and everlasting joy upon their heads: they shall obtain joy and gladness, and sorrow and sighing shall flee away."]
]

Isaiah36 = [
['Isaiah','Isa','36','001','1',"Now it came to pass in the fourteenth year of king Hezekiah, \emph{that} Sennacherib king of Assyria came up against all the defenced cities of Judah, and took them."],
['Isaiah','Isa','36','002','2',"And the king of Assyria sent Rabshakeh from Lachish to Jerusalem unto king Hezekiah with a great army. And he stood by the conduit of the upper pool in the highway of the fuller's field."],
['Isaiah','Isa','36','003','3',"Then came forth unto him Eliakim, Hilkiah's son, which was over the house, and Shebna the scribe, and Joah, Asaph's son, the recorder."],
['Isaiah','Isa','36','004','4',"And Rabshakeh said unto them, Say ye now to Hezekiah, Thus saith the great king, the king of Assyria, What confidence \emph{is} this wherein thou trustest?"],
['Isaiah','Isa','36','005','5',"I say, \emph{sayest} \emph{thou}, (but \emph{they} \emph{are} \emph{but} vain words) \emph{I} \emph{have} counsel and strength for war: now on whom dost thou trust, that thou rebellest against me?"],
['Isaiah','Isa','36','006','6',"Lo, thou trustest in the staff of this broken reed, on Egypt; whereon if a man lean, it will go into his hand, and pierce it: so \emph{is} Pharaoh king of Egypt to all that trust in him."],
['Isaiah','Isa','36','007','7',"But if thou say to me, We trust in the LORD our God: \emph{is} \emph{it} not he, whose high places and whose altars Hezekiah hath taken away, and said to Judah and to Jerusalem, Ye shall worship before this altar?"],
['Isaiah','Isa','36','008','8',"Now therefore give pledges, I pray thee, to my master the king of Assyria, and I will give thee two thousand horses, if thou be able on thy part to set riders upon them."],
['Isaiah','Isa','36','009','9',"How then wilt thou turn away the face of one captain of the least of my master's servants, and put thy trust on Egypt for chariots and for horsemen?"],
['Isaiah','Isa','36','010','10',"And am I now come up without the LORD against this land to destroy it? the LORD said unto me, Go up against this land, and destroy it."],
['Isaiah','Isa','36','011','11',"Then said Eliakim and Shebna and Joah unto Rabshakeh, Speak, I pray thee, unto thy servants in the Syrian language; for we understand \emph{it}: and speak not to us in the Jews' language, in the ears of the people that \emph{are} on the wall."],
['Isaiah','Isa','36','012','12',"But Rabshakeh said, Hath my master sent me to thy master and to thee to speak these words? \emph{hath} \emph{he} not \emph{sent} \emph{me} to the men that sit upon the wall, that they may eat their own dung, and drink their own piss with you?"],
['Isaiah','Isa','36','013','13',"Then Rabshakeh stood, and cried with a loud voice in the Jews' language, and said, Hear ye the words of the great king, the king of Assyria."],
['Isaiah','Isa','36','014','14',"Thus saith the king, Let not Hezekiah deceive you: for he shall not be able to deliver you."],
['Isaiah','Isa','36','015','15',"Neither let Hezekiah make you trust in the LORD, saying, The LORD will surely deliver us: this city shall not be delivered into the hand of the king of Assyria."],
['Isaiah','Isa','36','016','16',"Hearken not to Hezekiah: for thus saith the king of Assyria, Make \emph{an} \emph{agreement} with me \emph{by} a present, and come out to me: and eat ye every one of his vine, and every one of his fig tree, and drink ye every one the waters of his own cistern;"],
['Isaiah','Isa','36','017','17',"Until I come and take you away to a land like your own land, a land of corn and wine, a land of bread and vineyards."],
['Isaiah','Isa','36','018','18',"\emph{Beware} lest Hezekiah persuade you, saying, The LORD will deliver us. Hath any of the gods of the nations delivered his land out of the hand of the king of Assyria?"],
['Isaiah','Isa','36','019','19',"Where \emph{are} the gods of Hamath and Arphad? where \emph{are} the gods of Sepharvaim? and have they delivered Samaria out of my hand?"],
['Isaiah','Isa','36','020','20',"Who \emph{are} \emph{they} among all the gods of these lands, that have delivered their land out of my hand, that the LORD should deliver Jerusalem out of my hand?"],
['Isaiah','Isa','36','021','21',"But they held their peace, and answered him not a word: for the king's commandment was, saying, Answer him not."],
['Isaiah','Isa','36','022','22',"Then came Eliakim, the son of Hilkiah, that \emph{was} over the household, and Shebna the scribe, and Joah, the son of Asaph, the recorder, to Hezekiah with \emph{their} clothes rent, and told him the words of Rabshakeh."]
]

Isaiah37 = [
['Isaiah','Isa','37','001','1',"And it came to pass, when king Hezekiah heard \emph{it}, that he rent his clothes, and covered himself with sackcloth, and went into the house of the LORD."],
['Isaiah','Isa','37','002','2',"And he sent Eliakim, who \emph{was} over the household, and Shebna the scribe, and the elders of the priests covered with sackcloth, unto Isaiah the prophet the son of Amoz."],
['Isaiah','Isa','37','003','3',"And they said unto him, Thus saith Hezekiah, This day \emph{is} a day of trouble, and of rebuke, and of blasphemy: for the children are come to the birth, and \emph{there} \emph{is} not strength to bring forth."],
['Isaiah','Isa','37','004','4',"It may be the LORD thy God will hear the words of Rabshakeh, whom the king of Assyria his master hath sent to reproach the living God, and will reprove the words which the LORD thy God hath heard: wherefore lift up \emph{thy} prayer for the remnant that is left."],
['Isaiah','Isa','37','005','5',"So the servants of king Hezekiah came to Isaiah."],
['Isaiah','Isa','37','006','6',"And Isaiah said unto them, Thus shall ye say unto your master, Thus saith the LORD, Be not afraid of the words that thou hast heard, wherewith the servants of the king of Assyria have blasphemed me."],
['Isaiah','Isa','37','007','7',"Behold, I will send a blast upon him, and he shall hear a rumour, and return to his own land; and I will cause him to fall by the sword in his own land."],
['Isaiah','Isa','37','008','8',"So Rabshakeh returned, and found the king of Assyria warring against Libnah: for he had heard that he was departed from Lachish."],
['Isaiah','Isa','37','009','9',"And he heard say concerning Tirhakah king of Ethiopia, He is come forth to make war with thee. And when he heard \emph{it}, he sent messengers to Hezekiah, saying,"],
['Isaiah','Isa','37','010','10',"Thus shall ye speak to Hezekiah king of Judah, saying, Let not thy God, in whom thou trustest, deceive thee, saying, Jerusalem shall not be given into the hand of the king of Assyria."],
['Isaiah','Isa','37','011','11',"Behold, thou hast heard what the kings of Assyria have done to all lands by destroying them utterly; and shalt thou be delivered?"],
['Isaiah','Isa','37','012','12',"Have the gods of the nations delivered them which my fathers have destroyed, \emph{as} Gozan, and Haran, and Rezeph, and the children of Eden which \emph{were} in Telassar?"],
['Isaiah','Isa','37','013','13',"Where \emph{is} the king of Hamath, and the king of Arphad, and the king of the city of Sepharvaim, Hena, and Ivah?"],
['Isaiah','Isa','37','014','14',"And Hezekiah received the letter from the hand of the messengers, and read it: and Hezekiah went up unto the house of the LORD, and spread it before the LORD."],
['Isaiah','Isa','37','015','15',"And Hezekiah prayed unto the LORD, saying,"],
['Isaiah','Isa','37','016','16',"O LORD of hosts, God of Israel, that dwellest \emph{between} the cherubims, thou \emph{art} the God, \emph{even} thou alone, of all the kingdoms of the earth: thou hast made heaven and earth."],
['Isaiah','Isa','37','017','17',"Incline thine ear, O LORD, and hear; open thine eyes, O LORD, and see: and hear all the words of Sennacherib, which hath sent to reproach the living God."],
['Isaiah','Isa','37','018','18',"Of a truth, LORD, the kings of Assyria have laid waste all the nations, and their countries,"],
['Isaiah','Isa','37','019','19',"And have cast their gods into the fire: for they \emph{were} no gods, but the work of men's hands, wood and stone: therefore they have destroyed them."],
['Isaiah','Isa','37','020','20',"Now therefore, O LORD our God, save us from his hand, that all the kingdoms of the earth may know that thou \emph{art} the LORD, \emph{even} thou only."],
['Isaiah','Isa','37','021','21',"Then Isaiah the son of Amoz sent unto Hezekiah, saying, Thus saith the LORD God of Israel, Whereas thou hast prayed to me against Sennacherib king of Assyria:"],
['Isaiah','Isa','37','022','22',"This \emph{is} the word which the LORD hath spoken concerning him; The virgin, the daughter of Zion, hath despised thee, \emph{and} laughed thee to scorn; the daughter of Jerusalem hath shaken her head at thee."],
['Isaiah','Isa','37','023','23',"Whom hast thou reproached and blasphemed? and against whom hast thou exalted \emph{thy} voice, and lifted up thine eyes on high? \emph{even} against the Holy One of Israel."],
['Isaiah','Isa','37','024','24',"By thy servants hast thou reproached the Lord, and hast said, By the multitude of my chariots am I come up to the height of the mountains, to the sides of Lebanon; and I will cut down the tall cedars thereof, \emph{and} the choice fir trees thereof: and I will enter into the height of his border, \emph{and} the forest of his Carmel."],
['Isaiah','Isa','37','025','25',"I have digged, and drunk water; and with the sole of my feet have I dried up all the rivers of the besieged places."],
['Isaiah','Isa','37','026','26',"Hast thou not heard long ago, \emph{how} I have done it; \emph{and} of ancient times, that I have formed it? now have I brought it to pass, that thou shouldest be to lay waste defenced cities \emph{into} ruinous heaps."],
['Isaiah','Isa','37','027','27',"Therefore their inhabitants \emph{were} of small power, they were dismayed and confounded: they were \emph{as} the grass of the field, and \emph{as} the green herb, \emph{as} the grass on the housetops, and \emph{as} \emph{corn} blasted before it be grown up."],
['Isaiah','Isa','37','028','28',"But I know thy abode, and thy going out, and thy coming in, and thy rage against me."],
['Isaiah','Isa','37','029','29',"Because thy rage against me, and thy tumult, is come up into mine ears, therefore will I put my hook in thy nose, and my bridle in thy lips, and I will turn thee back by the way by which thou camest."],
['Isaiah','Isa','37','030','30',"And this \emph{shall} \emph{be} a sign unto thee, Ye shall eat \emph{this} year such as groweth of itself; and the second year that which springeth of the same: and in the third year sow ye, and reap, and plant vineyards, and eat the fruit thereof."],
['Isaiah','Isa','37','031','31',"And the remnant that is escaped of the house of Judah shall again take root downward, and bear fruit upward:"],
['Isaiah','Isa','37','032','32',"For out of Jerusalem shall go forth a remnant, and they that escape out of mount Zion: the zeal of the LORD of hosts shall do this."],
['Isaiah','Isa','37','033','33',"Therefore thus saith the LORD concerning the king of Assyria, He shall not come into this city, nor shoot an arrow there, nor come before it with shields, nor cast a bank against it."],
['Isaiah','Isa','37','034','34',"By the way that he came, by the same shall he return, and shall not come into this city, saith the LORD."],
['Isaiah','Isa','37','035','35',"For I will defend this city to save it for mine own sake, and for my servant David's sake."],
['Isaiah','Isa','37','036','36',"Then the angel of the LORD went forth, and smote in the camp of the Assyrians a hundred and fourscore and five thousand: and when they arose early in the morning, behold, they \emph{were} all dead corpses."],
['Isaiah','Isa','37','037','37',"So Sennacherib king of Assyria departed, and went and returned, and dwelt at Nineveh."],
['Isaiah','Isa','37','038','38',"And it came to pass, as he was worshipping in the house of Nisroch his god, that Adrammelech and Sharezer his sons smote him with the sword; and they escaped into the land of Armenia: and Esar-haddon his son reigned in his stead."]
]

Isaiah38 = [
['Isaiah','Isa','38','001','1',"In those days was Hezekiah sick unto death. And Isaiah the prophet the son of Amoz came unto him, and said unto him, Thus saith the LORD, Set thine house in order: for thou shalt die, and not live."],
['Isaiah','Isa','38','002','2',"Then Hezekiah turned his face toward the wall, and prayed unto the LORD,"],
['Isaiah','Isa','38','003','3',"And said, Remember now, O LORD, I beseech thee, how I have walked before thee in truth and with a perfect heart, and have done \emph{that} \emph{which} \emph{is} good in thy sight. And Hezekiah wept sore."],
['Isaiah','Isa','38','004','4',"Then came the word of the LORD to Isaiah, saying,"],
['Isaiah','Isa','38','005','5',"Go, and say to Hezekiah, Thus saith the LORD, the God of David thy father, I have heard thy prayer, I have seen thy tears: behold, I will add unto thy days fifteen years."],
['Isaiah','Isa','38','006','6',"And I will deliver thee and this city out of the hand of the king of Assyria: and I will defend this city."],
['Isaiah','Isa','38','007','7',"And this \emph{shall} \emph{be} a sign unto thee from the LORD, that the LORD will do this thing that he hath spoken;"],
['Isaiah','Isa','38','008','8',"Behold, I will bring again the shadow of the degrees, which is gone down in the sun dial of Ahaz, ten degrees backward. So the sun returned ten degrees, by which degrees it was gone down."],
['Isaiah','Isa','38','009','9',"The writing of Hezekiah king of Judah, when he had been sick, and was recovered of his sickness:"],
['Isaiah','Isa','38','010','10',"I said in the cutting off of my days, I shall go to the gates of the grave: I am deprived of the residue of my years."],
['Isaiah','Isa','38','011','11',"I said, I shall not see the LORD, \emph{even} the LORD, in the land of the living: I shall behold man no more with the inhabitants of the world."],
['Isaiah','Isa','38','012','12',"Mine age is departed, and is removed from me as a shepherd's tent: I have cut off like a weaver my life: he will cut me off with pining sickness: from day \emph{even} to night wilt thou make an end of me."],
['Isaiah','Isa','38','013','13',"I reckoned till morning, \emph{that}, as a lion, so will he break all my bones: from day \emph{even} to night wilt thou make an end of me."],
['Isaiah','Isa','38','014','14',"Like a crane \emph{or} a swallow, so did I chatter: I did mourn as a dove: mine eyes fail \emph{with} \emph{looking} upward: O LORD, I am oppressed; undertake for me."],
['Isaiah','Isa','38','015','15',"What shall I say? he hath both spoken unto me, and himself hath done \emph{it}: I shall go softly all my years in the bitterness of my soul."],
['Isaiah','Isa','38','016','16',"O Lord, by these \emph{things} \emph{men} live, and in all these \emph{things} \emph{is} the life of my spirit: so wilt thou recover me, and make me to live."],
['Isaiah','Isa','38','017','17',"Behold, for peace I had great bitterness: but thou hast in love to my soul \emph{delivered} \emph{it} from the pit of corruption: for thou hast cast all my sins behind thy back."],
['Isaiah','Isa','38','018','18',"For the grave cannot praise thee, death can \emph{not} celebrate thee: they that go down into the pit cannot hope for thy truth."],
['Isaiah','Isa','38','019','19',"The living, the living, he shall praise thee, as I \emph{do} this day: the father to the children shall make known thy truth."],
['Isaiah','Isa','38','020','20',"The LORD \emph{was} \emph{ready} to save me: therefore we will sing my songs to the stringed instruments all the days of our life in the house of the LORD."],
['Isaiah','Isa','38','021','21',"For Isaiah had said, Let them take a lump of figs, and lay \emph{it} for a plaister upon the boil, and he shall recover."],
['Isaiah','Isa','38','022','22',"Hezekiah also had said, What \emph{is} the sign that I shall go up to the house of the LORD?"]
]

Isaiah39 = [
['Isaiah','Isa','39','001','1',"At that time Merodach-baladan, the son of Baladan, king of Babylon, sent letters and a present to Hezekiah: for he had heard that he had been sick, and was recovered."],
['Isaiah','Isa','39','002','2',"And Hezekiah was glad of them, and shewed them the house of his precious things, the silver, and the gold, and the spices, and the precious ointment, and all the house of his armour, and all that was found in his treasures: there was nothing in his house, nor in all his dominion, that Hezekiah shewed them not."],
['Isaiah','Isa','39','003','3',"Then came Isaiah the prophet unto king Hezekiah, and said unto him, What said these men? and from whence came they unto thee? And Hezekiah said, They are come from a far country unto me, \emph{even} from Babylon."],
['Isaiah','Isa','39','004','4',"Then said he, What have they seen in thine house? And Hezekiah answered, All that \emph{is} in mine house have they seen: there is nothing among my treasures that I have not shewed them."],
['Isaiah','Isa','39','005','5',"Then said Isaiah to Hezekiah, Hear the word of the LORD of hosts:"],
['Isaiah','Isa','39','006','6',"Behold, the days come, that all that \emph{is} in thine house, and \emph{that} which thy fathers have laid up in store until this day, shall be carried to Babylon: nothing shall be left, saith the LORD."],
['Isaiah','Isa','39','007','7',"And of thy sons that shall issue from thee, which thou shalt beget, shall they take away; and they shall be eunuchs in the palace of the king of Babylon."],
['Isaiah','Isa','39','008','8',"Then said Hezekiah to Isaiah, Good \emph{is} the word of the LORD which thou hast spoken. He said moreover, For there shall be peace and truth in my days."]
]

Isaiah40 = [
['Isaiah','Isa','40','001','1',"Comfort ye, comfort ye my people, saith your God."],
['Isaiah','Isa','40','002','2',"Speak ye comfortably to Jerusalem, and cry unto her, that her warfare is accomplished, that her iniquity is pardoned: for she hath received of the LORD'S hand double for all her sins."],
['Isaiah','Isa','40','003','3',"The voice of him that crieth in the wilderness, Prepare ye the way of the LORD, make straight in the desert a highway for our God."],
['Isaiah','Isa','40','004','4',"Every valley shall be exalted, and every mountain and hill shall be made low: and the crooked shall be made straight, and the rough places plain:"],
['Isaiah','Isa','40','005','5',"And the glory of the LORD shall be revealed, and all flesh shall see \emph{it} together: for the mouth of the LORD hath spoken \emph{it}."],
['Isaiah','Isa','40','006','6',"The voice said, Cry. And he said, What shall I cry? All flesh \emph{is} grass, and all the goodliness thereof \emph{is} as the flower of the field:"],
['Isaiah','Isa','40','007','7',"The grass withereth, the flower fadeth: because the spirit of the LORD bloweth upon it: surely the people \emph{is} grass."],
['Isaiah','Isa','40','008','8',"The grass withereth, the flower fadeth: but the word of our God shall stand for ever."],
['Isaiah','Isa','40','009','9',"O Zion, that bringest good tidings, get thee up into the high mountain; O Jerusalem, that bringest good tidings, lift up thy voice with strength; lift \emph{it} up, be not afraid; say unto the cities of Judah, Behold your God!"],
['Isaiah','Isa','40','010','10',"Behold, the Lord GOD will come with strong \emph{hand}, and his arm shall rule for him: behold, his reward \emph{is} with him, and his work before him."],
['Isaiah','Isa','40','011','11',"He shall feed his flock like a shepherd: he shall gather the lambs with his arm, and carry \emph{them} in his bosom, \emph{and} shall gently lead those that are with young."],
['Isaiah','Isa','40','012','12',"Who hath measured the waters in the hollow of his hand, and meted out heaven with the span, and comprehended the dust of the earth in a measure, and weighed the mountains in scales, and the hills in a balance?"],
['Isaiah','Isa','40','013','13',"Who hath directed the Spirit of the LORD, or \emph{being} his counsellor hath taught him?"],
['Isaiah','Isa','40','014','14',"With whom took he counsel, and \emph{who} instructed him, and taught him in the path of judgment, and taught him knowledge, and shewed to him the way of understanding?"],
['Isaiah','Isa','40','015','15',"Behold, the nations \emph{are} as a drop of a bucket, and are counted as the small dust of the balance: behold, he taketh up the isles as a very little thing."],
['Isaiah','Isa','40','016','16',"And Lebanon \emph{is} not sufficient to burn, nor the beasts thereof sufficient for a burnt offering."],
['Isaiah','Isa','40','017','17',"All nations before him \emph{are} as nothing; and they are counted to him less than nothing, and vanity."],
['Isaiah','Isa','40','018','18',"To whom then will ye liken God? or what likeness will ye compare unto him?"],
['Isaiah','Isa','40','019','19',"The workman melteth a graven image, and the goldsmith spreadeth it over with gold, and casteth silver chains."],
['Isaiah','Isa','40','020','20',"He that \emph{is} so impoverished that he hath no oblation chooseth a tree \emph{that} will not rot; he seeketh unto him a cunning workman to prepare a graven image, \emph{that} shall not be moved."],
['Isaiah','Isa','40','021','21',"Have ye not known? have ye not heard? hath it not been told you from the beginning? have ye not understood from the foundations of the earth?"],
['Isaiah','Isa','40','022','22',"\emph{It} \emph{is} he that sitteth upon the circle of the earth, and the inhabitants thereof \emph{are} as grasshoppers; that stretcheth out the heavens as a curtain, and spreadeth them out as a tent to dwell in:"],
['Isaiah','Isa','40','023','23',"That bringeth the princes to nothing; he maketh the judges of the earth as vanity."],
['Isaiah','Isa','40','024','24',"Yea, they shall not be planted; yea, they shall not be sown: yea, their stock shall not take root in the earth: and he shall also blow upon them, and they shall wither, and the whirlwind shall take them away as stubble."],
['Isaiah','Isa','40','025','25',"To whom then will ye liken me, or shall I be equal? saith the Holy One."],
['Isaiah','Isa','40','026','26',"Lift up your eyes on high, and behold who hath created these \emph{things}, that bringeth out their host by number: he calleth them all by names by the greatness of his might, for that \emph{he} \emph{is} strong in power; not one faileth."],
['Isaiah','Isa','40','027','27',"Why sayest thou, O Jacob, and speakest, O Israel, My way is hid from the LORD, and my judgment is passed over from my God?"],
['Isaiah','Isa','40','028','28',"Hast thou not known? hast thou not heard, \emph{that} the everlasting God, the LORD, the Creator of the ends of the earth, fainteth not, neither is weary? \emph{there} \emph{is} no searching of his understanding."],
['Isaiah','Isa','40','029','29',"He giveth power to the faint; and to \emph{them} \emph{that} \emph{have} no might he increaseth strength."],
['Isaiah','Isa','40','030','30',"Even the youths shall faint and be weary, and the young men shall utterly fall:"],
['Isaiah','Isa','40','031','31',"But they that wait upon the LORD shall renew \emph{their} strength; they shall mount up with wings as eagles; they shall run, and not be weary; \emph{and} they shall walk, and not faint."]
]

Isaiah41 = [
['Isaiah','Isa','41','001','1',"Keep silence before me, O islands; and let the people renew \emph{their} strength: let them come near; then let them speak: let us come near together to judgment."],
['Isaiah','Isa','41','002','2',"Who raised up the righteous \emph{man} from the east, called him to his foot, gave the nations before him, and made \emph{him} rule over kings? he gave \emph{them} as the dust to his sword, \emph{and} as driven stubble to his bow."],
['Isaiah','Isa','41','003','3',"He pursued them, \emph{and} passed safely; \emph{even} by the way \emph{that} he had not gone with his feet."],
['Isaiah','Isa','41','004','4',"Who hath wrought and done \emph{it}, calling the generations from the beginning? I the LORD, the first, and with the last; I \emph{am} he."],
['Isaiah','Isa','41','005','5',"The isles saw \emph{it}, and feared; the ends of the earth were afraid, drew near, and came."],
['Isaiah','Isa','41','006','6',"They helped every one his neighbour; and \emph{every} \emph{one} said to his brother, Be of good courage."],
['Isaiah','Isa','41','007','7',"So the carpenter encouraged the goldsmith, \emph{and} he that smootheth \emph{with} the hammer him that smote the anvil, saying, It \emph{is} ready for the sodering: and he fastened it with nails, \emph{that} it should not be moved."],
['Isaiah','Isa','41','008','8',"But thou, Israel, \emph{art} my servant, Jacob whom I have chosen, the seed of Abraham my friend."],
['Isaiah','Isa','41','009','9',"\emph{Thou} whom I have taken from the ends of the earth, and called thee from the chief men thereof, and said unto thee, Thou \emph{art} my servant; I have chosen thee, and not cast thee away."],
['Isaiah','Isa','41','010','10',"Fear thou not; for I \emph{am} with thee: be not dismayed; for I \emph{am} thy God: I will strengthen thee; yea, I will help thee; yea, I will uphold thee with the right hand of my righteousness."],
['Isaiah','Isa','41','011','11',"Behold, all they that were incensed against thee shall be ashamed and confounded: they shall be as nothing; and they that strive with thee shall perish."],
['Isaiah','Isa','41','012','12',"Thou shalt seek them, and shalt not find them, \emph{even} them that contended with thee: they that war against thee shall be as nothing, and as a thing of nought."],
['Isaiah','Isa','41','013','13',"For I the LORD thy God will hold thy right hand, saying unto thee, Fear not; I will help thee."],
['Isaiah','Isa','41','014','14',"Fear not, thou worm Jacob, \emph{and} ye men of Israel; I will help thee, saith the LORD, and thy redeemer, the Holy One of Israel."],
['Isaiah','Isa','41','015','15',"Behold, I will make thee a new sharp threshing instrument having teeth: thou shalt thresh the mountains, and beat \emph{them} small, and shalt make the hills as chaff."],
['Isaiah','Isa','41','016','16',"Thou shalt fan them, and the wind shall carry them away, and the whirlwind shall scatter them: and thou shalt rejoice in the LORD, \emph{and} shalt glory in the Holy One of Israel."],
['Isaiah','Isa','41','017','17',"\emph{When} the poor and needy seek water, and \emph{there} \emph{is} none, \emph{and} their tongue faileth for thirst, I the LORD will hear them, I the God of Israel will not forsake them."],
['Isaiah','Isa','41','018','18',"I will open rivers in high places, and fountains in the midst of the valleys: I will make the wilderness a pool of water, and the dry land springs of water."],
['Isaiah','Isa','41','019','19',"I will plant in the wilderness the cedar, the shittah tree, and the myrtle, and the oil tree; I will set in the desert the fir tree, \emph{and} the pine, and the box tree together:"],
['Isaiah','Isa','41','020','20',"That they may see, and know, and consider, and understand together, that the hand of the LORD hath done this, and the Holy One of Israel hath created it."],
['Isaiah','Isa','41','021','21',"Produce your cause, saith the LORD; bring forth your strong \emph{reasons}, saith the King of Jacob."],
['Isaiah','Isa','41','022','22',"Let them bring \emph{them} forth, and shew us what shall happen: let them shew the former things, what they \emph{be}, that we may consider them, and know the latter end of them; or declare us things for to come."],
['Isaiah','Isa','41','023','23',"Shew the things that are to come hereafter, that we may know that ye \emph{are} gods: yea, do good, or do evil, that we may be dismayed, and behold \emph{it} together."],
['Isaiah','Isa','41','024','24',"Behold, ye \emph{are} of nothing, and your work of nought: an abomination \emph{is} \emph{he} \emph{that} chooseth you."],
['Isaiah','Isa','41','025','25',"I have raised up \emph{one} from the north, and he shall come: from the rising of the sun shall he call upon my name: and he shall come upon princes as \emph{upon} morter, and as the potter treadeth clay."],
['Isaiah','Isa','41','026','26',"Who hath declared from the beginning, that we may know? and beforetime, that we may say, \emph{He} \emph{is} righteous? yea, \emph{there} \emph{is} none that sheweth, yea, \emph{there} \emph{is} none that declareth, yea, \emph{there} \emph{is} none that heareth your words."],
['Isaiah','Isa','41','027','27',"The first \emph{shall} \emph{say} to Zion, Behold, behold them: and I will give to Jerusalem one that bringeth good tidings."],
['Isaiah','Isa','41','028','28',"For I beheld, and \emph{there} \emph{was} no man; even among them, and \emph{there} \emph{was} no counsellor, that, when I asked of them, could answer a word."],
['Isaiah','Isa','41','029','29',"Behold, they \emph{are} all vanity; their works \emph{are} nothing: their molten images \emph{are} wind and confusion."]
]

Isaiah42 = [
['Isaiah','Isa','42','001','1',"Behold my servant, whom I uphold; mine elect, \emph{in} \emph{whom} my soul delighteth; I have put my spirit upon him: he shall bring forth judgment to the Gentiles."],
['Isaiah','Isa','42','002','2',"He shall not cry, nor lift up, nor cause his voice to be heard in the street."],
['Isaiah','Isa','42','003','3',"A bruised reed shall he not break, and the smoking flax shall he not quench: he shall bring forth judgment unto truth."],
['Isaiah','Isa','42','004','4',"He shall not fail nor be discouraged, till he have set judgment in the earth: and the isles shall wait for his law."],
['Isaiah','Isa','42','005','5',"Thus saith God the LORD, he that created the heavens, and stretched them out; he that spread forth the earth, and that which cometh out of it; he that giveth breath unto the people upon it, and spirit to them that walk therein:"],
['Isaiah','Isa','42','006','6',"I the LORD have called thee in righteousness, and will hold thine hand, and will keep thee, and give thee for a covenant of the people, for a light of the Gentiles;"],
['Isaiah','Isa','42','007','7',"To open the blind eyes, to bring out the prisoners from the prison, \emph{and} them that sit in darkness out of the prison house."],
['Isaiah','Isa','42','008','8',"I \emph{am} the LORD: that \emph{is} my name: and my glory will I not give to another, neither my praise to graven images."],
['Isaiah','Isa','42','009','9',"Behold, the former things are come to pass, and new things do I declare: before they spring forth I tell you of them."],
['Isaiah','Isa','42','010','10',"Sing unto the LORD a new song, \emph{and} his praise from the end of the earth, ye that go down to the sea, and all that is therein; the isles, and the inhabitants thereof."],
['Isaiah','Isa','42','011','11',"Let the wilderness and the cities thereof lift up \emph{their} \emph{voice}, the villages \emph{that} Kedar doth inhabit: let the inhabitants of the rock sing, let them shout from the top of the mountains."],
['Isaiah','Isa','42','012','12',"Let them give glory unto the LORD, and declare his praise in the islands."],
['Isaiah','Isa','42','013','13',"The LORD shall go forth as a mighty man, he shall stir up jealousy like a man of war: he shall cry, yea, roar; he shall prevail against his enemies."],
['Isaiah','Isa','42','014','14',"I have long time holden my peace; I have been still, \emph{and} refrained myself: \emph{now} will I cry like a travailing woman; I will destroy and devour at once."],
['Isaiah','Isa','42','015','15',"I will make waste mountains and hills, and dry up all their herbs; and I will make the rivers islands, and I will dry up the pools."],
['Isaiah','Isa','42','016','16',"And I will bring the blind by a way \emph{that} they knew not; I will lead them in paths \emph{that} they have not known: I will make darkness light before them, and crooked things straight. These things will I do unto them, and not forsake them."],
['Isaiah','Isa','42','017','17',"They shall be turned back, they shall be greatly ashamed, that trust in graven images, that say to the molten images, Ye \emph{are} our gods."],
['Isaiah','Isa','42','018','18',"Hear, ye deaf; and look, ye blind, that ye may see."],
['Isaiah','Isa','42','019','19',"Who \emph{is} blind, but my servant? or deaf, as my messenger \emph{that} I sent? who \emph{is} blind as \emph{he} \emph{that} \emph{is} perfect, and blind as the LORD'S servant?"],
['Isaiah','Isa','42','020','20',"Seeing many things, but thou observest not; opening the ears, but he heareth not."],
['Isaiah','Isa','42','021','21',"The LORD is well pleased for his righteousness' sake; he will magnify the law, and make \emph{it} honourable."],
['Isaiah','Isa','42','022','22',"But this \emph{is} a people robbed and spoiled; \emph{they} \emph{are} all of them snared in holes, and they are hid in prison houses: they are for a prey, and none delivereth; for a spoil, and none saith, Restore."],
['Isaiah','Isa','42','023','23',"Who among you will give ear to this? \emph{who} will hearken and hear for the time to come?"],
['Isaiah','Isa','42','024','24',"Who gave Jacob for a spoil, and Israel to the robbers? did not the LORD, he against whom we have sinned? for they would not walk in his ways, neither were they obedient unto his law."],
['Isaiah','Isa','42','025','25',"Therefore he hath poured upon him the fury of his anger, and the strength of battle: and it hath set him on fire round about, yet he knew not; and it burned him, yet he laid \emph{it} not to heart."]
]

Isaiah43 = [
['Isaiah','Isa','43','001','1',"But now thus saith the LORD that created thee, O Jacob, and he that formed thee, O Israel, Fear not: for I have redeemed thee, I have called \emph{thee} by thy name; thou \emph{art} mine."],
['Isaiah','Isa','43','002','2',"When thou passest through the waters, I \emph{will} \emph{be} with thee; and through the rivers, they shall not overflow thee: when thou walkest through the fire, thou shalt not be burned; neither shall the flame kindle upon thee."],
['Isaiah','Isa','43','003','3',"For I \emph{am} the LORD thy God, the Holy One of Israel, thy Saviour: I gave Egypt \emph{for} thy ransom, Ethiopia and Seba for thee."],
['Isaiah','Isa','43','004','4',"Since thou wast precious in my sight, thou hast been honourable, and I have loved thee: therefore will I give men for thee, and people for thy life."],
['Isaiah','Isa','43','005','5',"Fear not: for I \emph{am} with thee: I will bring thy seed from the east, and gather thee from the west;"],
['Isaiah','Isa','43','006','6',"I will say to the north, Give up; and to the south, Keep not back: bring my sons from far, and my daughters from the ends of the earth;"],
['Isaiah','Isa','43','007','7',"\emph{Even} every one that is called by my name: for I have created him for my glory, I have formed him; yea, I have made him."],
['Isaiah','Isa','43','008','8',"Bring forth the blind people that have eyes, and the deaf that have ears."],
['Isaiah','Isa','43','009','9',"Let all the nations be gathered together, and let the people be assembled: who among them can declare this, and shew us former things? let them bring forth their witnesses, that they may be justified: or let them hear, and say, \emph{It} \emph{is} truth."],
['Isaiah','Isa','43','010','10',"Ye \emph{are} my witnesses, saith the LORD, and my servant whom I have chosen: that ye may know and believe me, and understand that I \emph{am} he: before me there was no God formed, neither shall there be after me."],
['Isaiah','Isa','43','011','11',"I, \emph{even} I, \emph{am} the LORD; and beside me \emph{there} \emph{is} no saviour."],
['Isaiah','Isa','43','012','12',"I have declared, and have saved, and I have shewed, when \emph{there} \emph{was} no strange \emph{god} among you: therefore ye \emph{are} my witnesses, saith the LORD, that I \emph{am} God."],
['Isaiah','Isa','43','013','13',"Yea, before the day \emph{was} I \emph{am} he; and \emph{there} \emph{is} none that can deliver out of my hand: I will work, and who shall let it?"],
['Isaiah','Isa','43','014','14',"Thus saith the LORD, your redeemer, the Holy One of Israel; For your sake I have sent to Babylon, and have brought down all their nobles, and the Chaldeans, whose cry \emph{is} in the ships."],
['Isaiah','Isa','43','015','15',"I \emph{am} the LORD, your Holy One, the creator of Israel, your King."],
['Isaiah','Isa','43','016','16',"Thus saith the LORD, which maketh a way in the sea, and a path in the mighty waters;"],
['Isaiah','Isa','43','017','17',"Which bringeth forth the chariot and horse, the army and the power; they shall lie down together, they shall not rise: they are extinct, they are quenched as tow."],
['Isaiah','Isa','43','018','18',"Remember ye not the former things, neither consider the things of old."],
['Isaiah','Isa','43','019','19',"Behold, I will do a new thing; now it shall spring forth; shall ye not know it? I will even make a way in the wilderness, \emph{and} rivers in the desert."],
['Isaiah','Isa','43','020','20',"The beast of the field shall honour me, the dragons and the owls: because I give waters in the wilderness, \emph{and} rivers in the desert, to give drink to my people, my chosen."],
['Isaiah','Isa','43','021','21',"This people have I formed for myself; they shall shew forth my praise."],
['Isaiah','Isa','43','022','22',"But thou hast not called upon me, O Jacob; but thou hast been weary of me, O Israel."],
['Isaiah','Isa','43','023','23',"Thou hast not brought me the small cattle of thy burnt offerings; neither hast thou honoured me with thy sacrifices. I have not caused thee to serve with an offering, nor wearied thee with incense."],
['Isaiah','Isa','43','024','24',"Thou hast bought me no sweet cane with money, neither hast thou filled me with the fat of thy sacrifices: but thou hast made me to serve with thy sins, thou hast wearied me with thine iniquities."],
['Isaiah','Isa','43','025','25',"I, \emph{even} I, \emph{am} he that blotteth out thy transgressions for mine own sake, and will not remember thy sins."],
['Isaiah','Isa','43','026','26',"Put me in remembrance: let us plead together: declare thou, that thou mayest be justified."],
['Isaiah','Isa','43','027','27',"Thy first father hath sinned, and thy teachers have transgressed against me."],
['Isaiah','Isa','43','028','28',"Therefore I have profaned the princes of the sanctuary, and have given Jacob to the curse, and Israel to reproaches."]
]

Isaiah44 = [
['Isaiah','Isa','44','001','1',"Yet now hear, O Jacob my servant; and Israel, whom I have chosen:"],
['Isaiah','Isa','44','002','2',"Thus saith the LORD that made thee, and formed thee from the womb, \emph{which} will help thee; Fear not, O Jacob, my servant; and thou, Jesurun, whom I have chosen."],
['Isaiah','Isa','44','003','3',"For I will pour water upon him that is thirsty, and floods upon the dry ground: I will pour my spirit upon thy seed, and my blessing upon thine offspring:"],
['Isaiah','Isa','44','004','4',"And they shall spring up \emph{as} among the grass, as willows by the water courses."],
['Isaiah','Isa','44','005','5',"One shall say, I \emph{am} the LORD'S; and another shall call \emph{himself} by the name of Jacob; and another shall subscribe \emph{with} his hand unto the LORD, and surname \emph{himself} by the name of Israel."],
['Isaiah','Isa','44','006','6',"Thus saith the LORD the King of Israel, and his redeemer the LORD of hosts; I \emph{am} the first, and I \emph{am} the last; and beside me \emph{there} \emph{is} no God."],
['Isaiah','Isa','44','007','7',"And who, as I, shall call, and shall declare it, and set it in order for me, since I appointed the ancient people? and the things that are coming, and shall come, let them shew unto them."],
['Isaiah','Isa','44','008','8',"Fear ye not, neither be afraid: have not I told thee from that time, and have declared \emph{it}? ye \emph{are} even my witnesses. Is there a God beside me? yea, \emph{there} \emph{is} no God; I know not \emph{any}."],
['Isaiah','Isa','44','009','9',"They that make a graven image \emph{are} all of them vanity; and their delectable things shall not profit; and they \emph{are} their own witnesses; they see not, nor know; that they may be ashamed."],
['Isaiah','Isa','44','010','10',"Who hath formed a god, or molten a graven image \emph{that} is profitable for nothing?"],
['Isaiah','Isa','44','011','11',"Behold, all his fellows shall be ashamed: and the workmen, they \emph{are} of men: let them all be gathered together, let them stand up; \emph{yet} they shall fear, \emph{and} they shall be ashamed together."],
['Isaiah','Isa','44','012','12',"The smith with the tongs both worketh in the coals, and fashioneth it with hammers, and worketh it with the strength of his arms: yea, he is hungry, and his strength faileth: he drinketh no water, and is faint."],
['Isaiah','Isa','44','013','13',"The carpenter stretcheth out \emph{his} rule; he marketh it out with a line; he fitteth it with planes, and he marketh it out with the compass, and maketh it after the figure of a man, according to the beauty of a man; that it may remain in the house."],
['Isaiah','Isa','44','014','14',"He heweth him down cedars, and taketh the cypress and the oak, which he strengtheneth for himself among the trees of the forest: he planteth an ash, and the rain doth nourish \emph{it}."],
['Isaiah','Isa','44','015','15',"Then shall it be for a man to burn: for he will take thereof, and warm himself; yea, he kindleth \emph{it}, and baketh bread; yea, he maketh a god, and worshippeth \emph{it}; he maketh it a graven image, and falleth down thereto."],
['Isaiah','Isa','44','016','16',"He burneth part thereof in the fire; with part thereof he eateth flesh; he roasteth roast, and is satisfied: yea, he warmeth \emph{himself}, and saith, Aha, I am warm, I have seen the fire:"],
['Isaiah','Isa','44','017','17',"And the residue thereof he maketh a god, \emph{even} his graven image: he falleth down unto it, and worshippeth \emph{it}, and prayeth unto it, and saith, Deliver me; for thou \emph{art} my god."],
['Isaiah','Isa','44','018','18',"They have not known nor understood: for he hath shut their eyes, that they cannot see; \emph{and} their hearts, that they cannot understand."],
['Isaiah','Isa','44','019','19',"And none considereth in his heart, neither \emph{is} \emph{there} knowledge nor understanding to say, I have burned part of it in the fire; yea, also I have baked bread upon the coals thereof; I have roasted flesh, and eaten \emph{it}: and shall I make the residue thereof an abomination? shall I fall down to the stock of a tree?"],
['Isaiah','Isa','44','020','20',"He feedeth on ashes: a deceived heart hath turned him aside, that he cannot deliver his soul, nor say, \emph{Is} \emph{there} not a lie in my right hand?"],
['Isaiah','Isa','44','021','21',"Remember these, O Jacob and Israel; for thou \emph{art} my servant: I have formed thee; thou \emph{art} my servant: O Israel, thou shalt not be forgotten of me."],
['Isaiah','Isa','44','022','22',"I have blotted out, as a thick cloud, thy transgressions, and, as a cloud, thy sins: return unto me; for I have redeemed thee."],
['Isaiah','Isa','44','023','23',"Sing, O ye heavens; for the LORD hath done \emph{it}: shout, ye lower parts of the earth: break forth into singing, ye mountains, O forest, and every tree therein: for the LORD hath redeemed Jacob, and glorified himself in Israel."],
['Isaiah','Isa','44','024','24',"Thus saith the LORD, thy redeemer, and he that formed thee from the womb, I \emph{am} the LORD that maketh all \emph{things}; that stretcheth forth the heavens alone; that spreadeth abroad the earth by myself;"],
['Isaiah','Isa','44','025','25',"That frustrateth the tokens of the liars, and maketh diviners mad; that turneth wise \emph{men} backward, and maketh their knowledge foolish;"],
['Isaiah','Isa','44','026','26',"That confirmeth the word of his servant, and performeth the counsel of his messengers; that saith to Jerusalem, Thou shalt be inhabited; and to the cities of Judah, Ye shall be built, and I will raise up the decayed places thereof:"],
['Isaiah','Isa','44','027','27',"That saith to the deep, Be dry, and I will dry up thy rivers:"],
['Isaiah','Isa','44','028','28',"That saith of Cyrus, \emph{He} \emph{is} my shepherd, and shall perform all my pleasure: even saying to Jerusalem, Thou shalt be built; and to the temple, Thy foundation shall be laid."]
]

Isaiah45 = [
['Isaiah','Isa','45','001','1',"Thus saith the LORD to his anointed, to Cyrus, whose right hand I have holden, to subdue nations before him; and I will loose the loins of kings, to open before him the two leaved gates; and the gates shall not be shut;"],
['Isaiah','Isa','45','002','2',"I will go before thee, and make the crooked places straight: I will break in pieces the gates of brass, and cut in sunder the bars of iron:"],
['Isaiah','Isa','45','003','3',"And I will give thee the treasures of darkness, and hidden riches of secret places, that thou mayest know that I, the LORD, which call \emph{thee} by thy name, \emph{am} the God of Israel."],
['Isaiah','Isa','45','004','4',"For Jacob my servant's sake, and Israel mine elect, I have even called thee by thy name: I have surnamed thee, though thou hast not known me."],
['Isaiah','Isa','45','005','5',"I \emph{am} the LORD, and \emph{there} \emph{is} none else, \emph{there} \emph{is} no God beside me: I girded thee, though thou hast not known me:"],
['Isaiah','Isa','45','006','6',"That they may know from the rising of the sun, and from the west, that \emph{there} \emph{is} none beside me. I \emph{am} the LORD, and \emph{there} \emph{is} none else."],
['Isaiah','Isa','45','007','7',"I form the light, and create darkness: I make peace, and create evil: I the LORD do all these \emph{things}."],
['Isaiah','Isa','45','008','8',"Drop down, ye heavens, from above, and let the skies pour down righteousness: let the earth open, and let them bring forth salvation, and let righteousness spring up together; I the LORD have created it."],
['Isaiah','Isa','45','009','9',"Woe unto him that striveth with his Maker! \emph{Let} the potsherd \emph{strive} with the potsherds of the earth. Shall the clay say to him that fashioneth it, What makest thou? or thy work, He hath no hands?"],
['Isaiah','Isa','45','010','10',"Woe unto him that saith unto \emph{his} father, What begettest thou? or to the woman, What hast thou brought forth?"],
['Isaiah','Isa','45','011','11',"Thus saith the LORD, the Holy One of Israel, and his Maker, Ask me of things to come concerning my sons, and concerning the work of my hands command ye me."],
['Isaiah','Isa','45','012','12',"I have made the earth, and created man upon it: I, \emph{even} my hands, have stretched out the heavens, and all their host have I commanded."],
['Isaiah','Isa','45','013','13',"I have raised him up in righteousness, and I will direct all his ways: he shall build my city, and he shall let go my captives, not for price nor reward, saith the LORD of hosts."],
['Isaiah','Isa','45','014','14',"Thus saith the LORD, The labour of Egypt, and merchandise of Ethiopia and of the Sabeans, men of stature, shall come over unto thee, and they shall be thine: they shall come after thee; in chains they shall come over, and they shall fall down unto thee, they shall make supplication unto thee, \emph{saying}, Surely God \emph{is} in thee; and \emph{there} \emph{is} none else, \emph{there} \emph{is} no God."],
['Isaiah','Isa','45','015','15',"Verily thou \emph{art} a God that hidest thyself, O God of Israel, the Saviour."],
['Isaiah','Isa','45','016','16',"They shall be ashamed, and also confounded, all of them: they shall go to confusion together \emph{that} \emph{are} makers of idols."],
['Isaiah','Isa','45','017','17',"\emph{But} Israel shall be saved in the LORD with an everlasting salvation: ye shall not be ashamed nor confounded world without end."],
['Isaiah','Isa','45','018','18',"For thus saith the LORD that created the heavens; God himself that formed the earth and made it; he hath established it, he created it not in vain, he formed it to be inhabited: I \emph{am} the LORD; and \emph{there} \emph{is} none else."],
['Isaiah','Isa','45','019','19',"I have not spoken in secret, in a dark place of the earth: I said not unto the seed of Jacob, Seek ye me in vain: I the LORD speak righteousness, I declare things that are right."],
['Isaiah','Isa','45','020','20',"Assemble yourselves and come; draw near together, ye \emph{that} \emph{are} escaped of the nations: they have no knowledge that set up the wood of their graven image, and pray unto a god \emph{that} cannot save."],
['Isaiah','Isa','45','021','21',"Tell ye, and bring \emph{them} near; yea, let them take counsel together: who hath declared this from ancient time? \emph{who} hath told it from that time? \emph{have} not I the LORD? and \emph{there} \emph{is} no God else beside me; a just God and a Saviour; \emph{there} \emph{is} none beside me."],
['Isaiah','Isa','45','022','22',"Look unto me, and be ye saved, all the ends of the earth: for I \emph{am} God, and \emph{there} \emph{is} none else."],
['Isaiah','Isa','45','023','23',"I have sworn by myself, the word is gone out of my mouth \emph{in} righteousness, and shall not return, That unto me every knee shall bow, every tongue shall swear."],
['Isaiah','Isa','45','024','24',"Surely, shall \emph{one} say, in the LORD have I righteousness and strength: \emph{even} to him shall \emph{men} come; and all that are incensed against him shall be ashamed."],
['Isaiah','Isa','45','025','25',"In the LORD shall all the seed of Israel be justified, and shall glory."]
]

Isaiah46 = [
['Isaiah','Isa','46','001','1',"Bel boweth down, Nebo stoopeth, their idols were upon the beasts, and upon the cattle: your carriages \emph{were} heavy loaden; \emph{they} \emph{are} a burden to the weary \emph{beast}."],
['Isaiah','Isa','46','002','2',"They stoop, they bow down together; they could not deliver the burden, but themselves are gone into captivity."],
['Isaiah','Isa','46','003','3',"Hearken unto me, O house of Jacob, and all the remnant of the house of Israel, which are borne \emph{by} \emph{me} from the belly, which are carried from the womb:"],
['Isaiah','Isa','46','004','4',"And \emph{even} to \emph{your} old age I \emph{am} he; and \emph{even} to hoar hairs will I carry \emph{you}: I have made, and I will bear; even I will carry, and will deliver \emph{you}."],
['Isaiah','Isa','46','005','5',"To whom will ye liken me, and make \emph{me} equal, and compare me, that we may be like?"],
['Isaiah','Isa','46','006','6',"They lavish gold out of the bag, and weigh silver in the balance, \emph{and} hire a goldsmith; and he maketh it a god: they fall down, yea, they worship."],
['Isaiah','Isa','46','007','7',"They bear him upon the shoulder, they carry him, and set him in his place, and he standeth; from his place shall he not remove: yea, \emph{one} shall cry unto him, yet can he not answer, nor save him out of his trouble."],
['Isaiah','Isa','46','008','8',"Remember this, and shew yourselves men: bring \emph{it} again to mind, O ye transgressors."],
['Isaiah','Isa','46','009','9',"Remember the former things of old: for I \emph{am} God, and \emph{there} \emph{is} none else; \emph{I} \emph{am} God, and \emph{there} \emph{is} none like me,"],
['Isaiah','Isa','46','010','10',"Declaring the end from the beginning, and from ancient times \emph{the} \emph{things} that are not \emph{yet} done, saying, My counsel shall stand, and I will do all my pleasure:"],
['Isaiah','Isa','46','011','11',"Calling a ravenous bird from the east, the man that executeth my counsel from a far country: yea, I have spoken \emph{it}, I will also bring it to pass; I have purposed \emph{it}, I will also do it."],
['Isaiah','Isa','46','012','12',"Hearken unto me, ye stouthearted, that \emph{are} far from righteousness:"],
['Isaiah','Isa','46','013','13',"I bring near my righteousness; it shall not be far off, and my salvation shall not tarry: and I will place salvation in Zion for Israel my glory."]
]

Isaiah47 = [
['Isaiah','Isa','47','001','1',"Come down, and sit in the dust, O virgin daughter of Babylon, sit on the ground: \emph{there} \emph{is} no throne, O daughter of the Chaldeans: for thou shalt no more be called tender and delicate."],
['Isaiah','Isa','47','002','2',"Take the millstones, and grind meal: uncover thy locks, make bare the leg, uncover the thigh, pass over the rivers."],
['Isaiah','Isa','47','003','3',"Thy nakedness shall be uncovered, yea, thy shame shall be seen: I will take vengeance, and I will not meet \emph{thee} \emph{as} a man."],
['Isaiah','Isa','47','004','4',"\emph{As} \emph{for} our redeemer, the LORD of hosts \emph{is} his name, the Holy One of Israel."],
['Isaiah','Isa','47','005','5',"Sit thou silent, and get thee into darkness, O daughter of the Chaldeans: for thou shalt no more be called, The lady of kingdoms."],
['Isaiah','Isa','47','006','6',"I was wroth with my people, I have polluted mine inheritance, and given them into thine hand: thou didst shew them no mercy; upon the ancient hast thou very heavily laid thy yoke."],
['Isaiah','Isa','47','007','7',"And thou saidst, I shall be a lady for ever: \emph{so} that thou didst not lay these \emph{things} to thy heart, neither didst remember the latter end of it."],
['Isaiah','Isa','47','008','8',"Therefore hear now this, \emph{thou} \emph{that} \emph{art} given to pleasures, that dwellest carelessly, that sayest in thine heart, I \emph{am}, and none else beside me; I shall not sit \emph{as} a widow, neither shall I know the loss of children:"],
['Isaiah','Isa','47','009','9',"But these two \emph{things} shall come to thee in a moment in one day, the loss of children, and widowhood: they shall come upon thee in their perfection for the multitude of thy sorceries, \emph{and} for the great abundance of thine enchantments."],
['Isaiah','Isa','47','010','10',"For thou hast trusted in thy wickedness: thou hast said, None seeth me. Thy wisdom and thy knowledge, it hath perverted thee; and thou hast said in thine heart, I \emph{am}, and none else beside me."],
['Isaiah','Isa','47','011','11',"Therefore shall evil come upon thee; thou shalt not know from whence it riseth: and mischief shall fall upon thee; thou shalt not be able to put it off: and desolation shall come upon thee suddenly, \emph{which} thou shalt not know."],
['Isaiah','Isa','47','012','12',"Stand now with thine enchantments, and with the multitude of thy sorceries, wherein thou hast laboured from thy youth; if so be thou shalt be able to profit, if so be thou mayest prevail."],
['Isaiah','Isa','47','013','13',"Thou art wearied in the multitude of thy counsels. Let now the astrologers, the stargazers, the monthly prognosticators, stand up, and save thee from \emph{these} \emph{things} that shall come upon thee."],
['Isaiah','Isa','47','014','14',"Behold, they shall be as stubble; the fire shall burn them; they shall not deliver themselves from the power of the flame: \emph{there} \emph{shall} not \emph{be} a coal to warm at, \emph{nor} fire to sit before it."],
['Isaiah','Isa','47','015','15',"Thus shall they be unto thee with whom thou hast laboured, \emph{even} thy merchants, from thy youth: they shall wander every one to his quarter; none shall save thee."]
]

Isaiah48 = [
['Isaiah','Isa','48','001','1',"Hear ye this, O house of Jacob, which are called by the name of Israel, and are come forth out of the waters of Judah, which swear by the name of the LORD, and make mention of the God of Israel, \emph{but} not in truth, nor in righteousness."],
['Isaiah','Isa','48','002','2',"For they call themselves of the holy city, and stay themselves upon the God of Israel; The LORD of hosts \emph{is} his name."],
['Isaiah','Isa','48','003','3',"I have declared the former things from the beginning; and they went forth out of my mouth, and I shewed them; I did \emph{them} suddenly, and they came to pass."],
['Isaiah','Isa','48','004','4',"Because I knew that thou \emph{art} obstinate, and thy neck \emph{is} an iron sinew, and thy brow brass;"],
['Isaiah','Isa','48','005','5',"I have even from the beginning declared \emph{it} to thee; before it came to pass I shewed \emph{it} thee: lest thou shouldest say, Mine idol hath done them, and my graven image, and my molten image, hath commanded them."],
['Isaiah','Isa','48','006','6',"Thou hast heard, see all this; and will not ye declare \emph{it}? I have shewed thee new things from this time, even hidden things, and thou didst not know them."],
['Isaiah','Isa','48','007','7',"They are created now, and not from the beginning; even before the day when thou heardest them not; lest thou shouldest say, Behold, I knew them."],
['Isaiah','Isa','48','008','8',"Yea, thou heardest not; yea, thou knewest not; yea, from that time \emph{that} thine ear was not opened: for I knew that thou wouldest deal very treacherously, and wast called a transgressor from the womb."],
['Isaiah','Isa','48','009','9',"For my name's sake will I defer mine anger, and for my praise will I refrain for thee, that I cut thee not off."],
['Isaiah','Isa','48','010','10',"Behold, I have refined thee, but not with silver; I have chosen thee in the furnace of affliction."],
['Isaiah','Isa','48','011','11',"For mine own sake, \emph{even} for mine own sake, will I do \emph{it}: for how should \emph{my} \emph{name} be polluted? and I will not give my glory unto another."],
['Isaiah','Isa','48','012','12',"Hearken unto me, O Jacob and Israel, my called; I \emph{am} he; I \emph{am} the first, I also \emph{am} the last."],
['Isaiah','Isa','48','013','13',"Mine hand also hath laid the foundation of the earth, and my right hand hath spanned the heavens: \emph{when} I call unto them, they stand up together."],
['Isaiah','Isa','48','014','14',"All ye, assemble yourselves, and hear; which among them hath declared these \emph{things}? The LORD hath loved him: he will do his pleasure on Babylon, and his arm \emph{shall} \emph{be} \emph{on} the Chaldeans."],
['Isaiah','Isa','48','015','15',"I, \emph{even} I, have spoken; yea, I have called him: I have brought him, and he shall make his way prosperous."],
['Isaiah','Isa','48','016','16',"Come ye near unto me, hear ye this; I have not spoken in secret from the beginning; from the time that it was, there \emph{am} I: and now the Lord GOD, and his Spirit, hath sent me."],
['Isaiah','Isa','48','017','17',"Thus saith the LORD, thy Redeemer, the Holy One of Israel; I \emph{am} the LORD thy God which teacheth thee to profit, which leadeth thee by the way \emph{that} thou shouldest go."],
['Isaiah','Isa','48','018','18',"O that thou hadst hearkened to my commandments! then had thy peace been as a river, and thy righteousness as the waves of the sea:"],
['Isaiah','Isa','48','019','19',"Thy seed also had been as the sand, and the offspring of thy bowels like the gravel thereof; his name should not have been cut off nor destroyed from before me."],
['Isaiah','Isa','48','020','20',"Go ye forth of Babylon, flee ye from the Chaldeans, with a voice of singing declare ye, tell this, utter it \emph{even} to the end of the earth; say ye, The LORD hath redeemed his servant Jacob."],
['Isaiah','Isa','48','021','21',"And they thirsted not \emph{when} he led them through the deserts: he caused the waters to flow out of the rock for them: he clave the rock also, and the waters gushed out."],
['Isaiah','Isa','48','022','22',"\emph{There} \emph{is} no peace, saith the LORD, unto the wicked."]
]

Isaiah49 = [
['Isaiah','Isa','49','001','1',"Listen, O isles, unto me; and hearken, ye people, from far; The LORD hath called me from the womb; from the bowels of my mother hath he made mention of my name."],
['Isaiah','Isa','49','002','2',"And he hath made my mouth like a sharp sword; in the shadow of his hand hath he hid me, and made me a polished shaft; in his quiver hath he hid me;"],
['Isaiah','Isa','49','003','3',"And said unto me, Thou \emph{art} my servant, O Israel, in whom I will be glorified."],
['Isaiah','Isa','49','004','4',"Then I said, I have laboured in vain, I have spent my strength for nought, and in vain: \emph{yet} surely my judgment \emph{is} with the LORD, and my work with my God."],
['Isaiah','Isa','49','005','5',"And now, saith the LORD that formed me from the womb \emph{to} \emph{be} his servant, to bring Jacob again to him, Though Israel be not gathered, yet shall I be glorious in the eyes of the LORD, and my God shall be my strength."],
['Isaiah','Isa','49','006','6',"And he said, It is a light thing that thou shouldest be my servant to raise up the tribes of Jacob, and to restore the preserved of Israel: I will also give thee for a light to the Gentiles, that thou mayest be my salvation unto the end of the earth."],
['Isaiah','Isa','49','007','7',"Thus saith the LORD, the Redeemer of Israel, \emph{and} his Holy One, to him whom man despiseth, to him whom the nation abhorreth, to a servant of rulers, Kings shall see and arise, princes also shall worship, because of the LORD that is faithful, \emph{and} the Holy One of Israel, and he shall choose thee."],
['Isaiah','Isa','49','008','8',"Thus saith the LORD, In an acceptable time have I heard thee, and in a day of salvation have I helped thee: and I will preserve thee, and give thee for a covenant of the people, to establish the earth, to cause to inherit the desolate heritages;"],
['Isaiah','Isa','49','009','9',"That thou mayest say to the prisoners, Go forth; to them that \emph{are} in darkness, Shew yourselves. They shall feed in the ways, and their pastures \emph{shall} \emph{be} in all high places."],
['Isaiah','Isa','49','010','10',"They shall not hunger nor thirst; neither shall the heat nor sun smite them: for he that hath mercy on them shall lead them, even by the springs of water shall he guide them."],
['Isaiah','Isa','49','011','11',"And I will make all my mountains a way, and my highways shall be exalted."],
['Isaiah','Isa','49','012','12',"Behold, these shall come from far: and, lo, these from the north and from the west; and these from the land of Sinim."],
['Isaiah','Isa','49','013','13',"Sing, O heavens; and be joyful, O earth; and break forth into singing, O mountains: for the LORD hath comforted his people, and will have mercy upon his afflicted."],
['Isaiah','Isa','49','014','14',"But Zion said, The LORD hath forsaken me, and my Lord hath forgotten me."],
['Isaiah','Isa','49','015','15',"Can a woman forget her sucking child, that she should not have compassion on the son of her womb? yea, they may forget, yet will I not forget thee."],
['Isaiah','Isa','49','016','16',"Behold, I have graven thee upon the palms of \emph{my} hands; thy walls \emph{are} continually before me."],
['Isaiah','Isa','49','017','17',"Thy children shall make haste; thy destroyers and they that made thee waste shall go forth of thee."],
['Isaiah','Isa','49','018','18',"Lift up thine eyes round about, and behold: all these gather themselves together, \emph{and} come to thee. \emph{As} I live, saith the LORD, thou shalt surely clothe thee with them all, as with an ornament, and bind them \emph{on} \emph{thee}, as a bride \emph{doeth}."],
['Isaiah','Isa','49','019','19',"For thy waste and thy desolate places, and the land of thy destruction, shall even now be too narrow by reason of the inhabitants, and they that swallowed thee up shall be far away."],
['Isaiah','Isa','49','020','20',"The children which thou shalt have, after thou hast lost the other, shall say again in thine ears, The place \emph{is} too strait for me: give place to me that I may dwell."],
['Isaiah','Isa','49','021','21',"Then shalt thou say in thine heart, Who hath begotten me these, seeing I have lost my children, and am desolate, a captive, and removing to and fro? and who hath brought up these? Behold, I was left alone; these, where \emph{had} they \emph{been}?"],
['Isaiah','Isa','49','022','22',"Thus saith the Lord GOD, Behold, I will lift up mine hand to the Gentiles, and set up my standard to the people: and they shall bring thy sons in \emph{their} arms, and thy daughters shall be carried upon \emph{their} shoulders."],
['Isaiah','Isa','49','023','23',"And kings shall be thy nursing fathers, and their queens thy nursing mothers: they shall bow down to thee with \emph{their} face toward the earth, and lick up the dust of thy feet; and thou shalt know that I \emph{am} the LORD: for they shall not be ashamed that wait for me."],
['Isaiah','Isa','49','024','24',"Shall the prey be taken from the mighty, or the lawful captive delivered?"],
['Isaiah','Isa','49','025','25',"But thus saith the LORD, Even the captives of the mighty shall be taken away, and the prey of the terrible shall be delivered: for I will contend with him that contendeth with thee, and I will save thy children."],
['Isaiah','Isa','49','026','26',"And I will feed them that oppress thee with their own flesh; and they shall be drunken with their own blood, as with sweet wine: and all flesh shall know that I the LORD \emph{am} thy Saviour and thy Redeemer, the mighty One of Jacob."]
]

Isaiah50 = [
['Isaiah','Isa','50','001','1',"Thus saith the LORD, Where \emph{is} the bill of your mother's divorcement, whom I have put away? or which of my creditors \emph{is} \emph{it} to whom I have sold you? Behold, for your iniquities have ye sold yourselves, and for your transgressions is your mother put away."],
['Isaiah','Isa','50','002','2',"Wherefore, when I came, \emph{was} \emph{there} no man? when I called, \emph{was} \emph{there} none to answer? Is my hand shortened at all, that it cannot redeem? or have I no power to deliver? behold, at my rebuke I dry up the sea, I make the rivers a wilderness: their fish stinketh, because \emph{there} \emph{is} no water, and dieth for thirst."],
['Isaiah','Isa','50','003','3',"I clothe the heavens with blackness, and I make sackcloth their covering."],
['Isaiah','Isa','50','004','4',"The Lord GOD hath given me the tongue of the learned, that I should know how to speak a word in season to \emph{him} \emph{that} \emph{is} weary: he wakeneth morning by morning, he wakeneth mine ear to hear as the learned."],
['Isaiah','Isa','50','005','5',"The Lord GOD hath opened mine ear, and I was not rebellious, neither turned away back."],
['Isaiah','Isa','50','006','6',"I gave my back to the smiters, and my cheeks to them that plucked off the hair: I hid not my face from shame and spitting."],
['Isaiah','Isa','50','007','7',"For the Lord GOD will help me; therefore shall I not be confounded: therefore have I set my face like a flint, and I know that I shall not be ashamed."],
['Isaiah','Isa','50','008','8',"\emph{He} \emph{is} near that justifieth me; who will contend with me? let us stand together: who \emph{is} mine adversary? let him come near to me."],
['Isaiah','Isa','50','009','9',"Behold, the Lord GOD will help me; who \emph{is} he \emph{that} shall condemn me? lo, they all shall wax old as a garment; the moth shall eat them up."],
['Isaiah','Isa','50','010','10',"Who \emph{is} among you that feareth the LORD, that obeyeth the voice of his servant, that walketh \emph{in} darkness, and hath no light? let him trust in the name of the LORD, and stay upon his God."],
['Isaiah','Isa','50','011','11',"Behold, all ye that kindle a fire, that compass \emph{yourselves} about with sparks: walk in the light of your fire, and in the sparks \emph{that} ye have kindled. This shall ye have of mine hand; ye shall lie down in sorrow."]
]

Isaiah51 = [
['Isaiah','Isa','51','001','1',"Hearken to me, ye that follow after righteousness, ye that seek the LORD: look unto the rock \emph{whence} ye are hewn, and to the hole of the pit \emph{whence} ye are digged."],
['Isaiah','Isa','51','002','2',"Look unto Abraham your father, and unto Sarah \emph{that} bare you: for I called him alone, and blessed him, and increased him."],
['Isaiah','Isa','51','003','3',"For the LORD shall comfort Zion: he will comfort all her waste places; and he will make her wilderness like Eden, and her desert like the garden of the LORD; joy and gladness shall be found therein, thanksgiving, and the voice of melody."],
['Isaiah','Isa','51','004','4',"Hearken unto me, my people; and give ear unto me, O my nation: for a law shall proceed from me, and I will make my judgment to rest for a light of the people."],
['Isaiah','Isa','51','005','5',"My righteousness \emph{is} near; my salvation is gone forth, and mine arms shall judge the people; the isles shall wait upon me, and on mine arm shall they trust."],
['Isaiah','Isa','51','006','6',"Lift up your eyes to the heavens, and look upon the earth beneath: for the heavens shall vanish away like smoke, and the earth shall wax old like a garment, and they that dwell therein shall die in like manner: but my salvation shall be for ever, and my righteousness shall not be abolished."],
['Isaiah','Isa','51','007','7',"Hearken unto me, ye that know righteousness, the people in whose heart \emph{is} my law; fear ye not the reproach of men, neither be ye afraid of their revilings."],
['Isaiah','Isa','51','008','8',"For the moth shall eat them up like a garment, and the worm shall eat them like wool: but my righteousness shall be for ever, and my salvation from generation to generation."],
['Isaiah','Isa','51','009','9',"Awake, awake, put on strength, O arm of the LORD; awake, as in the ancient days, in the generations of old. \emph{Art} thou not it that hath cut Rahab, \emph{and} wounded the dragon?"],
['Isaiah','Isa','51','010','10',"\emph{Art} thou not it which hath dried the sea, the waters of the great deep; that hath made the depths of the sea a way for the ransomed to pass over?"],
['Isaiah','Isa','51','011','11',"Therefore the redeemed of the LORD shall return, and come with singing unto Zion; and everlasting joy \emph{shall} \emph{be} upon their head: they shall obtain gladness and joy; \emph{and} sorrow and mourning shall flee away."],
['Isaiah','Isa','51','012','12',"I, \emph{even} I, \emph{am} he that comforteth you: who \emph{art} thou, that thou shouldest be afraid of a man \emph{that} shall die, and of the son of man \emph{which} shall be made \emph{as} grass;"],
['Isaiah','Isa','51','013','13',"And forgettest the LORD thy maker, that hath stretched forth the heavens, and laid the foundations of the earth; and hast feared continually every day because of the fury of the oppressor, as if he were ready to destroy? and where \emph{is} the fury of the oppressor?"],
['Isaiah','Isa','51','014','14',"The captive exile hasteneth that he may be loosed, and that he should not die in the pit, nor that his bread should fail."],
['Isaiah','Isa','51','015','15',"But I \emph{am} the LORD thy God, that divided the sea, whose waves roared: The LORD of hosts \emph{is} his name."],
['Isaiah','Isa','51','016','16',"And I have put my words in thy mouth, and I have covered thee in the shadow of mine hand, that I may plant the heavens, and lay the foundations of the earth, and say unto Zion, Thou \emph{art} my people."],
['Isaiah','Isa','51','017','17',"Awake, awake, stand up, O Jerusalem, which hast drunk at the hand of the LORD the cup of his fury; thou hast drunken the dregs of the cup of trembling, \emph{and} wrung \emph{them} out."],
['Isaiah','Isa','51','018','18',"\emph{There} \emph{is} none to guide her among all the sons \emph{whom} she hath brought forth; neither \emph{is} \emph{there} \emph{any} that taketh her by the hand of all the sons \emph{that} she hath brought up."],
['Isaiah','Isa','51','019','19',"These two \emph{things} are come unto thee; who shall be sorry for thee? desolation, and destruction, and the famine, and the sword: by whom shall I comfort thee?"],
['Isaiah','Isa','51','020','20',"Thy sons have fainted, they lie at the head of all the streets, as a wild bull in a net: they are full of the fury of the LORD, the rebuke of thy God."],
['Isaiah','Isa','51','021','21',"Therefore hear now this, thou afflicted, and drunken, but not with wine:"],
['Isaiah','Isa','51','022','22',"Thus saith thy Lord the LORD, and thy God \emph{that} pleadeth the cause of his people, Behold, I have taken out of thine hand the cup of trembling, \emph{even} the dregs of the cup of my fury; thou shalt no more drink it again:"],
['Isaiah','Isa','51','023','23',"But I will put it into the hand of them that afflict thee; which have said to thy soul, Bow down, that we may go over: and thou hast laid thy body as the ground, and as the street, to them that went over."]
]

Isaiah52 = [
['Isaiah','Isa','52','001','1',"Awake, awake; put on thy strength, O Zion; put on thy beautiful garments, O Jerusalem, the holy city: for henceforth there shall no more come into thee the uncircumcised and the unclean."],
['Isaiah','Isa','52','002','2',"Shake thyself from the dust; arise, \emph{and} sit down, O Jerusalem: loose thyself from the bands of thy neck, O captive daughter of Zion."],
['Isaiah','Isa','52','003','3',"For thus saith the LORD, Ye have sold yourselves for nought; and ye shall be redeemed without money."],
['Isaiah','Isa','52','004','4',"For thus saith the Lord GOD, My people went down aforetime into Egypt to sojourn there; and the Assyrian oppressed them without cause."],
['Isaiah','Isa','52','005','5',"Now therefore, what have I here, saith the LORD, that my people is taken away for nought? they that rule over them make them to howl, saith the LORD; and my name continually every day \emph{is} blasphemed."],
['Isaiah','Isa','52','006','6',"Therefore my people shall know my name: therefore \emph{they} \emph{shall} \emph{know} in that day that I \emph{am} he that doth speak: behold, \emph{it} \emph{is} I."],
['Isaiah','Isa','52','007','7',"How beautiful upon the mountains are the feet of him that bringeth good tidings, that publisheth peace; that bringeth good tidings of good, that publisheth salvation; that saith unto Zion, Thy God reigneth!"],
['Isaiah','Isa','52','008','8',"Thy watchmen shall lift up the voice; with the voice together shall they sing: for they shall see eye to eye, when the LORD shall bring again Zion."],
['Isaiah','Isa','52','009','9',"Break forth into joy, sing together, ye waste places of Jerusalem: for the LORD hath comforted his people, he hath redeemed Jerusalem."],
['Isaiah','Isa','52','010','10',"The LORD hath made bare his holy arm in the eyes of all the nations; and all the ends of the earth shall see the salvation of our God."],
['Isaiah','Isa','52','011','11',"Depart ye, depart ye, go ye out from thence, touch no unclean \emph{thing}; go ye out of the midst of her; be ye clean, that bear the vessels of the LORD."],
['Isaiah','Isa','52','012','12',"For ye shall not go out with haste, nor go by flight: for the LORD will go before you; and the God of Israel \emph{will} \emph{be} your rereward."],
['Isaiah','Isa','52','013','13',"Behold, my servant shall deal prudently, he shall be exalted and extolled, and be very high."],
['Isaiah','Isa','52','014','14',"As many were astonied at thee; his visage was so marred more than any man, and his form more than the sons of men:"],
['Isaiah','Isa','52','015','15',"So shall he sprinkle many nations; the kings shall shut their mouths at him: for \emph{that} which had not been told them shall they see; and \emph{that} which they had not heard shall they consider."]
]

Isaiah53 = [
['Isaiah','Isa','53','001','1',"Who hath believed our report? and to whom is the arm of the LORD revealed?"],
['Isaiah','Isa','53','002','2',"For he shall grow up before him as a tender plant, and as a root out of a dry ground: he hath no form nor comeliness; and when we shall see him, \emph{there} \emph{is} no beauty that we should desire him."],
['Isaiah','Isa','53','003','3',"He is despised and rejected of men; a man of sorrows, and acquainted with grief: and we hid as it were \emph{our} faces from him; he was despised, and we esteemed him not."],
['Isaiah','Isa','53','004','4',"Surely he hath borne our griefs, and carried our sorrows: yet we did esteem him stricken, smitten of God, and afflicted."],
['Isaiah','Isa','53','005','5',"But he \emph{was} wounded for our transgressions, \emph{he} \emph{was} bruised for our iniquities: the chastisement of our peace \emph{was} upon him; and with his stripes we are healed."],
['Isaiah','Isa','53','006','6',"All we like sheep have gone astray; we have turned every one to his own way; and the LORD hath laid on him the iniquity of us all."],
['Isaiah','Isa','53','007','7',"He was oppressed, and he was afflicted, yet he opened not his mouth: he is brought as a lamb to the slaughter, and as a sheep before her shearers is dumb, so he openeth not his mouth."],
['Isaiah','Isa','53','008','8',"He was taken from prison and from judgment: and who shall declare his generation? for he was cut off out of the land of the living: for the transgression of my people was he stricken."],
['Isaiah','Isa','53','009','9',"And he made his grave with the wicked, and with the rich in his death; because he had done no violence, neither \emph{was} \emph{any} deceit in his mouth."],
['Isaiah','Isa','53','010','10',"Yet it pleased the LORD to bruise him; he hath put \emph{him} to grief: when thou shalt make his soul an offering for sin, he shall see \emph{his} seed, he shall prolong \emph{his} days, and the pleasure of the LORD shall prosper in his hand."],
['Isaiah','Isa','53','011','11',"He shall see of the travail of his soul, \emph{and} shall be satisfied: by his knowledge shall my righteous servant justify many; for he shall bear their iniquities."],
['Isaiah','Isa','53','012','12',"Therefore will I divide him \emph{a} \emph{portion} with the great, and he shall divide the spoil with the strong; because he hath poured out his soul unto death: and he was numbered with the transgressors; and he bare the sin of many, and made intercession for the transgressors."]
]

Isaiah54 = [
['Isaiah','Isa','54','001','1',"Sing, O barren, thou \emph{that} didst not bear; break forth into singing, and cry aloud, thou \emph{that} didst not travail with child: for more \emph{are} the children of the desolate than the children of the married wife, saith the LORD."],
['Isaiah','Isa','54','002','2',"Enlarge the place of thy tent, and let them stretch forth the curtains of thine habitations: spare not, lengthen thy cords, and strengthen thy stakes;"],
['Isaiah','Isa','54','003','3',"For thou shalt break forth on the right hand and on the left; and thy seed shall inherit the Gentiles, and make the desolate cities to be inhabited."],
['Isaiah','Isa','54','004','4',"Fear not; for thou shalt not be ashamed: neither be thou confounded; for thou shalt not be put to shame: for thou shalt forget the shame of thy youth, and shalt not remember the reproach of thy widowhood any more."],
['Isaiah','Isa','54','005','5',"For thy Maker \emph{is} thine husband; the LORD of hosts \emph{is} his name; and thy Redeemer the Holy One of Israel; The God of the whole earth shall he be called."],
['Isaiah','Isa','54','006','6',"For the LORD hath called thee as a woman forsaken and grieved in spirit, and a wife of youth, when thou wast refused, saith thy God."],
['Isaiah','Isa','54','007','7',"For a small moment have I forsaken thee; but with great mercies will I gather thee."],
['Isaiah','Isa','54','008','8',"In a little wrath I hid my face from thee for a moment; but with everlasting kindness will I have mercy on thee, saith the LORD thy Redeemer."],
['Isaiah','Isa','54','009','9',"For this \emph{is} \emph{as} the waters of Noah unto me: for \emph{as} I have sworn that the waters of Noah should no more go over the earth; so have I sworn that I would not be wroth with thee, nor rebuke thee."],
['Isaiah','Isa','54','010','10',"For the mountains shall depart, and the hills be removed; but my kindness shall not depart from thee, neither shall the covenant of my peace be removed, saith the LORD that hath mercy on thee."],
['Isaiah','Isa','54','011','11',"O thou afflicted, tossed with tempest, \emph{and} not comforted, behold, I will lay thy stones with fair colours, and lay thy foundations with sapphires."],
['Isaiah','Isa','54','012','12',"And I will make thy windows of agates, and thy gates of carbuncles, and all thy borders of pleasant stones."],
['Isaiah','Isa','54','013','13',"And all thy children \emph{shall} \emph{be} taught of the LORD; and great \emph{shall} \emph{be} the peace of thy children."],
['Isaiah','Isa','54','014','14',"In righteousness shalt thou be established: thou shalt be far from oppression; for thou shalt not fear: and from terror; for it shall not come near thee."],
['Isaiah','Isa','54','015','15',"Behold, they shall surely gather together, \emph{but} not by me: whosoever shall gather together against thee shall fall for thy sake."],
['Isaiah','Isa','54','016','16',"Behold, I have created the smith that bloweth the coals in the fire, and that bringeth forth an instrument for his work; and I have created the waster to destroy."],
['Isaiah','Isa','54','017','17',"No weapon that is formed against thee shall prosper; and every tongue \emph{that} shall rise against thee in judgment thou shalt condemn. This \emph{is} the heritage of the servants of the LORD, and their righteousness \emph{is} of me, saith the LORD."]
]

Isaiah55 = [
['Isaiah','Isa','55','001','1',"Ho, every one that thirsteth, come ye to the waters, and he that hath no money; come ye, buy, and eat; yea, come, buy wine and milk without money and without price."],
['Isaiah','Isa','55','002','2',"Wherefore do ye spend money for \emph{that} \emph{which} \emph{is} not bread? and your labour for \emph{that} \emph{which} satisfieth not? hearken diligently unto me, and eat ye \emph{that} \emph{which} \emph{is} good, and let your soul delight itself in fatness."],
['Isaiah','Isa','55','003','3',"Incline your ear, and come unto me: hear, and your soul shall live; and I will make an everlasting covenant with you, \emph{even} the sure mercies of David."],
['Isaiah','Isa','55','004','4',"Behold, I have given him \emph{for} a witness to the people, a leader and commander to the people."],
['Isaiah','Isa','55','005','5',"Behold, thou shalt call a nation \emph{that} thou knowest not, and nations \emph{that} knew not thee shall run unto thee because of the LORD thy God, and for the Holy One of Israel; for he hath glorified thee."],
['Isaiah','Isa','55','006','6',"Seek ye the LORD while he may be found, call ye upon him while he is near:"],
['Isaiah','Isa','55','007','7',"Let the wicked forsake his way, and the unrighteous man his thoughts: and let him return unto the LORD, and he will have mercy upon him; and to our God, for he will abundantly pardon."],
['Isaiah','Isa','55','008','8',"For my thoughts \emph{are} not your thoughts, neither \emph{are} your ways my ways, saith the LORD."],
['Isaiah','Isa','55','009','9',"For \emph{as} the heavens are higher than the earth, so are my ways higher than your ways, and my thoughts than your thoughts."],
['Isaiah','Isa','55','010','10',"For as the rain cometh down, and the snow from heaven, and returneth not thither, but watereth the earth, and maketh it bring forth and bud, that it may give seed to the sower, and bread to the eater:"],
['Isaiah','Isa','55','011','11',"So shall my word be that goeth forth out of my mouth: it shall not return unto me void, but it shall accomplish that which I please, and it shall prosper \emph{in} \emph{the} \emph{thing} whereto I sent it."],
['Isaiah','Isa','55','012','12',"For ye shall go out with joy, and be led forth with peace: the mountains and the hills shall break forth before you into singing, and all the trees of the field shall clap \emph{their} hands."],
['Isaiah','Isa','55','013','13',"Instead of the thorn shall come up the fir tree, and instead of the brier shall come up the myrtle tree: and it shall be to the LORD for a name, for an everlasting sign \emph{that} shall not be cut off."]
]

Isaiah56 = [
['Isaiah','Isa','56','001','1',"Thus saith the LORD, Keep ye judgment, and do justice: for my salvation \emph{is} near to come, and my righteousness to be revealed."],
['Isaiah','Isa','56','002','2',"Blessed \emph{is} the man \emph{that} doeth this, and the son of man \emph{that} layeth hold on it; that keepeth the sabbath from polluting it, and keepeth his hand from doing any evil."],
['Isaiah','Isa','56','003','3',"Neither let the son of the stranger, that hath joined himself to the LORD, speak, saying, The LORD hath utterly separated me from his people: neither let the eunuch say, Behold, I \emph{am} a dry tree."],
['Isaiah','Isa','56','004','4',"For thus saith the LORD unto the eunuchs that keep my sabbaths, and choose \emph{the} \emph{things} that please me, and take hold of my covenant;"],
['Isaiah','Isa','56','005','5',"Even unto them will I give in mine house and within my walls a place and a name better than of sons and of daughters: I will give them an everlasting name, that shall not be cut off."],
['Isaiah','Isa','56','006','6',"Also the sons of the stranger, that join themselves to the LORD, to serve him, and to love the name of the LORD, to be his servants, every one that keepeth the sabbath from polluting it, and taketh hold of my covenant;"],
['Isaiah','Isa','56','007','7',"Even them will I bring to my holy mountain, and make them joyful in my house of prayer: their burnt offerings and their sacrifices \emph{shall} \emph{be} accepted upon mine altar; for mine house shall be called an house of prayer for all people."],
['Isaiah','Isa','56','008','8',"The Lord GOD which gathereth the outcasts of Israel saith, Yet will I gather \emph{others} to him, beside those that are gathered unto him."],
['Isaiah','Isa','56','009','9',"All ye beasts of the field, come to devour, \emph{yea}, all ye beasts in the forest."],
['Isaiah','Isa','56','010','10',"His watchmen \emph{are} blind: they are all ignorant, they \emph{are} all dumb dogs, they cannot bark; sleeping, lying down, loving to slumber."],
['Isaiah','Isa','56','011','11',"Yea, \emph{they} \emph{are} greedy dogs \emph{which} can never have enough, and they \emph{are} shepherds \emph{that} cannot understand: they all look to their own way, every one for his gain, from his quarter."],
['Isaiah','Isa','56','012','12',"Come ye, \emph{say} \emph{they}, I will fetch wine, and we will fill ourselves with strong drink; and to morrow shall be as this day, \emph{and} much more abundant."]
]

Isaiah57 = [
['Isaiah','Isa','57','001','1',"The righteous perisheth, and no man layeth \emph{it} to heart: and merciful men \emph{are} taken away, none considering that the righteous is taken away from the evil \emph{to} \emph{come}."],
['Isaiah','Isa','57','002','2',"He shall enter into peace: they shall rest in their beds, \emph{each} \emph{one} walking \emph{in} his uprightness."],
['Isaiah','Isa','57','003','3',"But draw near hither, ye sons of the sorceress, the seed of the adulterer and the whore."],
['Isaiah','Isa','57','004','4',"Against whom do ye sport yourselves? against whom make ye a wide mouth, \emph{and} draw out the tongue? \emph{are} ye not children of transgression, a seed of falsehood,"],
['Isaiah','Isa','57','005','5',"Enflaming yourselves with idols under every green tree, slaying the children in the valleys under the clifts of the rocks?"],
['Isaiah','Isa','57','006','6',"Among the smooth \emph{stones} of the stream \emph{is} thy portion; they, they \emph{are} thy lot: even to them hast thou poured a drink offering, thou hast offered a meat offering. Should I receive comfort in these?"],
['Isaiah','Isa','57','007','7',"Upon a lofty and high mountain hast thou set thy bed: even thither wentest thou up to offer sacrifice."],
['Isaiah','Isa','57','008','8',"Behind the doors also and the posts hast thou set up thy remembrance: for thou hast discovered \emph{thyself} \emph{to} \emph{another} than me, and art gone up; thou hast enlarged thy bed, and made thee \emph{a} \emph{covenant} with them; thou lovedst their bed where thou sawest \emph{it}."],
['Isaiah','Isa','57','009','9',"And thou wentest to the king with ointment, and didst increase thy perfumes, and didst send thy messengers far off, and didst debase \emph{thyself} \emph{even} unto hell."],
['Isaiah','Isa','57','010','10',"Thou art wearied in the greatness of thy way; \emph{yet} saidst thou not, There is no hope: thou hast found the life of thine hand; therefore thou wast not grieved."],
['Isaiah','Isa','57','011','11',"And of whom hast thou been afraid or feared, that thou hast lied, and hast not remembered me, nor laid \emph{it} to thy heart? have not I held my peace even of old, and thou fearest me not?"],
['Isaiah','Isa','57','012','12',"I will declare thy righteousness, and thy works; for they shall not profit thee."],
['Isaiah','Isa','57','013','13',"When thou criest, let thy companies deliver thee; but the wind shall carry them all away; vanity shall take \emph{them}: but he that putteth his trust in me shall possess the land, and shall inherit my holy mountain;"],
['Isaiah','Isa','57','014','14',"And shall say, Cast ye up, cast ye up, prepare the way, take up the stumblingblock out of the way of my people."],
['Isaiah','Isa','57','015','15',"For thus saith the high and lofty One that inhabiteth eternity, whose name \emph{is} Holy; I dwell in the high and holy \emph{place}, with him also \emph{that} \emph{is} of a contrite and humble spirit, to revive the spirit of the humble, and to revive the heart of the contrite ones."],
['Isaiah','Isa','57','016','16',"For I will not contend for ever, neither will I be always wroth: for the spirit should fail before me, and the souls \emph{which} I have made."],
['Isaiah','Isa','57','017','17',"For the iniquity of his covetousness was I wroth, and smote him: I hid me, and was wroth, and he went on frowardly in the way of his heart."],
['Isaiah','Isa','57','018','18',"I have seen his ways, and will heal him: I will lead him also, and restore comforts unto him and to his mourners."],
['Isaiah','Isa','57','019','19',"I create the fruit of the lips; Peace, peace to \emph{him} \emph{that} \emph{is} far off, and to \emph{him} \emph{that} \emph{is} near, saith the LORD; and I will heal him."],
['Isaiah','Isa','57','020','20',"But the wicked \emph{are} like the troubled sea, when it cannot rest, whose waters cast up mire and dirt."],
['Isaiah','Isa','57','021','21',"\emph{There} \emph{is} no peace, saith my God, to the wicked."]
]

Isaiah58 = [
['Isaiah','Isa','58','001','1',"Cry aloud, spare not, lift up thy voice like a trumpet, and shew my people their transgression, and the house of Jacob their sins."],
['Isaiah','Isa','58','002','2',"Yet they seek me daily, and delight to know my ways, as a nation that did righteousness, and forsook not the ordinance of their God: they ask of me the ordinances of justice; they take delight in approaching to God."],
['Isaiah','Isa','58','003','3',"Wherefore have we fasted, \emph{say} \emph{they}, and thou seest not? \emph{wherefore} have we afflicted our soul, and thou takest no knowledge? Behold, in the day of your fast ye find pleasure, and exact all your labours."],
['Isaiah','Isa','58','004','4',"Behold, ye fast for strife and debate, and to smite with the fist of wickedness: ye shall not fast as \emph{ye} \emph{do} \emph{this} day, to make your voice to be heard on high."],
['Isaiah','Isa','58','005','5',"Is it such a fast that I have chosen? a day for a man to afflict his soul? \emph{is} \emph{it} to bow down his head as a bulrush, and to spread sackcloth and ashes \emph{under} \emph{him}? wilt thou call this a fast, and an acceptable day to the LORD?"],
['Isaiah','Isa','58','006','6',"\emph{Is} not this the fast that I have chosen? to loose the bands of wickedness, to undo the heavy burdens, and to let the oppressed go free, and that ye break every yoke?"],
['Isaiah','Isa','58','007','7',"\emph{Is} \emph{it} not to deal thy bread to the hungry, and that thou bring the poor that are cast out to thy house? when thou seest the naked, that thou cover him; and that thou hide not thyself from thine own flesh?"],
['Isaiah','Isa','58','008','8',"Then shall thy light break forth as the morning, and thine health shall spring forth speedily: and thy righteousness shall go before thee; the glory of the LORD shall be thy rereward."],
['Isaiah','Isa','58','009','9',"Then shalt thou call, and the LORD shall answer; thou shalt cry, and he shall say, Here I \emph{am}. If thou take away from the midst of thee the yoke, the putting forth of the finger, and speaking vanity;"],
['Isaiah','Isa','58','010','10',"And \emph{if} thou draw out thy soul to the hungry, and satisfy the afflicted soul; then shall thy light rise in obscurity, and thy darkness \emph{be} as the noonday:"],
['Isaiah','Isa','58','011','11',"And the LORD shall guide thee continually, and satisfy thy soul in drought, and make fat thy bones: and thou shalt be like a watered garden, and like a spring of water, whose waters fail not."],
['Isaiah','Isa','58','012','12',"And \emph{they} \emph{that} \emph{shall} \emph{be} of thee shall build the old waste places: thou shalt raise up the foundations of many generations; and thou shalt be called, The repairer of the breach, The restorer of paths to dwell in."],
['Isaiah','Isa','58','013','13',"If thou turn away thy foot from the sabbath, \emph{from} doing thy pleasure on my holy day; and call the sabbath a delight, the holy of the LORD, honourable; and shalt honour him, not doing thine own ways, nor finding thine own pleasure, nor speaking \emph{thine} \emph{own} words:"],
['Isaiah','Isa','58','014','14',"Then shalt thou delight thyself in the LORD; and I will cause thee to ride upon the high places of the earth, and feed thee with the heritage of Jacob thy father: for the mouth of the LORD hath spoken \emph{it}."]
]

Isaiah59 = [
['Isaiah','Isa','59','001','1',"Behold, the LORD'S hand is not shortened, that it cannot save; neither his ear heavy, that it cannot hear:"],
['Isaiah','Isa','59','002','2',"But your iniquities have separated between you and your God, and your sins have hid \emph{his} face from you, that he will not hear."],
['Isaiah','Isa','59','003','3',"For your hands are defiled with blood, and your fingers with iniquity; your lips have spoken lies, your tongue hath muttered perverseness."],
['Isaiah','Isa','59','004','4',"None calleth for justice, nor \emph{any} pleadeth for truth: they trust in vanity, and speak lies; they conceive mischief, and bring forth iniquity."],
['Isaiah','Isa','59','005','5',"They hatch cockatrice' eggs, and weave the spider's web: he that eateth of their eggs dieth, and that which is crushed breaketh out into a viper."],
['Isaiah','Isa','59','006','6',"Their webs shall not become garments, neither shall they cover themselves with their works: their works \emph{are} works of iniquity, and the act of violence \emph{is} in their hands."],
['Isaiah','Isa','59','007','7',"Their feet run to evil, and they make haste to shed innocent blood: their thoughts \emph{are} thoughts of iniquity; wasting and destruction \emph{are} in their paths."],
['Isaiah','Isa','59','008','8',"The way of peace they know not; and \emph{there} \emph{is} no judgment in their goings: they have made them crooked paths: whosoever goeth therein shall not know peace."],
['Isaiah','Isa','59','009','9',"Therefore is judgment far from us, neither doth justice overtake us: we wait for light, but behold obscurity; for brightness, \emph{but} we walk in darkness."],
['Isaiah','Isa','59','010','10',"We grope for the wall like the blind, and we grope as if \emph{we} \emph{had} no eyes: we stumble at noonday as in the night; \emph{we} \emph{are} in desolate places as dead \emph{men}."],
['Isaiah','Isa','59','011','11',"We roar all like bears, and mourn sore like doves: we look for judgment, but \emph{there} \emph{is} none; for salvation, \emph{but} it is far off from us."],
['Isaiah','Isa','59','012','12',"For our transgressions are multiplied before thee, and our sins testify against us: for our transgressions \emph{are} with us; and \emph{as} \emph{for} our iniquities, we know them;"],
['Isaiah','Isa','59','013','13',"In transgressing and lying against the LORD, and departing away from our God, speaking oppression and revolt, conceiving and uttering from the heart words of falsehood."],
['Isaiah','Isa','59','014','14',"And judgment is turned away backward, and justice standeth afar off: for truth is fallen in the street, and equity cannot enter."],
['Isaiah','Isa','59','015','15',"Yea, truth faileth; and he \emph{that} departeth from evil maketh himself a prey: and the LORD saw \emph{it}, and it displeased him that \emph{there} \emph{was} no judgment."],
['Isaiah','Isa','59','016','16',"And he saw that \emph{there} \emph{was} no man, and wondered that \emph{there} \emph{was} no intercessor: therefore his arm brought salvation unto him; and his righteousness, it sustained him."],
['Isaiah','Isa','59','017','17',"For he put on righteousness as a breastplate, and an helmet of salvation upon his head; and he put on the garments of vengeance \emph{for} clothing, and was clad with zeal as a cloke."],
['Isaiah','Isa','59','018','18',"According to \emph{their} deeds, accordingly he will repay, fury to his adversaries, recompence to his enemies; to the islands he will repay recompence."],
['Isaiah','Isa','59','019','19',"So shall they fear the name of the LORD from the west, and his glory from the rising of the sun. When the enemy shall come in like a flood, the Spirit of the LORD shall lift up a standard against him."],
['Isaiah','Isa','59','020','20',"And the Redeemer shall come to Zion, and unto them that turn from transgression in Jacob, saith the LORD."],
['Isaiah','Isa','59','021','21',"As for me, this \emph{is} my covenant with them, saith the LORD; My spirit that \emph{is} upon thee, and my words which I have put in thy mouth, shall not depart out of thy mouth, nor out of the mouth of thy seed, nor out of the mouth of thy seed's seed, saith the LORD, from henceforth and for ever."]
]

Isaiah60 = [
['Isaiah','Isa','60','001','1',"Arise, shine; for thy light is come, and the glory of the LORD is risen upon thee."],
['Isaiah','Isa','60','002','2',"For, behold, the darkness shall cover the earth, and gross darkness the people: but the LORD shall arise upon thee, and his glory shall be seen upon thee."],
['Isaiah','Isa','60','003','3',"And the Gentiles shall come to thy light, and kings to the brightness of thy rising."],
['Isaiah','Isa','60','004','4',"Lift up thine eyes round about, and see: all they gather themselves together, they come to thee: thy sons shall come from far, and thy daughters shall be nursed at \emph{thy} side."],
['Isaiah','Isa','60','005','5',"Then thou shalt see, and flow together, and thine heart shall fear, and be enlarged; because the abundance of the sea shall be converted unto thee, the forces of the Gentiles shall come unto thee."],
['Isaiah','Isa','60','006','6',"The multitude of camels shall cover thee, the dromedaries of Midian and Ephah; all they from Sheba shall come: they shall bring gold and incense; and they shall shew forth the praises of the LORD."],
['Isaiah','Isa','60','007','7',"All the flocks of Kedar shall be gathered together unto thee, the rams of Nebaioth shall minister unto thee: they shall come up with acceptance on mine altar, and I will glorify the house of my glory."],
['Isaiah','Isa','60','008','8',"Who \emph{are} these \emph{that} fly as a cloud, and as the doves to their windows?"],
['Isaiah','Isa','60','009','9',"Surely the isles shall wait for me, and the ships of Tarshish first, to bring thy sons from far, their silver and their gold with them, unto the name of the LORD thy God, and to the Holy One of Israel, because he hath glorified thee."],
['Isaiah','Isa','60','010','10',"And the sons of strangers shall build up thy walls, and their kings shall minister unto thee: for in my wrath I smote thee, but in my favour have I had mercy on thee."],
['Isaiah','Isa','60','011','11',"Therefore thy gates shall be open continually; they shall not be shut day nor night; that \emph{men} may bring unto thee the forces of the Gentiles, and \emph{that} their kings \emph{may} \emph{be} brought."],
['Isaiah','Isa','60','012','12',"For the nation and kingdom that will not serve thee shall perish; yea, \emph{those} nations shall be utterly wasted."],
['Isaiah','Isa','60','013','13',"The glory of Lebanon shall come unto thee, the fir tree, the pine tree, and the box together, to beautify the place of my sanctuary; and I will make the place of my feet glorious."],
['Isaiah','Isa','60','014','14',"The sons also of them that afflicted thee shall come bending unto thee; and all they that despised thee shall bow themselves down at the soles of thy feet; and they shall call thee, The city of the LORD, The Zion of the Holy One of Israel."],
['Isaiah','Isa','60','015','15',"Whereas thou hast been forsaken and hated, so that no man went through \emph{thee}, I will make thee an eternal excellency, a joy of many generations."],
['Isaiah','Isa','60','016','16',"Thou shalt also suck the milk of the Gentiles, and shalt suck the breast of kings: and thou shalt know that I the LORD \emph{am} thy Saviour and thy Redeemer, the mighty One of Jacob."],
['Isaiah','Isa','60','017','17',"For brass I will bring gold, and for iron I will bring silver, and for wood brass, and for stones iron: I will also make thy officers peace, and thine exactors righteousness."],
['Isaiah','Isa','60','018','18',"Violence shall no more be heard in thy land, wasting nor destruction within thy borders; but thou shalt call thy walls Salvation, and thy gates Praise."],
['Isaiah','Isa','60','019','19',"The sun shall be no more thy light by day; neither for brightness shall the moon give light unto thee: but the LORD shall be unto thee an everlasting light, and thy God thy glory."],
['Isaiah','Isa','60','020','20',"Thy sun shall no more go down; neither shall thy moon withdraw itself: for the LORD shall be thine everlasting light, and the days of thy mourning shall be ended."],
['Isaiah','Isa','60','021','21',"Thy people also \emph{shall} \emph{be} all righteous: they shall inherit the land for ever, the branch of my planting, the work of my hands, that I may be glorified."],
['Isaiah','Isa','60','022','22',"A little one shall become a thousand, and a small one a strong nation: I the LORD will hasten it in his time."]
]

Isaiah61 = [
['Isaiah','Isa','61','001','1',"The Spirit of the Lord GOD \emph{is} upon me; because the LORD hath anointed me to preach good tidings unto the meek; he hath sent me to bind up the brokenhearted, to proclaim liberty to the captives, and the opening of the prison to \emph{them} \emph{that} \emph{are} bound;"],
['Isaiah','Isa','61','002','2',"To proclaim the acceptable year of the LORD, and the day of vengeance of our God; to comfort all that mourn;"],
['Isaiah','Isa','61','003','3',"To appoint unto them that mourn in Zion, to give unto them beauty for ashes, the oil of joy for mourning, the garment of praise for the spirit of heaviness; that they might be called trees of righteousness, the planting of the LORD, that he might be glorified."],
['Isaiah','Isa','61','004','4',"And they shall build the old wastes, they shall raise up the former desolations, and they shall repair the waste cities, the desolations of many generations."],
['Isaiah','Isa','61','005','5',"And strangers shall stand and feed your flocks, and the sons of the alien \emph{shall} \emph{be} your plowmen and your vinedressers."],
['Isaiah','Isa','61','006','6',"But ye shall be named the Priests of the LORD: \emph{men} shall call you the Ministers of our God: ye shall eat the riches of the Gentiles, and in their glory shall ye boast yourselves."],
['Isaiah','Isa','61','007','7',"For your shame \emph{ye} \emph{shall} \emph{have} double; and \emph{for} confusion they shall rejoice in their portion: therefore in their land they shall possess the double: everlasting joy shall be unto them."],
['Isaiah','Isa','61','008','8',"For I the LORD love judgment, I hate robbery for burnt offering; and I will direct their work in truth, and I will make an everlasting covenant with them."],
['Isaiah','Isa','61','009','9',"And their seed shall be known among the Gentiles, and their offspring among the people: all that see them shall acknowledge them, that they \emph{are} the seed \emph{which} the LORD hath blessed."],
['Isaiah','Isa','61','010','10',"I will greatly rejoice in the LORD, my soul shall be joyful in my God; for he hath clothed me with the garments of salvation, he hath covered me with the robe of righteousness, as a bridegroom decketh \emph{himself} with ornaments, and as a bride adorneth \emph{herself} with her jewels."],
['Isaiah','Isa','61','011','11',"For as the earth bringeth forth her bud, and as the garden causeth the things that are sown in it to spring forth; so the Lord GOD will cause righteousness and praise to spring forth before all the nations."]
]

Isaiah62 = [
['Isaiah','Isa','62','001','1',"For Zion's sake will I not hold my peace, and for Jerusalem's sake I will not rest, until the righteousness thereof go forth as brightness, and the salvation thereof as a lamp \emph{that} burneth."],
['Isaiah','Isa','62','002','2',"And the Gentiles shall see thy righteousness, and all kings thy glory: and thou shalt be called by a new name, which the mouth of the LORD shall name."],
['Isaiah','Isa','62','003','3',"Thou shalt also be a crown of glory in the hand of the LORD, and a royal diadem in the hand of thy God."],
['Isaiah','Isa','62','004','4',"Thou shalt no more be termed Forsaken; neither shall thy land any more be termed Desolate: but thou shalt be called Hephzi-bah, and thy land Beulah: for the LORD delighteth in thee, and thy land shall be married."],
['Isaiah','Isa','62','005','5',"For \emph{as} a young man marrieth a virgin, \emph{so} shall thy sons marry thee: and \emph{as} the bridegroom rejoiceth over the bride, \emph{so} shall thy God rejoice over thee."],
['Isaiah','Isa','62','006','6',"I have set watchmen upon thy walls, O Jerusalem, \emph{which} shall never hold their peace day nor night: ye that make mention of the LORD, keep not silence,"],
['Isaiah','Isa','62','007','7',"And give him no rest, till he establish, and till he make Jerusalem a praise in the earth."],
['Isaiah','Isa','62','008','8',"The LORD hath sworn by his right hand, and by the arm of his strength, Surely I will no more give thy corn \emph{to} \emph{be} meat for thine enemies; and the sons of the stranger shall not drink thy wine, for the which thou hast laboured:"],
['Isaiah','Isa','62','009','9',"But they that have gathered it shall eat it, and praise the LORD; and they that have brought it together shall drink it in the courts of my holiness."],
['Isaiah','Isa','62','010','10',"Go through, go through the gates; prepare ye the way of the people; cast up, cast up the highway; gather out the stones; lift up a standard for the people."],
['Isaiah','Isa','62','011','11',"Behold, the LORD hath proclaimed unto the end of the world, Say ye to the daughter of Zion, Behold, thy salvation cometh; behold, his reward \emph{is} with him, and his work before him."],
['Isaiah','Isa','62','012','12',"And they shall call them, The holy people, The redeemed of the LORD: and thou shalt be called, Sought out, A city not forsaken."]
]

Isaiah63 = [
['Isaiah','Isa','63','001','1',"Who \emph{is} this that cometh from Edom, with dyed garments from Bozrah? this \emph{that} \emph{is} glorious in his apparel, travelling in the greatness of his strength? I that speak in righteousness, mighty to save."],
['Isaiah','Isa','63','002','2',"Wherefore \emph{art} \emph{thou} red in thine apparel, and thy garments like him that treadeth in the winefat?"],
['Isaiah','Isa','63','003','3',"I have trodden the winepress alone; and of the people \emph{there} \emph{was} none with me: for I will tread them in mine anger, and trample them in my fury; and their blood shall be sprinkled upon my garments, and I will stain all my raiment."],
['Isaiah','Isa','63','004','4',"For the day of vengeance \emph{is} in mine heart, and the year of my redeemed is come."],
['Isaiah','Isa','63','005','5',"And I looked, and \emph{there} \emph{was} none to help; and I wondered that \emph{there} \emph{was} none to uphold: therefore mine own arm brought salvation unto me; and my fury, it upheld me."],
['Isaiah','Isa','63','006','6',"And I will tread down the people in mine anger, and make them drunk in my fury, and I will bring down their strength to the earth."],
['Isaiah','Isa','63','007','7',"I will mention the lovingkindnesses of the LORD, \emph{and} the praises of the LORD, according to all that the LORD hath bestowed on us, and the great goodness toward the house of Israel, which he hath bestowed on them according to his mercies, and according to the multitude of his lovingkindnesses."],
['Isaiah','Isa','63','008','8',"For he said, Surely they \emph{are} my people, children \emph{that} will not lie: so he was their Saviour."],
['Isaiah','Isa','63','009','9',"In all their affliction he was afflicted, and the angel of his presence saved them: in his love and in his pity he redeemed them; and he bare them, and carried them all the days of old."],
['Isaiah','Isa','63','010','10',"But they rebelled, and vexed his holy Spirit: therefore he was turned to be their enemy, \emph{and} he fought against them."],
['Isaiah','Isa','63','011','11',"Then he remembered the days of old, Moses, \emph{and} his people, \emph{saying}, Where \emph{is} he that brought them up out of the sea with the shepherd of his flock? where \emph{is} he that put his holy Spirit within him?"],
['Isaiah','Isa','63','012','12',"That led \emph{them} by the right hand of Moses with his glorious arm, dividing the water before them, to make himself an everlasting name?"],
['Isaiah','Isa','63','013','13',"That led them through the deep, as an horse in the wilderness, \emph{that} they should not stumble?"],
['Isaiah','Isa','63','014','14',"As a beast goeth down into the valley, the Spirit of the LORD caused him to rest: so didst thou lead thy people, to make thyself a glorious name."],
['Isaiah','Isa','63','015','15',"Look down from heaven, and behold from the habitation of thy holiness and of thy glory: where \emph{is} thy zeal and thy strength, the sounding of thy bowels and of thy mercies toward me? are they restrained?"],
['Isaiah','Isa','63','016','16',"Doubtless thou \emph{art} our father, though Abraham be ignorant of us, and Israel acknowledge us not: thou, O LORD, \emph{art} our father, our redeemer; thy name \emph{is} from everlasting."],
['Isaiah','Isa','63','017','17',"O LORD, why hast thou made us to err from thy ways, \emph{and} hardened our heart from thy fear? Return for thy servants' sake, the tribes of thine inheritance."],
['Isaiah','Isa','63','018','18',"The people of thy holiness have possessed \emph{it} but a little while: our adversaries have trodden down thy sanctuary."],
['Isaiah','Isa','63','019','19',"We are \emph{thine}: thou never barest rule over them; they were not called by thy name."]
]

Isaiah64 = [
['Isaiah','Isa','64','001','1',"Oh that thou wouldest rend the heavens, that thou wouldest come down, that the mountains might flow down at thy presence,"],
['Isaiah','Isa','64','002','2',"As \emph{when} the melting fire burneth, the fire causeth the waters to boil, to make thy name known to thine adversaries, \emph{that} the nations may tremble at thy presence!"],
['Isaiah','Isa','64','003','3',"When thou didst terrible things \emph{which} we looked not for, thou camest down, the mountains flowed down at thy presence."],
['Isaiah','Isa','64','004','4',"For since the beginning of the world \emph{men} have not heard, nor perceived by the ear, neither hath the eye seen, O God, beside thee, \emph{what} he hath prepared for him that waiteth for him."],
['Isaiah','Isa','64','005','5',"Thou meetest him that rejoiceth and worketh righteousness, \emph{those} \emph{that} remember thee in thy ways: behold, thou art wroth; for we have sinned: in those is continuance, and we shall be saved."],
['Isaiah','Isa','64','006','6',"But we are all as an unclean \emph{thing}, and all our righteousnesses \emph{are} as filthy rags; and we all do fade as a leaf; and our iniquities, like the wind, have taken us away."],
['Isaiah','Isa','64','007','7',"And \emph{there} \emph{is} none that calleth upon thy name, that stirreth up himself to take hold of thee: for thou hast hid thy face from us, and hast consumed us, because of our iniquities."],
['Isaiah','Isa','64','008','8',"But now, O LORD, thou \emph{art} our father; we \emph{are} the clay, and thou our potter; and we all \emph{are} the work of thy hand."],
['Isaiah','Isa','64','009','9',"Be not wroth very sore, O LORD, neither remember iniquity for ever: behold, see, we beseech thee, we \emph{are} all thy people."],
['Isaiah','Isa','64','010','10',"Thy holy cities are a wilderness, Zion is a wilderness, Jerusalem a desolation."],
['Isaiah','Isa','64','011','11',"Our holy and our beautiful house, where our fathers praised thee, is burned up with fire: and all our pleasant things are laid waste."],
['Isaiah','Isa','64','012','12',"Wilt thou refrain thyself for these \emph{things}, O LORD? wilt thou hold thy peace, and afflict us very sore?"]
]

Isaiah65 = [
['Isaiah','Isa','65','001','1',"I am sought of \emph{them} \emph{that} asked not \emph{for} \emph{me}; I am found of \emph{them} \emph{that} sought me not: I said, Behold me, behold me, unto a nation \emph{that} was not called by my name."],
['Isaiah','Isa','65','002','2',"I have spread out my hands all the day unto a rebellious people, which walketh in a way \emph{that} \emph{was} not good, after their own thoughts;"],
['Isaiah','Isa','65','003','3',"A people that provoketh me to anger continually to my face; that sacrificeth in gardens, and burneth incense upon altars of brick;"],
['Isaiah','Isa','65','004','4',"Which remain among the graves, and lodge in the monuments, which eat swine's flesh, and broth of abominable \emph{things} \emph{is} \emph{in} their vessels;"],
['Isaiah','Isa','65','005','5',"Which say, Stand by thyself, come not near to me; for I am holier than thou. These \emph{are} a smoke in my nose, a fire that burneth all the day."],
['Isaiah','Isa','65','006','6',"Behold, \emph{it} \emph{is} written before me: I will not keep silence, but will recompense, even recompense into their bosom,"],
['Isaiah','Isa','65','007','7',"Your iniquities, and the iniquities of your fathers together, saith the LORD, which have burned incense upon the mountains, and blasphemed me upon the hills: therefore will I measure their former work into their bosom."],
['Isaiah','Isa','65','008','8',"Thus saith the LORD, As the new wine is found in the cluster, and \emph{one} saith, Destroy it not; for a blessing \emph{is} in it: so will I do for my servants' sakes, that I may not destroy them all."],
['Isaiah','Isa','65','009','9',"And I will bring forth a seed out of Jacob, and out of Judah an inheritor of my mountains: and mine elect shall inherit it, and my servants shall dwell there."],
['Isaiah','Isa','65','010','10',"And Sharon shall be a fold of flocks, and the valley of Achor a place for the herds to lie down in, for my people that have sought me."],
['Isaiah','Isa','65','011','11',"But ye \emph{are} they that forsake the LORD, that forget my holy mountain, that prepare a table for that troop, and that furnish the drink offering unto that number."],
['Isaiah','Isa','65','012','12',"Therefore will I number you to the sword, and ye shall all bow down to the slaughter: because when I called, ye did not answer; when I spake, ye did not hear; but did evil before mine eyes, and did choose \emph{that} wherein I delighted not."],
['Isaiah','Isa','65','013','13',"Therefore thus saith the Lord GOD, Behold, my servants shall eat, but ye shall be hungry: behold, my servants shall drink, but ye shall be thirsty: behold, my servants shall rejoice, but ye shall be ashamed:"],
['Isaiah','Isa','65','014','14',"Behold, my servants shall sing for joy of heart, but ye shall cry for sorrow of heart, and shall howl for vexation of spirit."],
['Isaiah','Isa','65','015','15',"And ye shall leave your name for a curse unto my chosen: for the Lord GOD shall slay thee, and call his servants by another name:"],
['Isaiah','Isa','65','016','16',"That he who blesseth himself in the earth shall bless himself in the God of truth; and he that sweareth in the earth shall swear by the God of truth; because the former troubles are forgotten, and because they are hid from mine eyes."],
['Isaiah','Isa','65','017','17',"For, behold, I create new heavens and a new earth: and the former shall not be remembered, nor come into mind."],
['Isaiah','Isa','65','018','18',"But be ye glad and rejoice for ever \emph{in} \emph{that} which I create: for, behold, I create Jerusalem a rejoicing, and her people a joy."],
['Isaiah','Isa','65','019','19',"And I will rejoice in Jerusalem, and joy in my people: and the voice of weeping shall be no more heard in her, nor the voice of crying."],
['Isaiah','Isa','65','020','20',"There shall be no more thence an infant of days, nor an old man that hath not filled his days: for the child shall die an hundred years old; but the sinner \emph{being} an hundred years old shall be accursed."],
['Isaiah','Isa','65','021','21',"And they shall build houses, and inhabit \emph{them}; and they shall plant vineyards, and eat the fruit of them."],
['Isaiah','Isa','65','022','22',"They shall not build, and another inhabit; they shall not plant, and another eat: for as the days of a tree \emph{are} the days of my people, and mine elect shall long enjoy the work of their hands."],
['Isaiah','Isa','65','023','23',"They shall not labour in vain, nor bring forth for trouble; for they \emph{are} the seed of the blessed of the LORD, and their offspring with them."],
['Isaiah','Isa','65','024','24',"And it shall come to pass, that before they call, I will answer; and while they are yet speaking, I will hear."],
['Isaiah','Isa','65','025','25',"The wolf and the lamb shall feed together, and the lion shall eat straw like the bullock: and dust \emph{shall} \emph{be} the serpent's meat. They shall not hurt nor destroy in all my holy mountain, saith the LORD."]
]

Isaiah66 = [
['Isaiah','Isa','66','001','1',"Thus saith the LORD, The heaven \emph{is} my throne, and the earth \emph{is} my footstool: where \emph{is} the house that ye build unto me? and where \emph{is} the place of my rest?"],
['Isaiah','Isa','66','002','2',"For all those \emph{things} hath mine hand made, and all those \emph{things} have been, saith the LORD: but to this \emph{man} will I look, \emph{even} to \emph{him} \emph{that} \emph{is} poor and of a contrite spirit, and trembleth at my word."],
['Isaiah','Isa','66','003','3',"He that killeth an ox \emph{is} \emph{as} \emph{if} he slew a man; he that sacrificeth a lamb, \emph{as} \emph{if} he cut off a dog's neck; he that offereth an oblation, \emph{as} \emph{if} \emph{he} \emph{offered} swine's blood; he that burneth incense, \emph{as} \emph{if} he blessed an idol. Yea, they have chosen their own ways, and their soul delighteth in their abominations."],
['Isaiah','Isa','66','004','4',"I also will choose their delusions, and will bring their fears upon them; because when I called, none did answer; when I spake, they did not hear: but they did evil before mine eyes, and chose \emph{that} in which I delighted not."],
['Isaiah','Isa','66','005','5',"Hear the word of the LORD, ye that tremble at his word; Your brethren that hated you, that cast you out for my name's sake, said, Let the LORD be glorified: but he shall appear to your joy, and they shall be ashamed."],
['Isaiah','Isa','66','006','6',"A voice of noise from the city, a voice from the temple, a voice of the LORD that rendereth recompence to his enemies."],
['Isaiah','Isa','66','007','7',"Before she travailed, she brought forth; before her pain came, she was delivered of a man child."],
['Isaiah','Isa','66','008','8',"Who hath heard such a thing? who hath seen such things? Shall the earth be made to bring forth in one day? \emph{or} shall a nation be born at once? for as soon as Zion travailed, she brought forth her children."],
['Isaiah','Isa','66','009','9',"Shall I bring to the birth, and not cause to bring forth? saith the LORD: shall I cause to bring forth, and shut \emph{the} \emph{womb}? saith thy God."],
['Isaiah','Isa','66','010','10',"Rejoice ye with Jerusalem, and be glad with her, all ye that love her: rejoice for joy with her, all ye that mourn for her:"],
['Isaiah','Isa','66','011','11',"That ye may suck, and be satisfied with the breasts of her consolations; that ye may milk out, and be delighted with the abundance of her glory."],
['Isaiah','Isa','66','012','12',"For thus saith the LORD, Behold, I will extend peace to her like a river, and the glory of the Gentiles like a flowing stream: then shall ye suck, ye shall be borne upon \emph{her} sides, and be dandled upon \emph{her} knees."],
['Isaiah','Isa','66','013','13',"As one whom his mother comforteth, so will I comfort you; and ye shall be comforted in Jerusalem."],
['Isaiah','Isa','66','014','14',"And when ye see \emph{this}, your heart shall rejoice, and your bones shall flourish like an herb: and the hand of the LORD shall be known toward his servants, and \emph{his} indignation toward his enemies."],
['Isaiah','Isa','66','015','15',"For, behold, the LORD will come with fire, and with his chariots like a whirlwind, to render his anger with fury, and his rebuke with flames of fire."],
['Isaiah','Isa','66','016','16',"For by fire and by his sword will the LORD plead with all flesh: and the slain of the LORD shall be many."],
['Isaiah','Isa','66','017','17',"They that sanctify themselves, and purify themselves in the gardens behind one \emph{tree} in the midst, eating swine's flesh, and the abomination, and the mouse, shall be consumed together, saith the LORD."],
['Isaiah','Isa','66','018','18',"For I \emph{know} their works and their thoughts: it shall come, that I will gather all nations and tongues; and they shall come, and see my glory."],
['Isaiah','Isa','66','019','19',"And I will set a sign among them, and I will send those that escape of them unto the nations, \emph{to} Tarshish, Pul, and Lud, that draw the bow, \emph{to} Tubal, and Javan, \emph{to} the isles afar off, that have not heard my fame, neither have seen my glory; and they shall declare my glory among the Gentiles."],
['Isaiah','Isa','66','020','20',"And they shall bring all your brethren \emph{for} an offering unto the LORD out of all nations upon horses, and in chariots, and in litters, and upon mules, and upon swift beasts, to my holy mountain Jerusalem, saith the LORD, as the children of Israel bring an offering in a clean vessel into the house of the LORD."],
['Isaiah','Isa','66','021','21',"And I will also take of them for priests \emph{and} for Levites, saith the LORD."],
['Isaiah','Isa','66','022','22',"For as the new heavens and the new earth, which I will make, shall remain before me, saith the LORD, so shall your seed and your name remain."],
['Isaiah','Isa','66','023','23',"And it shall come to pass, \emph{that} from one new moon to another, and from one sabbath to another, shall all flesh come to worship before me, saith the LORD."],
['Isaiah','Isa','66','024','24',"And they shall go forth, and look upon the carcases of the men that have transgressed against me: for their worm shall not die, neither shall their fire be quenched; and they shall be an abhorring unto all flesh."]
]

def main():
    ProcessVerses('Isaiah57',Isaiah57)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{The righteous perisheth, and no man layeth \emph{it} to heart: and merciful men \emph{are} taken away, none considering that the righteous is taken away from the evil \emph{to} \emph{come}.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{He shall enter into peace: they shall rest in their beds, \emph{each} \emph{one} walking \emph{in} his uprightness.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{But draw near hither, ye sons of the sorceress, the seed of the adulterer and the whore.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{Against whom do ye sport yourselves? against whom make ye a wide mouth, \emph{and} draw out the tongue? \emph{are} ye not children of transgression, a seed of falsehood,}
[5] \textcolor[cmyk]{0.99998,1,0,0}{Enflaming yourselves with idols under every green tree, slaying the child

companies & 1 \\ \hline
considering & 1 \\ \hline
contend & 1 \\ \hline
contrite & 2 \\ \hline
covetousness & 1 \\ \hline
create & 1 \\ \hline
criest & 1 \\ \hline
debase & 1 \\ \hline
declare & 1 \\ \hline
deliver & 1 \\ \hline
didst & 3 \\ \hline
dirt & 1 \\ \hline
discovered & 1 \\ \hline
do & 1 \\ \hline
doors & 1 \\ \hline
draw & 2 \\ \hline
drink & 1 \\ \hline
dwell & 1 \\ \hline
enlarged & 1 \\ \hline
enter & 1 \\ \hline
eternity & 1 \\ \hline
even & 3 \\ \hline
ever & 1 \\ \hline
every & 1 \\ \hline
evil & 1 \\ \hline
fail & 1 \\ \hline
falsehood & 1 \\ \hline
far & 2 \\ \hline
feared & 1 \\ \hline
fearest & 1 \\ \hline
for & 4 \\ \hline
found & 1 \\ \hline
from & 1 \\ \hline
frowardly & 1 \\ \hline
fruit & 1 \\ \hline
gone & 1 \\ \hline
greatness & 1 \\ \hline
green & 1 \\ \hline
grieved & 1 \\ \hline
hand & 1 \\ \hline
hast & 10 \\ \hline
have & 3 \\ \hline
he & 2 \\ \hline
heal & 2 \\ \hline
heart & 4 \\ \hline
held & 1 \\ \hline
hell & 1 \\ \hline
hid & 1 \\ \hline
high & 3